# <font size="6">**Disco**Stream v1.1 ![visitors](https://visitor-badge.glitch.me/badge?page_id=DiscoStream&left_color=blue&right_color=orange) [![GitHub](https://img.shields.io/badge/github-%23121011.svg?style=for-the-badge&logo=github&logoColor=white)](https://github.com/WASasquatch/disco-diffusion-portrait-playground/)</font>

<hr>

#### This Colab is maintained by **WAS**asquatch <font size="2">(<font color="7e87f6">Discord</font>: WAS\#0263) 

## **Other Tools:**

- [Pinegraph | You are a creator](https://pinegraph.com/) - Stable and Disco Diffusion under one sleek web app
- [Stability.AI Easy Diffusion](https://rebrand.ly/easy-diffusion) - WAS's Stable Diffusion notebook based on NOP's original notebook
- [WAS's Disco Diffusion v5.6-8 - Portrait Playground](https://rebrand.ly/portrait-playground) - A fork of Disco Diffusion that additionally, DiscoStream is forked from.
- [OpenAI Guided Diffusion - Model Training / Fine-Tuning](https://rebrand.ly/guided-diffusion) - A implementation of OpenAI's Guided Diffusion in Google Colabs
- [Noodle Soup Prompts](https://rebrand.ly/noodle-soup-prompts) - Terminology Database for prompt exploration
- [Portrait Generator `init_image` Prep](https://rebrand.ly/dd-image-prep) - Prep portraits other other `init_images` as approximations for better DD improvisation. 

<br>

-----

<br> 

### DiscoStream is based on Disco Diffusion
- http://discodiffusion.com/
- https://github.com/alembics/disco-diffusion
- [LICENSES](https://github.com/WASasquatch/discostream/blob/main/LICENSE)
- For issues, join the [Disco Diffusion Discord](https://discord.gg/msEZBy4HxA) and contact **WAS**asquatch <font size="2">(<font color="7e87f6">Discord</font>: WAS\#0263)</font>. 
- Please do not contact somnai or gandamu regarding DiscoStream. DiscoStream is a fork of Disco Diffusion.

<br>

## **DiscoStream Changelog:**
- 8/12/22) Ported WAS's Disco Diffusion 5.6-8 - Portrait Playground
- 8/12/22) Patched cutn scheduling
- 8/12/22) Go BIG V2 Optional Installation
- 8/14/22) Patched NSP installation for local use
- 8/14/22) Fixed AdaBins installation falling after disco_xform_utils
- 8/14/22) Better installation logic and ordering
- 8/14/22) NSP runtime prompts option added
- 8/15/22) NSP runtime go big support
- 8/17/22) Patched cutn scheduling
- 8/17/22) Patched go big scaling without maximize on
- 8/19/22) Patch settings loading
- 8/21/22) Added [DD Image Prep](https://rebrand.ly/dd-image-prep) to DiscoStream
- 8/21/22) Added list functionality to `init_images`. You can now specify a init image for each `n_batch` pass. 
- 8/21/22) Added author and copyright to metadata
- 8/22/22) Added Architecture Diffusion 1.5m Model by jerostephan
- 8/30/22) Patched animation print function

In [ ]:
#@title 1.1 Check GPU Status

# Import future print
from __future__ import print_function
try:
    import __builtin__
except ImportError:
    import builtins as __builtin__

# Emoticon fun!
import subprocess
try:
    import emoji
except ImportError:
     multipip_res = subprocess.run(['pip', '-q', 'install', 'emoji'], stdout=subprocess.PIPE).stdout.decode('utf-8')
finally:
    import emoji

# Override Print Function
def print(message, *args, **kwargs):
    if 'defaultprint' in kwargs:
        kwargs.pop('defaultprint')
        return __builtin__.print(message, *args, **kwargs)
    else:
        return __builtin__.print(emoji.emojize(message), *args, **kwargs)

import subprocess, sys

simple_nvidia_smi_display = False #@param {type:"boolean"}
nvidiasmi_simple = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')
if nvidiasmi_simple.find('NVIDIA-SMI has failed') != -1:
    import warnings
    warnings.filterwarnings("ignore")
    sys.exit('NVIDIA-SMI has failed! No GPU is allocated to this runtime!')
if simple_nvidia_smi_display:
    print(nvidiasmi_simple, defaultprint=True)
else:
    nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_output, defaultprint=True)
    print(nvidiasmi_ecc_note, defaultprint=True)

gpu_name = nvidiasmi_simple.split(':')[1].split('(')[0].strip()

gpu_warning = None
if 'Tesla T4' or 'Tesla P100' in gpu_name:
  gpu_warning = ":warning: \33[33m\u001b[4mWARNING:\33[0m \033[1mYou are using a limited VRAM GPU (\33[33m"+gpu_name+"\33[0m\033[1m). \n         Be cautious when using resolutions above 720p, or heavier models such ViTL14+, RN50x4+, etc.\33[0m"
  print(gpu_warning)

In [ ]:
#@title 1.2 Setup DiscoStream Environment

# Import future print
from __future__ import print_function
try:
    import __builtin__
except ImportError:
    import builtins as __builtin__

# Emoticon fun!
try:
    import subprocess
    import emoji
except ImportError:
     multipip_res = subprocess.run(['pip', '-q', 'install', 'emoji'], stdout=subprocess.PIPE).stdout.decode('utf-8')
finally:
    import emoji

# Override Print Function
def print(message, *args, **kwargs):
    if 'defaultprint' in kwargs:
        kwargs.pop('defaultprint')
        return __builtin__.print(message, *args, **kwargs)
    else:
        return __builtin__.print(emoji.emojize(message), *args, **kwargs)

LINE_UP = '\033[1A'
LINE_CLEAR = '\033[K'

import subprocess, os, sys, ipykernel

appname = 'DiscoStream'

def gitclone(url, targetdir=None):
    if targetdir:
        res = subprocess.run(['git', 'clone', '--quiet', url, targetdir], stdout=subprocess.PIPE).stdout.decode('utf-8')
    else:
        res = subprocess.run(['git', 'clone', '--quiet', url], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res, defaultprint=True)

def pipi(modulestr):
    res = subprocess.run(['pip', 'install', modulestr + ' --quiet'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res, defaultprint=True)

def pipie(modulestr):
    res = subprocess.run(['git', 'install', '-e', modulestr + ' --quiet'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res, defaultprint=True)

def wget(url, outputdir):
    res = subprocess.run(['wget', '-q', '--show-progress', url, '-P', f'{outputdir}'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res, defaultprint=True)


try:
    from google.colab import drive
    print("Google Colab detected.")
    is_colab = True
    #@markdown If you connect your Google Drive, you can save the final image of each run on your drive.
    google_drive = True #@param {type:"boolean"}
    #@markdown Click here if you'd like to save the diffusion model checkpoint file to (and/or load from) your Google Drive:
    save_models_to_google_drive = True #@param {type:"boolean"}
except:
    is_colab = False
    google_drive = False
    save_models_to_google_drive = False
    print("Google Colab not detected.")

if is_colab:
    if google_drive is True:
        print("\n:hourglass_not_done: Mounting Google Drive...")
        drive.mount('/content/drive')
        root_path = '/content/drive/MyDrive/AI/Disco_Diffusion'
    else:
        root_path = '/content'
else:
    root_path = os.getcwd()

import os
  
def createPath(filepath):
    os.makedirs(filepath, exist_ok=True)

initDirPath = f'{root_path}/init_images'
createPath(initDirPath)
outDirPath = f'{root_path}/images_out'
createPath(outDirPath)

if is_colab:
    if google_drive and not save_models_to_google_drive or not google_drive:
        model_path = '/content/models'
        createPath(model_path)
    if google_drive and save_models_to_google_drive:
        model_path = f'{root_path}/models'
        createPath(model_path)
else:
    model_path = f'{root_path}/models'
    createPath(model_path)

# libraries = f'{root_path}/libraries'
# createPath(libraries)

# INSTALL DEPENDENCIES

print(f"\n--[ Installing {appname} Dependencies ]--\n")

import pathlib, shutil, sys

# There are some reports that with a T4 or V100 on Colab, downgrading to a previous version of PyTorch may be necessary.
# .. but there are also reports that downgrading breaks them!  If you're facing issues, you may want to try uncommenting and running this code.
# nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
# cards_requiring_downgrade = ["Tesla T4", "V100"]
# if is_colab:
#     if any(cardstr in nvidiasmi_output for cardstr in cards_requiring_downgrade):
#         print("Downgrading pytorch. This can take a couple minutes ...")
#         downgrade_pytorch_result = subprocess.run(['pip', 'install', 'torch==1.10.2', 'torchvision==0.11.3', '-q'], stdout=subprocess.PIPE).stdout.decode('utf-8')
#         print("pytorch downgraded.")

#@markdown Check this if you want to use CPU
useCPU = False #@param {type:"boolean"}

if not is_colab:
    # If running locally, there's a good chance your env will need this in order to not crash upon np.matmul() or similar operations.
    os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

PROJECT_DIR = os.path.abspath(os.getcwd())
USE_ADABINS = True

if is_colab:
    if not google_drive:
        root_path = f'/content'
        model_path = '/content/models' 
else:
    root_path = os.getcwd()
    model_path = f'{root_path}/models'

try:
    print(":hourglass_not_done: Downgrading ipwidgets...\n")
    ipwidgets_res = subprocess.run(['pip', 'install', 'ipywidgets==7.7.2'], stdout=subprocess.PIPE).stdout.decode('utf-8')
except subprocess.CalledProcessError as e:
    print("Unable to downgrade ipwidgets!\n", e.output)
finally:
    if 'installed ipywidgets-7.7.2' in ipwidgets_res:
        from google.colab import output
        output.enable_custom_widget_manager()
        print(":check_mark_button: \33[32mInstalled ipwidgets 7 successfully!\33[0m\n")

try:
    print(":hourglass_not_done: Installing OS Dependencies...")
    multipip_res = subprocess.run(['pip', '-q', 'install', 'lpips', 'datetime', 'timm', 'ftfy', 'einops', 'pytorch-lightning', 'omegaconf'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(multipip_res, defaultprint=True)
except BaseException as e:
    raise ("\r\rUnable to download OS dependenceis!\n").format(e)
finally:
    print("\r\r:check_mark_button: \33[32mInstalled OS dependencies successfully.\33[0m\n")

if is_colab:
    print(":hourglass_not_done: Installing ImageMagick...")
    imagemagick_res = subprocess.run(['apt', 'install -qq', 'imagemagick'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(imagemagick_res, defaultprint=True)

try:
    print(":hourglass_not_done: Installing CLIP...")
    from CLIP import clip
except:
    if not os.path.exists("CLIP"):
        gitclone("https://github.com/openai/CLIP")
    sys.path.append(f'{PROJECT_DIR}/CLIP')
finally:
    from CLIP import clip
    if 'CLIP' in sys.modules:
        print(":check_mark_button: \33[32mInstalled CLIP successfully.\33[0m\n")

try:
    print(":hourglass_not_done: Installing Open CLIP...")
    import open_clip
except:
    if not os.path.exists("open_clip/src"):
        gitclone("https://github.com/mlfoundations/open_clip.git")
    sys.path.append(f'{PROJECT_DIR}/open_clip/src')
finally:
    import open_clip
    if 'open_clip' in sys.modules:
        print(":check_mark_button: \33[32mInstalled Open CLIP successfully.\33[0m\n")

try:
    print(":hourglass_not_done: Installing Guided Diffusion...")
    from guided_diffusion.script_util import create_model_and_diffusion
except:
    if not os.path.exists("guided-diffusion"):
        gitclone("https://github.com/kostarion/guided-diffusion")
    sys.path.append(f'{PROJECT_DIR}/guided-diffusion')
finally:
    from guided_diffusion.script_util import create_model_and_diffusion
    if 'guided_diffusion.script_util' in sys.modules:
        print(":check_mark_button: \33[32mInstalled Guided Diffusion successfully.\33[0m\n")

try:
    print(":hourglass_not_done: Installing resize_right...")
    from resize_right import resize
except:
    if not os.path.exists("ResizeRight"):
        gitclone("https://github.com/assafshocher/ResizeRight.git")
    sys.path.append(f'{PROJECT_DIR}/ResizeRight')
finally:
    from resize_right import resize
    if 'resize_right' in sys.modules:
        print(":check_mark_button: \33[32mInstalled resize_right successfully.\33[0m\n")

try:
    print(":hourglass_not_done: Installing PyTorch3d Lite...")
    import py3d_tools as p3dT
except:
    if not os.path.exists('pytorch3d-lite'):
        gitclone("https://github.com/MSFTserver/pytorch3d-lite.git")
    sys.path.append(f'{PROJECT_DIR}/pytorch3d-lite')
finally:
    import py3d_tools as p3dT
    if 'py3d_tools' in sys.modules:
        print(":check_mark_button: \33[32mInstalled PyTorch3d Lite successfully.\33[0m\n")

try:
    print(":hourglass_not_done: Installing Midas Dependenceis...")
    from midas.dpt_depth import DPTDepthModel
except:
    if not os.path.exists('MiDaS'):
        gitclone("https://github.com/isl-org/MiDaS.git")
    if not os.path.exists('MiDaS/midas_utils.py'):
        shutil.move('MiDaS/utils.py', 'MiDaS/midas_utils.py')
    if not os.path.exists(f'{model_path}/dpt_large-midas-2f21e586.pt'):
        wget("https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-midas-2f21e586.pt", model_path)
finally:
    sys.path.append(f'{PROJECT_DIR}/MiDaS')
    from midas.dpt_depth import DPTDepthModel
    if 'midas.dpt_depth' in sys.modules:
        print(":check_mark_button: \33[32mMidas Dependenceis installed.\33[0m\n")

# AdaBins stuff
if USE_ADABINS:
    try:
        print(":hourglass_not_done: Installing AdaBins...")
        from infer import InferenceHelper
    except:
        if not os.path.exists("AdaBins"):
            gitclone("https://github.com/shariqfarooq123/AdaBins.git")
        if not os.path.exists(f'{PROJECT_DIR}/pretrained/AdaBins_nyu.pt'):
            createPath(f'{PROJECT_DIR}/pretrained')
            wget("https://cloudflare-ipfs.com/ipfs/Qmd2mMnDLWePKmgfS8m6ntAg4nhV5VkUyAydYBp8cWWeB7/AdaBins_nyu.pt", f'{PROJECT_DIR}/pretrained')
        sys.path.append(f'{PROJECT_DIR}/AdaBins')
    finally:
        from infer import InferenceHelper
        if 'infer' in sys.modules:
            print(":check_mark_button: \33[32mInstalled AdaBins successfully.\33[0m\n")
    MAX_ADABINS_AREA = 500000

try:
    print(":hourglass_not_done: Installing Disco Xform Utilities...")
    import disco_xform_utils as dxf
    sys.path.append(PROJECT_DIR)
except:
    if not os.path.exists("disco-diffusion"):
        gitclone("https://github.com/alembics/disco-diffusion.git")
    if not os.path.exists('disco_xform_utils.py'):
        shutil.move('disco-diffusion/disco_xform_utils.py', 'disco_xform_utils.py')
    sys.path.append(PROJECT_DIR)
finally:
    import disco_xform_utils as dxf
    if 'disco_xform_utils' in sys.modules:
        print(":check_mark_button: \33[32mInstalled Disco Xform Utilities successfuly.\33[0m\n")

try:
    print(":hourglass_not_done: Initializing general imports...\n")
    from dataclasses import dataclass
    from functools import partial
    import cv2, gc, io, math, timm, lpips, requests, json, random, hashlib, time, warnings, pprint, traceback
    import pandas as pd
    from pprint import *
    from IPython import display
    from glob import glob
    from types import SimpleNamespace
    from tqdm.notebook import tqdm
    from guided_diffusion.script_util import create_model_and_diffusion, model_and_diffusion_defaults
    from datetime import datetime
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import Output
    from functools import partial
    if is_colab:
        os.chdir('/content')
        from google.colab import files
    else:
        os.chdir(f'{PROJECT_DIR}')
    from IPython.display import Image as ipyimg
    from numpy import asarray
    from einops import rearrange, repeat
    from omegaconf import OmegaConf
    from os.path import exists
    from urllib.parse import urlparse
except ImportError:
    raise ImportError("Unable to load dependency!")
finally:
    print(":check_mark_button: \33[32mGeneral imports initialized.\33[0m\n")

warnings.filterwarnings("ignore", category=UserWarning)

# Install easing_functions
try:
    print(":hourglass_not_done: Installing Cutn Easing Functions and Dependenceis...")
    multipip_res = subprocess.run(['pip', 'install', 'easing_functions', 'matplotlib', '--quiet'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(multipip_res, defaultprint=True)
except ImportError:
    raise ImportError('Unable to import Cutn Easing Functions and dependenceis.')
finally:
    from easing_functions import *
    if 'easing_functions' in sys.modules:
      print(":check_mark_button: \33[32mCutn Easing Functions installed successfuly.\33[0m\n")

# Noodle Soup prompts
nsp = False
use_this_prompt = False
try:
    print(":hourglass_not_done: Installing Noodle Soup Prompts...")
    wget('https://raw.githubusercontent.com/WASasquatch/noodle-soup-prompts/main/nsp_pantry.py', './')
except ImportError:
    nsp = False
    raise ImportError('Unable to import Noodle Soup Prompts!')
finally:
    import nsp_pantry
    from nsp_pantry import nspterminology, nsp_parse

if nsp_parse and nspterminology:
    print("\r\r:check_mark_button: \33[32mNSP installed successfuly.\33[0m \x1B[3mMmm... Noodle Soup.\x1B[0m\n")
    nsp = True

#@markdown ---
# Go Big Support=
#@markdown <font color="yellow">**WARNING:**</font> Go BIG v2 requires installing Pillow v9. This requires a restart of the runtime after this cell has completed. Then you must run this cell again. 
install_go_big_v2_support = False #@param{type: 'boolean'}

# Check Pillow Version
pil_warning = None
if install_go_big_v2_support:
    upgrade_pillow = subprocess.run(['pip', '-q', 'install', 'Pillow==9.0.0'],stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(upgrade_pillow)
    import PIL as PIL
    from PIL import Image, ImageFilter, ImageOps, ImageStat, ImageEnhance, ImageDraw
    from PIL.PngImagePlugin import PngInfo
    if not Image.__version__.startswith('9'):
        pil_warning = f":warning: \33[33m\u001b[4mWARNING:\33[0m \033[1mYou are using Pillow verison \33[33m{Image.__version__}\33[0m\033[1m, but version \33[32m9.0.0\33[0m \033[1mwas just installed. \n         Please go to Runtime -> Restart and Run all\33[0m"
        print("\n"+pil_warning+"\n")
    else: 
        print(f":check_mark_button: \33[32mGo BIG v2 support installed successfuly!\33[0m\n")
else:
    import PIL as PIL
    from PIL import Image, ImageFilter, ImageOps
    from PIL.PngImagePlugin import PngInfo
    if Image:
      pil_warning = f":warning: \33[33m\u001b[4mWARNING:\33[0m \033[1mYou are using Pillow verison \33[33m{Image.__version__}\33[0m\033[1m, but version \33[32m9.0.0\33[0m \033[1mis required for Go BIG v2. \n         If you want to use Go BIG v2, please enable Go BIG v2 Support, Run Setup {appname} Environment again, and go to Runtime -> Restart and Run all\33[0m"
      print(pil_warning+"\n")

# Initialize Torch
try:
    print(":hourglass_not_done: Initializing Torch...")
    import torch, torchvision
    from torch import nn
    from torch.nn import functional as F
    import torchvision.transforms as T
    import torchvision.transforms.functional as TF
except ImportError:
    raise ImportError("Unable to initialize Torch!")
finally:
    print(":check_mark_button: \33[32mTorch initialized.\33[0m")

DEVICE = torch.device('cuda:0' if (torch.cuda.is_available() and not useCPU) else 'cpu')
print('Using device:', DEVICE)
device = DEVICE # At least one of the modules expects this name..

if not useCPU:
    if torch.cuda.get_device_capability(DEVICE) == (8,0): ## A100 fix thanks to Emad
        print(':warning: Disabling CUDNN for A100 gpu', file=sys.stderr)
        torch.backends.cudnn.enabled = False

# Sampling Mode Switch
def get_resampling_mode():
    try:
        from PIL import __version__, Image
        major_ver = int(__version__.split('.')[0])
        if major_ver >= 9:
            return Image.Resampling.LANCZOS
        else:
            return Image.LANCZOS
    except Exception as ex:
        return 1  # 'Lanczos' irrespective of version.

print("\n:check_mark_button: \033[1mInstallation of Dependencies Complete\033[0m")
print("\n--[ Initializaing Midas Function ]--")

# DEFINE MIDAS FUNCTION

try:
    from midas.dpt_depth import DPTDepthModel
    from midas.midas_net import MidasNet
    from midas.midas_net_custom import MidasNet_small
    from midas.transforms import Resize, NormalizeImage, PrepareForNet
except ImportError:
    raise ImportError("Unable to load midas modules!")

# Initialize MiDaS depth model.
# It remains resident in VRAM and likely takes around 2GB VRAM.
# You could instead initialize it for each frame (and free it after each frame) to save VRAM.. but initializing it is slow.
default_models = {
    "midas_v21_small": f"{model_path}/midas_v21_small-70d6b9c8.pt",
    "midas_v21": f"{model_path}/midas_v21-f6b98070.pt",
    "dpt_large": f"{model_path}/dpt_large-midas-2f21e586.pt",
    "dpt_hybrid": f"{model_path}/dpt_hybrid-midas-501f0c75.pt",
    "dpt_hybrid_nyu": f"{model_path}/dpt_hybrid_nyu-2ce69ec7.pt",}


def init_midas_depth_model(midas_model_type="dpt_large", optimize=True):
    midas_model = None
    net_w = None
    net_h = None
    resize_mode = None
    normalization = None

    print(f"Initializing MiDaS '{midas_model_type}' depth model...")
    # load network
    midas_model_path = default_models[midas_model_type]

    if midas_model_type == "dpt_large": # DPT-Large
        midas_model = DPTDepthModel(
            path=midas_model_path,
            backbone="vitl16_384",
            non_negative=True,
        )
        net_w, net_h = 384, 384
        resize_mode = "minimal"
        normalization = NormalizeImage(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    elif midas_model_type == "dpt_hybrid": #DPT-Hybrid
        midas_model = DPTDepthModel(
            path=midas_model_path,
            backbone="vitb_rn50_384",
            non_negative=True,
        )
        net_w, net_h = 384, 384
        resize_mode="minimal"
        normalization = NormalizeImage(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    elif midas_model_type == "dpt_hybrid_nyu": #DPT-Hybrid-NYU
        midas_model = DPTDepthModel(
            path=midas_model_path,
            backbone="vitb_rn50_384",
            non_negative=True,
        )
        net_w, net_h = 384, 384
        resize_mode="minimal"
        normalization = NormalizeImage(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    elif midas_model_type == "midas_v21":
        midas_model = MidasNet(midas_model_path, non_negative=True)
        net_w, net_h = 384, 384
        resize_mode="upper_bound"
        normalization = NormalizeImage(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
    elif midas_model_type == "midas_v21_small":
        midas_model = MidasNet_small(midas_model_path, features=64, backbone="efficientnet_lite3", exportable=True, non_negative=True, blocks={'expand': True})
        net_w, net_h = 256, 256
        resize_mode="upper_bound"
        normalization = NormalizeImage(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
    else:
        print(f"midas_model_type '{midas_model_type}' not implemented")
        assert False

    midas_transform = T.Compose(
        [
            Resize(
                net_w,
                net_h,
                resize_target=None,
                keep_aspect_ratio=True,
                ensure_multiple_of=32,
                resize_method=resize_mode,
                image_interpolation_method=cv2.INTER_CUBIC,
            ),
            normalization,
            PrepareForNet(),
        ]
    )

    midas_model.eval()
    
    if optimize==True:
        if DEVICE == torch.device("cuda"):
            midas_model = midas_model.to(memory_format=torch.channels_last)  
            midas_model = midas_model.half()

    midas_model.to(DEVICE)

    print(f"MiDaS '{midas_model_type}' depth model initialized.")
    return midas_model, midas_transform, net_w, net_h, resize_mode, normalization

print("\n:check_mark_button: \033[1mMidas Function Initialized\033[0m")
print("\n--[ Initializing Main Functions ]--")

# DEFINE MAIN FUNCTIONS

# https://gist.github.com/adefossez/0646dbe9ed4005480a2407c62aac8869

# GO BIG GLOBALS
current_slice = None
go_big_prompts = None
runtime_prompts = []
original_seed = None
last_init = None

def interp(t):
    return 3 * t**2 - 2 * t ** 3

def perlin(width, height, scale=10, device=None):
    gx, gy = torch.randn(2, width + 1, height + 1, 1, 1, device=device)
    xs = torch.linspace(0, 1, scale + 1)[:-1, None].to(device)
    ys = torch.linspace(0, 1, scale + 1)[None, :-1].to(device)
    wx = 1 - interp(xs)
    wy = 1 - interp(ys)
    dots = 0
    dots += wx * wy * (gx[:-1, :-1] * xs + gy[:-1, :-1] * ys)
    dots += (1 - wx) * wy * (-gx[1:, :-1] * (1 - xs) + gy[1:, :-1] * ys)
    dots += wx * (1 - wy) * (gx[:-1, 1:] * xs - gy[:-1, 1:] * (1 - ys))
    dots += (1 - wx) * (1 - wy) * (-gx[1:, 1:] * (1 - xs) - gy[1:, 1:] * (1 - ys))
    return dots.permute(0, 2, 1, 3).contiguous().view(width * scale, height * scale)

def perlin_ms(octaves, width, height, grayscale, device=device):
    out_array = [0.5] if grayscale else [0.5, 0.5, 0.5]
    # out_array = [0.0] if grayscale else [0.0, 0.0, 0.0]
    for i in range(1 if grayscale else 3):
        scale = 2 ** len(octaves)
        oct_width = width
        oct_height = height
        for oct in octaves:
            p = perlin(oct_width, oct_height, scale, device)
            out_array[i] += p * oct
            scale //= 2
            oct_width *= 2
            oct_height *= 2
    return torch.cat(out_array)

def create_perlin_noise(octaves=[1, 1, 1, 1], width=2, height=2, grayscale=True):
    out = perlin_ms(octaves, width, height, grayscale)
    if grayscale:
        out = TF.resize(size=(side_y, side_x), img=out.unsqueeze(0))
        out = TF.to_pil_image(out.clamp(0, 1)).convert('RGB')
    else:
        out = out.reshape(-1, 3, out.shape[0]//3, out.shape[1])
        out = TF.resize(size=(side_y, side_x), img=out)
        out = TF.to_pil_image(out.clamp(0, 1).squeeze())

    out = ImageOps.autocontrast(out)
    return out

def regen_perlin():
    if perlin_mode == 'color':
        init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, False)
        init2 = create_perlin_noise([1.5**-i*0.5 for i in range(8)], 4, 4, False)
    elif perlin_mode == 'gray':
        init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, True)
        init2 = create_perlin_noise([1.5**-i*0.5 for i in range(8)], 4, 4, True)
    else:
        init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, False)
        init2 = create_perlin_noise([1.5**-i*0.5 for i in range(8)], 4, 4, True)

    init = TF.to_tensor(init).add(TF.to_tensor(init2)).div(2).to(device).unsqueeze(0).mul(2).sub(1)
    del init2
    return init.expand(batch_size, -1, -1, -1)

def fetch(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        r = requests.get(url_or_path)
        r.raise_for_status()
        fd = io.BytesIO()
        fd.write(r.content)
        fd.seek(0)
        return fd
    return open(url_or_path, 'rb')

def read_image_workaround(path):
    """OpenCV reads images as BGR, Pillow saves them as RGB. Work around
    this incompatibility to avoid colour inversions."""
    im_tmp = cv2.imread(path)
    return cv2.cvtColor(im_tmp, cv2.COLOR_BGR2RGB)

def parse_prompt(prompt):
    if prompt.startswith('http://') or prompt.startswith('https://'):
        vals = prompt.rsplit(':', 2)
        vals = [vals[0] + ':' + vals[1], *vals[2:]]
    else:
        vals = prompt.rsplit(':', 1)
    vals = vals + ['', '1'][len(vals):]
    vals[1] = eval(vals[1])
    if isinstance(vals[1], int) or isinstance(vals[1], float):
      return vals[0], [ float(x) for x in [float(vals[1])]*1000 ]
    elif isinstance(vals[1], list):
      return vals[0], [ float(x) for x in vals[1] ]
    else:
      raise TypeError("Only strings that evaluate to integers, floats and lists are allowed")

def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))

def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()

def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]

def resample(input, size, align_corners=True):
    n, c, h, w = input.shape
    dh, dw = size

    input = input.reshape([n * c, 1, h, w])

    if dh < h:
        kernel_h = lanczos(ramp(dh / h, 2), 2).to(input.device, input.dtype)
        pad_h = (kernel_h.shape[0] - 1) // 2
        input = F.pad(input, (0, 0, pad_h, pad_h), 'reflect')
        input = F.conv2d(input, kernel_h[None, None, :, None])

    if dw < w:
        kernel_w = lanczos(ramp(dw / w, 2), 2).to(input.device, input.dtype)
        pad_w = (kernel_w.shape[0] - 1) // 2
        input = F.pad(input, (pad_w, pad_w, 0, 0), 'reflect')
        input = F.conv2d(input, kernel_w[None, None, None, :])

    input = input.reshape([n, c, h, w])
    return F.interpolate(input, size, mode='bicubic', align_corners=align_corners)

class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, skip_augs=False):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.skip_augs = skip_augs
        self.augs = T.Compose([
            T.RandomHorizontalFlip(p=0.5),
            T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
            T.RandomAffine(degrees=15, translate=(0.1, 0.1)),
            T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
            T.RandomPerspective(distortion_scale=0.4, p=0.7),
            T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
            T.RandomGrayscale(p=0.15),
            T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
            # T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        ])

    def forward(self, input):
        input = T.Pad(input.shape[2]//4, fill=0)(input)
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)

        cutouts = []
        for ch in range(self.cutn):
            if ch > self.cutn - self.cutn//4:
                cutout = input.clone()
            else:
                size = int(max_size * torch.zeros(1,).normal_(mean=.8, std=.3).clip(float(self.cut_size/max_size), 1.))
                offsetx = torch.randint(0, abs(sideX - size + 1), ())
                offsety = torch.randint(0, abs(sideY - size + 1), ())
                cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]

            if not self.skip_augs:
                cutout = self.augs(cutout)
            cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))
            del cutout

        cutouts = torch.cat(cutouts, dim=0)
        return cutouts

cutout_debug = False
padargs = {}

class MakeCutoutsDango(nn.Module):
    def __init__(self, cut_size,
                 Overview=4, 
                 InnerCrop = 0, IC_Size_Pow=0.5, IC_Grey_P = 0.2
                 ):
        super().__init__()
        self.cut_size = cut_size
        self.Overview = Overview
        self.InnerCrop = InnerCrop
        self.IC_Size_Pow = IC_Size_Pow
        self.IC_Grey_P = IC_Grey_P
        if args.animation_mode == 'None':
          self.augs = T.Compose([
              T.RandomHorizontalFlip(p=0.5),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.RandomAffine(degrees=10, translate=(0.05, 0.05),  interpolation = T.InterpolationMode.BILINEAR),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.RandomGrayscale(p=0.1),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
          ])
        elif args.animation_mode == 'Video Input':
          self.augs = T.Compose([
              T.RandomHorizontalFlip(p=0.5),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.RandomAffine(degrees=15, translate=(0.1, 0.1)),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.RandomPerspective(distortion_scale=0.4, p=0.7),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.RandomGrayscale(p=0.15),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              # T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
          ])
        elif  args.animation_mode == '2D' or args.animation_mode == '3D':
          self.augs = T.Compose([
              T.RandomHorizontalFlip(p=0.4),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.RandomAffine(degrees=10, translate=(0.05, 0.05),  interpolation = T.InterpolationMode.BILINEAR),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.RandomGrayscale(p=0.1),
              T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
              T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.3),
          ])
          

    def forward(self, input):
        cutouts = []
        gray = T.Grayscale(3)
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        l_size = max(sideX, sideY)
        output_shape = [1,3,self.cut_size,self.cut_size] 
        output_shape_2 = [1,3,self.cut_size+2,self.cut_size+2]
        pad_input = F.pad(input,((sideY-max_size)//2,(sideY-max_size)//2,(sideX-max_size)//2,(sideX-max_size)//2), **padargs)
        cutout = resize(pad_input, out_shape=output_shape)

        if self.Overview>0:
            if self.Overview<=4:
                if self.Overview>=1:
                    cutouts.append(cutout)
                if self.Overview>=2:
                    cutouts.append(gray(cutout))
                if self.Overview>=3:
                    cutouts.append(TF.hflip(cutout))
                if self.Overview==4:
                    cutouts.append(gray(TF.hflip(cutout)))
            else:
                cutout = resize(pad_input, out_shape=output_shape)
                for _ in range(self.Overview):
                    cutouts.append(cutout)

            if cutout_debug:
                if is_colab:
                    TF.to_pil_image(cutouts[0].clamp(0, 1).squeeze(0)).save("/content/cutout_overview0.jpg",quality=99)
                else:
                    TF.to_pil_image(cutouts[0].clamp(0, 1).squeeze(0)).save("cutout_overview0.jpg",quality=99)

                              
        if self.InnerCrop >0:
            for i in range(self.InnerCrop):
                size = int(torch.rand([])**self.IC_Size_Pow * (max_size - min_size) + min_size)
                offsetx = torch.randint(0, sideX - size + 1, ())
                offsety = torch.randint(0, sideY - size + 1, ())
                cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
                if i <= int(self.IC_Grey_P * self.InnerCrop):
                    cutout = gray(cutout)
                cutout = resize(cutout, out_shape=output_shape)
                cutouts.append(cutout)
            if cutout_debug:
                if is_colab:
                    TF.to_pil_image(cutouts[-1].clamp(0, 1).squeeze(0)).save("/content/cutout_InnerCrop.jpg",quality=99)
                else:
                    TF.to_pil_image(cutouts[-1].clamp(0, 1).squeeze(0)).save("cutout_InnerCrop.jpg",quality=99)
        cutouts = torch.cat(cutouts)
        if skip_augs is not True: cutouts=self.augs(cutouts)
        return cutouts

def spherical_dist_loss(x, y):
    x = F.normalize(x, dim=-1)
    y = F.normalize(y, dim=-1)
    return (x - y).norm(dim=-1).div(2).arcsin().pow(2).mul(2)     

def tv_loss(input):
    """L2 total variation loss, as in Mahendran et al."""
    input = F.pad(input, (0, 1, 0, 1), 'replicate')
    x_diff = input[..., :-1, 1:] - input[..., :-1, :-1]
    y_diff = input[..., 1:, :-1] - input[..., :-1, :-1]
    return (x_diff**2 + y_diff**2).mean([1, 2, 3])


def range_loss(input):
    return (input - input.clamp(-1, 1)).pow(2).mean([1, 2, 3])

stop_on_next_loop = False  # Make sure GPU memory doesn't get corrupted from cancelling the run mid-way through, allow a full frame to complete
TRANSLATION_SCALE = 1.0/200.0

def do_3d_step(img_filepath, frame_num, midas_model, midas_transform):
  if args.key_frames:
    translation_x = args.translation_x_series[frame_num]
    translation_y = args.translation_y_series[frame_num]
    translation_z = args.translation_z_series[frame_num]
    rotation_3d_x = args.rotation_3d_x_series[frame_num]
    rotation_3d_y = args.rotation_3d_y_series[frame_num]
    rotation_3d_z = args.rotation_3d_z_series[frame_num]
    print(
        f'translation_x: {translation_x}',
        f'translation_y: {translation_y}',
        f'translation_z: {translation_z}',
        f'rotation_3d_x: {rotation_3d_x}',
        f'rotation_3d_y: {rotation_3d_y}',
        f'rotation_3d_z: {rotation_3d_z}',
    )

  translate_xyz = [-translation_x*TRANSLATION_SCALE, translation_y*TRANSLATION_SCALE, -translation_z*TRANSLATION_SCALE]
  rotate_xyz_degrees = [rotation_3d_x, rotation_3d_y, rotation_3d_z]
  print('translation:',translate_xyz)
  print('rotation:',rotate_xyz_degrees)
  rotate_xyz = [math.radians(rotate_xyz_degrees[0]), math.radians(rotate_xyz_degrees[1]), math.radians(rotate_xyz_degrees[2])]
  rot_mat = p3dT.euler_angles_to_matrix(torch.tensor(rotate_xyz, device=device), "XYZ").unsqueeze(0)
  print("rot_mat: " + str(rot_mat))
  next_step_pil = dxf.transform_image_3d(img_filepath, midas_model, midas_transform, DEVICE,
                                          rot_mat, translate_xyz, args.near_plane, args.far_plane,
                                          args.fov, padding_mode=args.padding_mode,
                                          sampling_mode=args.sampling_mode, midas_weight=args.midas_weight)
  return next_step_pil

def symmetry_transformation_fn(x):
    if args.use_horizontal_symmetry:
        [n, c, h, w] = x.size()
        x = torch.concat((x[:, :, :, :w//2], torch.flip(x[:, :, :, :w//2], [-1])), -1)
        print("horizontal symmetry applied")
    if args.use_vertical_symmetry:
        [n, c, h, w] = x.size()
        x = torch.concat((x[:, :, :h//2, :], torch.flip(x[:, :, :h//2, :], [-2])), -2)
        print("vertical symmetry applied")
    return x

## DD Image Prep Funciton
def init_image_prep(img, gblur, noise, nblur, ngrey=False, gdouble=False, pilimg=False):
    valid_ext = ['.jpg','.jpeg','.gif','.png']
    file_ext = os.path.splitext(img)[1] if type(img) is str else True
    if file_ext in valid_ext or file_ext == True :
        im1 = img if pilimg else PIL.Image.open(fetch(img)).convert('RGB')
        im2 = im1.filter(ImageFilter.GaussianBlur(radius = gblur))
        im3 = np.asarray(im2)
        im3 = cv2.cvtColor(im3, cv2.COLOR_BGR2RGB)
        gn = np.random.normal(0,noise,im3.size)
        gn = gn.reshape(im3.shape[0],im3.shape[1],im3.shape[2]).astype('uint8')
        if gdouble:
            gn2 = np.random.normal(0,noise,im3.size)
            gn2 = gn2.reshape(im3.shape[0],im3.shape[1],im3.shape[2]).astype('uint8')
        if ngrey:
            gn = cv2.cvtColor(gn, cv2.COLOR_BGR2GRAY)
            gn = cv2.cvtColor(gn, cv2.COLOR_BGR2RGB)
            if gdouble:
                gn2 = cv2.cvtColor(gn2, cv2.COLOR_BGR2GRAY)
                gn2 = cv2.cvtColor(gn2, cv2.COLOR_BGR2RGB)
        if gdouble:
            gn = cv2.add(gn,gn2)
        gn = cv2.GaussianBlur(gn,(nblur,nblur),cv2.BORDER_DEFAULT)
        gi = cv2.add(im3,gn)
        gi = cv2.cvtColor(gi, cv2.COLOR_BGR2RGB)
        im1 = PIL.Image.fromarray(gi)
        #im2 = im1.filter(ImageFilter.GaussianBlur(radius = nblur))
        im2 = im2.filter(ImageFilter.SHARPEN)
        return im2
    else:
        return None

# Basic image display
def displayJsImage(name, img):
    from IPython.display import display, Javascript, clear_output
    from google.colab.output import eval_js
    from base64 import b64encode
    from google.colab import files
    img = np.asarray(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ext = os.path.splitext(name)[1].replace('.', '', 1)
    ext = ext if ext != 'jpg' else 'jpeg'
    js = Javascript('''
        async function showImage(name, image, width, height) {
        img = document.getElementById(name);
        cont = document.getElementById(name+'_container');
        if(img == null && cont == null) {
            cont = document.createElement('div');
            cont.id = name+'_container';
            link = document.createElement('a');
            link.href = image;
            link.target = '_blank';
            img = document.createElement('img');
            img.id = name;
            img.class = "resultImage"
            cont.style = 'display:inline-block;width:auto;text-align:center;font-size:14px;font-weight:bold;background-color:rgba(0,0,0,0.1);border-radius:5px;padding:2px;margin:2px;'
            document.body.appendChild(cont)
            cont.innerHTML = '<p style="margin:3px auto;width:180px;white-space:nowrap;overflow:hidden;text-overflow:ellipsis;">'+name+'</p>';
            cont.appendChild(link);
            link.appendChild(img);
        }
        img.src = image;
        //img.width = (width / 2);
        //img.height = (height / 2);
        img.style = "margin: 5px; vertical-align: text-top; max-width: 256px; max-height: 512px;";
        }
    ''')
    height, width = img.shape[:2]
    ret, data = cv2.imencode('.png', img)
    data = b64encode(data)
    data = data.decode()
    data = 'data:image/' + ext + ';base64,' + data
    display(js)
    eval_js(f'showImage("{name}", "{data}", {width}, {height})')

def do_run():
  seed = args.seed
  print(range(args.start_frame, args.max_frames), defaultprint=True)

  if (args.animation_mode == "3D") and (args.midas_weight > 0.0):
      midas_model, midas_transform, midas_net_w, midas_net_h, midas_resize_mode, midas_normalization = init_midas_depth_model(args.midas_depth_model)
  for frame_num in range(args.start_frame, args.max_frames):
      if stop_on_next_loop:
        break
      
      display.clear_output(wait=True)

      # Print Frame progress if animation mode is on
      if args.animation_mode != "None":
        batchBar = tqdm(range(args.max_frames), desc ="Frames")
        batchBar.n = frame_num
        batchBar.refresh()

      
      # Inits if not video frames
      if args.animation_mode != "Video Input":
        if args.init_image in ['','none', 'None', 'NONE']:
          init_image = None
        else:
          init_image = args.init_image
        init_scale = args.init_scale
        skip_steps = args.skip_steps

      if args.animation_mode == "2D":
        if args.key_frames:
          angle = args.angle_series[frame_num]
          zoom = args.zoom_series[frame_num]
          translation_x = args.translation_x_series[frame_num]
          translation_y = args.translation_y_series[frame_num]
          print(
              f'angle: {angle}',
              f'zoom: {zoom}',
              f'translation_x: {translation_x}',
              f'translation_y: {translation_y}',
          )
        
        if frame_num > 0:
          seed += 1
          if resume_run and frame_num == start_frame:
            img_0 = cv2.imread(batchFolder+f"/{batch_name}({batchNum})_{start_frame-1:04}.png")
          else:
            img_0 = cv2.imread('prevFrame.png')
          center = (1*img_0.shape[1]//2, 1*img_0.shape[0]//2)
          trans_mat = np.float32(
              [[1, 0, translation_x],
              [0, 1, translation_y]]
          )
          rot_mat = cv2.getRotationMatrix2D( center, angle, zoom )
          trans_mat = np.vstack([trans_mat, [0,0,1]])
          rot_mat = np.vstack([rot_mat, [0,0,1]])
          transformation_matrix = np.matmul(rot_mat, trans_mat)
          img_0 = cv2.warpPerspective(
              img_0,
              transformation_matrix,
              (img_0.shape[1], img_0.shape[0]),
              borderMode=cv2.BORDER_WRAP
          )

          cv2.imwrite('prevFrameScaled.png', img_0)
          init_image = 'prevFrameScaled.png'
          init_scale = args.frames_scale
          skip_steps = args.calc_frames_skip_steps

      if args.animation_mode == "3D":
        if frame_num > 0:
          seed += 1    
          if resume_run and frame_num == start_frame:
            img_filepath = batchFolder+f"/{batch_name}({batchNum})_{start_frame-1:04}.png"
            if turbo_mode and frame_num > turbo_preroll:
              shutil.copyfile(img_filepath, 'oldFrameScaled.png')
          else:
            img_filepath = '/content/prevFrame.png' if is_colab else 'prevFrame.png'

          next_step_pil = do_3d_step(img_filepath, frame_num, midas_model, midas_transform)
          next_step_pil.save('prevFrameScaled.png')

          ### Turbo mode - skip some diffusions, use 3d morph for clarity and to save time
          if turbo_mode:
            if frame_num == turbo_preroll: #start tracking oldframe
              next_step_pil.save('oldFrameScaled.png')#stash for later blending          
            elif frame_num > turbo_preroll:
              #set up 2 warped image sequences, old & new, to blend toward new diff image
              old_frame = do_3d_step('oldFrameScaled.png', frame_num, midas_model, midas_transform)
              old_frame.save('oldFrameScaled.png')
              if frame_num % int(turbo_steps) != 0: 
                print('turbo skip this frame: skipping clip diffusion steps')
                filename = f'{args.batch_name}({args.batchNum})_{frame_num:04}.png'
                blend_factor = ((frame_num % int(turbo_steps))+1)/int(turbo_steps)
                print('turbo skip this frame: skipping clip diffusion steps and saving blended frame')
                newWarpedImg = cv2.imread('prevFrameScaled.png')#this is already updated..
                oldWarpedImg = cv2.imread('oldFrameScaled.png')
                blendedImage = cv2.addWeighted(newWarpedImg, blend_factor, oldWarpedImg,1-blend_factor, 0.0)
                cv2.imwrite(f'{batchFolder}/{filename}',blendedImage)
                next_step_pil.save(f'{img_filepath}') # save it also as prev_frame to feed next iteration
                if vr_mode:
                  generate_eye_views(TRANSLATION_SCALE,batchFolder,filename,frame_num,midas_model, midas_transform)
                continue
              else:
                #if not a skip frame, will run diffusion and need to blend.
                oldWarpedImg = cv2.imread('prevFrameScaled.png')
                cv2.imwrite(f'oldFrameScaled.png',oldWarpedImg)#swap in for blending later 
                print('clip/diff this frame - generate clip diff image')

          init_image = 'prevFrameScaled.png'
          init_scale = args.frames_scale
          skip_steps = args.calc_frames_skip_steps

      if  args.animation_mode == "Video Input":
        init_scale = args.video_init_frames_scale
        skip_steps = args.calc_frames_skip_steps
        if not video_init_seed_continuity:
          seed += 1
        if video_init_flow_warp:
          if frame_num == 0: 
            skip_steps = args.video_init_skip_steps
            init_image = f'{videoFramesFolder}/{frame_num+1:04}.jpg'
          if frame_num > 0: 
            prev = PIL.Image.open(batchFolder+f"/{batch_name}({batchNum})_{frame_num-1:04}.png")
            
            frame1_path = f'{videoFramesFolder}/{frame_num:04}.jpg'
            frame2 = PIL.Image.open(f'{videoFramesFolder}/{frame_num+1:04}.jpg')
            flo_path = f"/{flo_folder}/{frame1_path.split('/')[-1]}.npy"
            
            init_image = 'warped.png'
            print(video_init_flow_blend, defaultprint=True)
            weights_path = None
            if video_init_check_consistency:
                # TBD
                pass
 
            warp(prev, frame2, flo_path, blend=video_init_flow_blend, weights_path=weights_path).save(init_image)
            
        else:
          init_image = f'{videoFramesFolder}/{frame_num+1:04}.jpg'


      loss_values = []

      #if seed is not None:
      #    random.seed(seed)
      #    np.random.seed(seed)
      #    torch.manual_seed(seed)
      
      # CLIP init prompts original location...
      
      run_clip_init = False
      cur_t = None
  
      def cond_fn(x, t, y=None):
          with torch.enable_grad():
              x_is_NaN = False
              x = x.detach().requires_grad_()
              n = x.shape[0]
              if use_secondary_model is True:
                alpha = torch.tensor(diffusion.sqrt_alphas_cumprod[cur_t], device=device, dtype=torch.float32)
                sigma = torch.tensor(diffusion.sqrt_one_minus_alphas_cumprod[cur_t], device=device, dtype=torch.float32)
                cosine_t = alpha_sigma_to_t(alpha, sigma)
                out = secondary_model(x, cosine_t[None].repeat([n])).pred
                fac = diffusion.sqrt_one_minus_alphas_cumprod[cur_t]
                x_in = out * fac + x * (1 - fac)
                x_in_grad = torch.zeros_like(x_in)
              else:
                my_t = torch.ones([n], device=device, dtype=torch.long) * cur_t
                out = diffusion.p_mean_variance(model, x, my_t, clip_denoised=False, model_kwargs={'y': y})
                fac = diffusion.sqrt_one_minus_alphas_cumprod[cur_t]
                x_in = out['pred_xstart'] * fac + x * (1 - fac)
                x_in_grad = torch.zeros_like(x_in)
              for model_stat in model_stats:
                for i in range(args.cutn_batches):
                    t_int = int(t.item())+1 #errors on last step without +1, need to find source
                    #when using SLIP Base model the dimensions need to be hard coded to avoid AttributeError: 'VisionTransformer' object has no attribute 'input_resolution'
                    try:
                        input_resolution=model_stat["clip_model"].visual.input_resolution
                    except:
                        input_resolution=224

                    cuts = MakeCutoutsDango(input_resolution,
                            Overview= args.cut_overview[1000-t_int], 
                            InnerCrop = args.cut_innercut[1000-t_int],
                            IC_Size_Pow=args.cut_ic_pow[1000-t_int],
                            IC_Grey_P = args.cut_icgray_p[1000-t_int]
                            )
                    clip_in = normalize(cuts(x_in.add(1).div(2)))
                    image_embeds = model_stat["clip_model"].encode_image(clip_in).float()
                    dists = spherical_dist_loss(image_embeds.unsqueeze(1), model_stat["target_embeds"].unsqueeze(0))
                    dists = spherical_dist_loss(image_embeds.unsqueeze(1), model_stat["target_embeds"].unsqueeze(0))
                    dists = dists.view([args.cut_overview[1000-t_int]+args.cut_innercut[1000-t_int], n, -1])

                    step_weights = []
                    for i in range(0, len(model_stat["weights"])):
                      step_weights += [model_stat["weights"][i][1000-t_int]]
                      
                    if np.absolute(np.sum(step_weights)) < 1e-3: raise RuntimeError('The weights must not sum to 0.')
                    step_weights_divisor = np.absolute(np.sum(step_weights))
                    step_weights = [item / step_weights_divisor for item in step_weights]
                    step_weights = torch.tensor(step_weights, device=device)
                    losses = dists.mul(step_weights).sum(2).mean(0)
                    loss_values.append(losses.sum().item()) # log loss, probably shouldn't do per cutn_batch
                    x_in_grad += torch.autograd.grad(losses.sum() * clip_guidance_scale, x_in)[0] / cutn_batches

              #print("step_weights are " + str(step_weights))
              tv_losses = tv_loss(x_in)
              if use_secondary_model is True:
                range_losses = range_loss(out)
              else:
                range_losses = range_loss(out['pred_xstart'])
              sat_losses = torch.abs(x_in - x_in.clamp(min=-1,max=1)).mean()
              loss = tv_losses.sum() * tv_scale + range_losses.sum() * range_scale + sat_losses.sum() * sat_scale
              if init is not None and init_scale:
                  init_losses = lpips_model(x_in, init)
                  loss = loss + init_losses.sum() * init_scale
              x_in_grad += torch.autograd.grad(loss, x_in)[0]
              if torch.isnan(x_in_grad).any()==False:
                  grad = -torch.autograd.grad(x_in, x, x_in_grad)[0]
              else:
                # print("NaN'd")
                x_is_NaN = True
                grad = torch.zeros_like(x)
          if args.clamp_grad and x_is_NaN == False:
              magnitude = grad.square().mean().sqrt()
              return grad * magnitude.clamp(max=args.clamp_max) / magnitude  #min=-0.02, min=-clamp_max, 
          return grad
  
      if args.diffusion_sampling_mode == 'ddim':
          sample_fn = diffusion.ddim_sample_loop_progressive
      else:
          sample_fn = diffusion.plms_sample_loop_progressive


      image_display = Output()
      current_img = None
      seed_list = []

      for i in range(args.n_batches):

          # Increment Batch Seed
          if increment_n_batch_seed:
              if i == 0:
                  global original_seed
                  original_seed = seed
              seed_list.append(seed)
              seed += 1

          if seed is not None:
              np.random.seed(seed)
              random.seed(seed)
              torch.manual_seed(seed)
              torch.cuda.manual_seed_all(seed)
              torch.backends.cudnn.deterministic = True

          # CLIP Text Prompt Setup

          target_embeds, weights = [], []
          
          if args.prompts_series is not None and frame_num >= len(args.prompts_series):
            frame_prompt = args.prompts_series[-1]
          elif args.prompts_series is not None:
            frame_prompt = args.prompts_series[frame_num]
          else:
            frame_prompt = []
        
          if args.image_prompts_series is not None and frame_num >= len(args.image_prompts_series):
            print(args.image_prompts_series, defaultprint=True)
            image_prompt = args.image_prompts_series[-1]
          elif args.image_prompts_series is not None:
            image_prompt = args.image_prompts_series[frame_num]
          else:
            image_prompt = []

          import pprint
          from pprint import pprint, pformat

          # Parse Runtime Noodle Soup Prompts
          if generate_nsp_at_runtime:
              frame_prompt = nsp_parse(frame_prompt)
              runtime_prompts.append(frame_prompt)

          # Go BIG Prompt Recall
          if args.go_big:
              global go_big_prompts
              if go_big_prompts == None:
                  go_big_prompts = frame_prompt
              else:
                  frame_prompt = go_big_prompts

          model_stats = []
          for clip_model in clip_models:
                cutn = 16
                model_stat = {"clip_model":None,"target_embeds":[],"make_cutouts":None,"weights":[]}
                model_stat["clip_model"] = clip_model

                for prompt in frame_prompt:

                    txt, weight = parse_prompt(prompt)
                    txt = clip_model.encode_text(clip.tokenize(prompt).to(device)).float()
                    
                    if args.fuzzy_prompt:
                        for i in range(25):
                            model_stat["target_embeds"].append((txt + torch.randn(txt.shape).cuda() * args.rand_mag).clamp(0,1))
                            model_stat["weights"].append(weight)
                    else:
                        model_stat["target_embeds"].append(txt)
                        model_stat["weights"].append(weight)

                if image_prompt:
                  model_stat["make_cutouts"] = MakeCutouts(clip_model.visual.input_resolution, cutn, skip_augs=skip_augs) 
                  for prompt in image_prompt:
                      path, weight = parse_prompt(prompt)
                      img = Image.open(fetch(path)).convert('RGB')
                      img = TF.resize(img, min(side_x, side_y, *img.size), T.InterpolationMode.LANCZOS)
                      batch = model_stat["make_cutouts"](TF.to_tensor(img).to(device).unsqueeze(0).mul(2).sub(1))
                      embed = clip_model.encode_image(normalize(batch)).float()
                      if fuzzy_prompt:
                          for i in range(25):
                              model_stat["target_embeds"].append((embed + torch.randn(embed.shape).cuda() * rand_mag).clamp(0,1))
                              weights.extend([weight / cutn] * cutn)
                      else:
                          model_stat["target_embeds"].append(embed)
                          model_stat["weights"].extend([weight / cutn] * cutn)
            
                model_stat["target_embeds"] = torch.cat(model_stat["target_embeds"])
                model_stats.append(model_stat)
    
          # if no init_masked is provided, we make one with the render mask
          def make_masked_init(image, mask):
            image = np.array(image)
            image = image.astype(np.float32)/255.0
            image = image[None].transpose(0,3,1,2)
            image = torch.from_numpy(image)

            mask = np.array(mask)
            mask = mask.astype(np.float32)/255.0
            mask = mask[None,None]
            mask = torch.from_numpy(mask)

            masked_image = (0+mask)*image
            return masked_image

          batch_init = False
          
          # CLIP init Prompts
          if not run_clip_init:
              init = None
              if args.init_image is not None:
                  if type(args.init_image) is str:
                      init_img = args.init_image
                  else:
                      init_img = args.init_image[i] if i < len(args.init_image) else args.init_image[-1]
                  if type(args.init_image) is list:
                      batch_init = f'Working on init_image {i+1}: {init_img}'
                  init_img = PIL.Image.open(fetch(init_img)).convert('RGB')
                  if args.do_prep_init_image and current_slice is None:
                      init_img = init_image_prep(init_img, args.gaussian_blur_radius, args.gaussian_noise_amount, args.gaussian_noise_blur_radius, args.gaussian_noise_greyscale, args.gaussian_noise_double, True)
                  init_img = init_img.resize((args.side_x, args.side_y), get_resampling_mode())
                  if args.do_go_big_slice_prep and current_slice is not None:
                      init_img = init_image_prep(init_img, args.gaussian_blur_radius, args.gaussian_noise_amount, args.gaussian_noise_blur_radius, args.gaussian_noise_greyscale, args.gaussian_noise_double, True)
                  if args.init_masked is not None:
                    init_masked_img = PIL.Image.open(fetch(args.init_masked)).convert('RGB')
                    init = TF.to_tensor(init_masked_img).to(device).unsqueeze(0).mul(2).sub(1)
                  else:
                    init = TF.to_tensor(init_img).to(device).unsqueeze(0).mul(2).sub(1)
                  init_img = init_img.convert('RGBA')
                
              rmask = None
              if args.render_mask is not None:
                rmask_img = PIL.Image.open(fetch(args.render_mask)).convert('L')
                rmask_img = rmask_img.resize((args.side_x, args.side_y), get_resampling_mode())
                rmask = TF.to_tensor(rmask_img).to(device).unsqueeze(0)
                if args.init_masked is None:
                  init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, False)
                  init = TF.to_pil_image(init.clamp(0, 1).squeeze())
                  init_mask = make_masked_init(init, rmask_img).to(device)
                  init_mask = TF.to_pil_image(init_mask.clamp(0, 1).squeeze())
                  init = TF.to_tensor(init_mask).to(device).unsqueeze(0).mul(2).sub(1)

              if (args.perlin_init == True) and (args.init_image == None):
                  if args.perlin_mode == 'color':
                      init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, False)
                      init2 = create_perlin_noise([1.5**-i*0.5 for i in range(8)], 4, 4, False)
                  elif args.perlin_mode == 'gray':
                    init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, True)
                    init2 = create_perlin_noise([1.5**-i*0.5 for i in range(8)], 4, 4, True)
                  else:
                    init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, False)
                    init2 = create_perlin_noise([1.5**-i*0.5 for i in range(8)], 4, 4, True)
                  # init = TF.to_tensor(init).add(TF.to_tensor(init2)).div(2).to(device)
                  init = TF.to_tensor(init).add(TF.to_tensor(init2)).div(2).to(device).unsqueeze(0).mul(2).sub(1)
                  del init2
              if args.init_image is None or type(args.init_image) is str:
                 run_clip_init = True


          if args.animation_mode == 'None':
            display.clear_output(wait=True)
            barMax = args.n_batches-1 if args.n_batches > 1 else args.n_batches
            descType = 'Slices' if current_slice is not None else 'Batches'
            batchBar = tqdm(range(barMax), desc=descType)
            batchBar.n = i
            batchBar.refresh()
          print('', defaultprint=True)
          display.display(image_display)
          import pprint
          if current_slice is not None:
              print(f":scissors: \033[1mSlice {str(current_slice)} :seedling: Seed: {seed}\033[0m")
          else:
              print(f":card_index_dividers: \033[1mBatch {str(i)} :seedling: Seed: {seed}\033[0m")
          print(':black_nib: Prompt:', pprint.pformat(frame_prompt, indent=4))
          if batch_init:
              print(':framed_picture:', batch_init)
          gc.collect()
          torch.cuda.empty_cache()
          cur_t = diffusion.num_timesteps - skip_steps - 1
          total_steps = cur_t

          if perlin_init:
              init = regen_perlin()

          if args.diffusion_sampling_mode == 'ddim':
              samples = sample_fn(
                  model,
                  (batch_size, 3, args.side_y, args.side_x),
                  clip_denoised=clip_denoised,
                  model_kwargs={},
                  cond_fn=cond_fn,
                  progress=True,
                  skip_timesteps=skip_steps,
                  init_image=init,
                  randomize_class=randomize_class,
                  eta=eta,
                  transformation_fn=symmetry_transformation_fn,
                  transformation_percent=args.transformation_percent
              )
          else:
              samples = sample_fn(
                  model,
                  (batch_size, 3, args.side_y, args.side_x),
                  clip_denoised=clip_denoised,
                  model_kwargs={},
                  cond_fn=cond_fn,
                  progress=True,
                  skip_timesteps=skip_steps,
                  init_image=init,
                  randomize_class=randomize_class,
                  order=2,
              )
          
          # with run_display:
          #display.clear_output(wait=True)
          for j, sample in enumerate(samples):    
            cur_t -= 1
            intermediateStep = False
            if args.steps_per_checkpoint is not None:
                if j % steps_per_checkpoint == 0 and j > 0:
                  intermediateStep = True
            elif j in args.intermediate_saves:
              intermediateStep = True
            with image_display:
              if j % args.display_rate == 0 or cur_t == -1 or intermediateStep == True:
                  for k, image in enumerate(sample['pred_xstart']):
                      # tqdm.write(f'Batch {i}, step {j}, output {k}:')
                      current_time = datetime.now().strftime('%y%m%d-%H%M%S_%f')
                      percent = math.ceil(j/total_steps*100)
                      filename_figure = f'{args.batch_name}({args.batchNum})_figure.png'
                      if args.n_batches > 0:
                        #if intermediates are saved to the subfolder, don't append a step or percentage to the name
                        if cur_t == -1 and args.intermediates_in_subfolder is True:
                          save_num = f'{frame_num:04}' if animation_mode != "None" else i
                          filename = f'{args.batch_name}({args.batchNum})_{save_num}.png'
                          filename_figure = f'{args.batch_name}({args.batchNum})_{save_num}_figure.png'
                        else:
                          #If we're working with percentages, append it
                          if args.steps_per_checkpoint is not None:
                            filename = f'{args.batch_name}({args.batchNum})_{i:04}-{percent:02}%.png'
                            filename_figure = f'{args.batch_name}({args.batchNum})_{i:04}-{percent:02}%_figure.png'
                          # Or else, iIf we're working with specific steps, append those
                          else:
                            filename = f'{args.batch_name}({args.batchNum})_{i:04}-{j:03}.png'
                            filename_figure = f'{args.batch_name}({args.batchNum})_{i:04}-{j:03}_figure.png'

                      image = TF.to_pil_image(image.add(1).div(2).clamp(0, 1))
                      if j % args.display_rate == 0 or cur_t == -1:
                        image.save('progress.png')
                        display.clear_output(wait=True)
                        display.display(display.Image('progress.png'))
                      if args.steps_per_checkpoint is not None:
                        if j % args.steps_per_checkpoint == 0 and j > 0:
                          if args.intermediates_in_subfolder is True:
                            image.save(f'{partialFolder}/{filename}')
                          else:
                            image.save(f'{batchFolder}/{filename}')
                      else:
                        if j in args.intermediate_saves:
                          if args.intermediates_in_subfolder is True:
                            image.save(f'{partialFolder}/{filename}')
                          else:
                            image.save(f'{batchFolder}/{filename}')
                      if cur_t == -1:
                        if args.save_metadata:
                          metadata = PngInfo()
                          if args.meta_author is not None and args.meta_author.lower() not in ['','none']:
                            metadata.add_text("author", args.meta_author)
                          if args.meta_copyright is not None and args.meta_copyright not in ['','none']:
                            metadata.add_text("copyright", args.meta_copyright)
                          metadata.add_text("software", appname)
                          metadata.add_text("prompt",str(frame_prompt))
                          metadata.add_text("seed", str(seed))
                          metadata.add_text("randomized_prompt", str(frame_prompt))
                        if frame_num == 0:
                          save_settings()
                        if args.animation_mode != "None":
                          image.save('prevFrame.png')
                        if render_mask:
                          image2 = image.copy()
                          image2.putalpha(rmask_img)
                          image3 = image2.copy()
                          image3 = Image.alpha_composite(init_img, image3)
                          image = image3.copy()
                          image.save('progress.png')
                        if args.save_metadata:
                          image.save(f'{batchFolder}/{filename}', pnginfo=metadata)
                          if save_plot_graph:
                              plt.savefig(f'{batchFolder}/{filename_figure}')
                          #loss_values = []  
                        else:
                          print(f'Saving image to: {batchFolder}/{filename}')
                          image.save(f'{batchFolder}/{filename}')
                          if save_plot_graph:
                            plt.savefig(f'{batchFolder}/{filename_figure}')
                          #loss_values = []  
                        if args.animation_mode == "3D":
                          # If turbo, save a blended image
                          if turbo_mode and frame_num > 0:
                            # Mix new image with prevFrameScaled
                            blend_factor = (1)/int(turbo_steps)
                            newFrame = cv2.imread('prevFrame.png') # This is already updated..
                            prev_frame_warped = cv2.imread('prevFrameScaled.png')
                            blendedImage = cv2.addWeighted(newFrame, blend_factor, prev_frame_warped, (1-blend_factor), 0.0)
                            cv2.imwrite(f'{batchFolder}/{filename}',blendedImage)
                            if save_plot_graph:
                                plt.savefig(f'{batchFolder}/{filename_figure}')
                          else:
                            image.save(f'{batchFolder}/{filename}')
                            if save_plot_graph:
                                plt.savefig(f'{batchFolder}/{filename_figure}')
                          if vr_mode:
                            generate_eye_views(TRANSLATION_SCALE, batchFolder, filename, frame_num, midas_model, midas_transform)
                        
                        # if frame_num != args.max_frames-1:
                        #   display.clear_output()
                        
          # All conditions end. Process complete
          if seed_list:
              print(':seedling: Seed List:', str(seed_list))
          plt.plot(np.array(loss_values), 'r')
          loss_values = []
      seed_list = []

def generate_eye_views(trans_scale,batchFolder,filename,frame_num,midas_model, midas_transform):
   for i in range(2):
      theta = vr_eye_angle * (math.pi/180)
      ray_origin = math.cos(theta) * vr_ipd / 2 * (-1.0 if i==0 else 1.0)
      ray_rotation = (theta if i==0 else -theta)
      translate_xyz = [-(ray_origin)*trans_scale, 0,0]
      rotate_xyz = [0, (ray_rotation), 0]
      rot_mat = p3dT.euler_angles_to_matrix(torch.tensor(rotate_xyz, device=device), "XYZ").unsqueeze(0)
      transformed_image = dxf.transform_image_3d(f'{batchFolder}/{filename}', midas_model, midas_transform, DEVICE,
                                                      rot_mat, translate_xyz, args.near_plane, args.far_plane,
                                                      args.fov, padding_mode=args.padding_mode,
                                                      sampling_mode=args.sampling_mode, midas_weight=args.midas_weight,spherical=True)
      eye_file_path = batchFolder+f"/frame_{frame_num:04}" + ('_l' if i==0 else '_r')+'.png'
      transformed_image.save(eye_file_path)

def save_settings():
    setting_list = {
      'generate_nsp_at_runtime': generate_nsp_at_runtime,
      'text_prompts': text_prompts,
      'runtime_prompts': None if not runtime_prompts else runtime_prompts,
      'go_big_prompts': None if not go_big_prompts else go_big_prompts,
      'image_prompts': image_prompts,
      'clip_guidance_scale': clip_guidance_scale,
      'tv_scale': tv_scale,
      'range_scale': range_scale,
      'sat_scale': sat_scale,
      # 'cutn': cutn,
      'cutn_batches': cutn_batches,
      'max_frames': max_frames,
      'interp_spline': interp_spline,
      'go_big': go_big,
      'go_big_scale': go_big_scale,
      'gobig_skip_percent': gobig_skip_percent,
      'gobig_overlap': gobig_overlap,
      'gobig_maximize': gobig_maximize,
      'go_big_init': go_big_init,
      'go_big_init_skipresize': go_big_init_skipresize,
      'init_masked': init_masked,
      'render_mask': render_mask,
      # 'rotation_per_frame': rotation_per_frame,
      'init_image': init_image,
      'init_scale': init_scale,
      'skip_steps': skip_steps,
      'do_prep_init_image': do_prep_init_image,
      'do_go_big_slice_prep': do_go_big_slice_prep,
      'gaussian_blur_radius': gaussian_blur_radius,
      'gaussian_noise_amount': gaussian_noise_amount,
      'gaussian_noise_blur_radius': gaussian_noise_blur_radius,
      'gaussian_noise_greyscale': gaussian_noise_greyscale,
      'gaussian_noise_double': gaussian_noise_double,
      # 'zoom_per_frame': zoom_per_frame,
      'frames_scale': frames_scale,
      'frames_skip_steps': frames_skip_steps,
      'perlin_init': perlin_init,
      'perlin_mode': perlin_mode,
      'skip_augs': skip_augs,
      'randomize_class': randomize_class,
      'clip_denoised': clip_denoised,
      'clamp_grad': clamp_grad,
      'clamp_max': clamp_max,
      'seed': seed,
      'fuzzy_prompt': fuzzy_prompt,
      'rand_mag': rand_mag,
      'eta': eta,
      'width': width_height[0],
      'height': width_height[1],
      'diffusion_model': diffusion_model,
      'use_secondary_model': use_secondary_model,
      'use_checkpoint': use_checkpoint,
      'steps': steps,
      'diffusion_steps': diffusion_steps,
      'diffusion_sampling_mode': diffusion_sampling_mode,
      'ViTB32': ViTB32,
      'ViTB16': ViTB16,
      'ViTL14': ViTL14,
      'ViTL14_336px': ViTL14_336px,
      'RN101': RN101,
      'RN50': RN50,
      'RN50x4': RN50x4,
      'RN50x16': RN50x16,
      'RN50x64': RN50x64,
      'ViTB32_laion2b_e16': ViTB32_laion2b_e16,
      'ViTB32_laion400m_e31': ViTB32_laion400m_e31,
      'ViTB32_laion400m_32': ViTB32_laion400m_32,
      'ViTB32quickgelu_laion400m_e31': ViTB32quickgelu_laion400m_e31,
      'ViTB32quickgelu_laion400m_e32': ViTB32quickgelu_laion400m_e32,
      'ViTB16_laion400m_e31': ViTB16_laion400m_e31,
      'ViTB16_laion400m_e32': ViTB16_laion400m_e32,
      'RN50_yffcc15m': RN50_yffcc15m,
      'RN50_cc12m': RN50_cc12m,
      'RN50_quickgelu_yfcc15m': RN50_quickgelu_yfcc15m,
      'RN50_quickgelu_cc12m': RN50_quickgelu_cc12m,
      'RN101_yfcc15m': RN101_yfcc15m,
      'RN101_quickgelu_yfcc15m': RN101_quickgelu_yfcc15m,
      'cut_overview': str(cut_overview),
      'cut_innercut': str(cut_innercut),
      'cut_ic_pow': str(cut_ic_pow),
      'cut_icgray_p': str(cut_icgray_p),
      'key_frames': key_frames,
      'max_frames': max_frames,
      'angle': angle,
      'zoom': zoom,
      'translation_x': translation_x,
      'translation_y': translation_y,
      'translation_z': translation_z,
      'rotation_3d_x': rotation_3d_x,
      'rotation_3d_y': rotation_3d_y,
      'rotation_3d_z': rotation_3d_z,
      'midas_depth_model': midas_depth_model,
      'midas_weight': midas_weight,
      'near_plane': near_plane,
      'far_plane': far_plane,
      'fov': fov,
      'padding_mode': padding_mode,
      'sampling_mode': sampling_mode,
      'video_init_path':video_init_path,
      'extract_nth_frame':extract_nth_frame,
      'video_init_seed_continuity': video_init_seed_continuity,
      'turbo_mode':turbo_mode,
      'turbo_steps':turbo_steps,
      'turbo_preroll':turbo_preroll,
      'use_horizontal_symmetry':use_horizontal_symmetry,
      'use_vertical_symmetry':use_vertical_symmetry,
      'transformation_percent':transformation_percent,
      #video init settings
      'video_init_steps': video_init_steps,
      'video_init_clip_guidance_scale': video_init_clip_guidance_scale,
      'video_init_tv_scale': video_init_tv_scale,
      'video_init_range_scale': video_init_range_scale,
      'video_init_sat_scale': video_init_sat_scale,
      'video_init_cutn_batches': video_init_cutn_batches,
      'video_init_skip_steps': video_init_skip_steps,
      'video_init_frames_scale': video_init_frames_scale,
      'video_init_frames_skip_steps': video_init_frames_skip_steps,
      #warp settings
      'video_init_flow_warp':video_init_flow_warp,
      'video_init_flow_blend':video_init_flow_blend,
      'video_init_check_consistency':video_init_check_consistency,
      'video_init_blend_mode':video_init_blend_mode
    }
    # print('Settings:', setting_list)
    with open(f"{batchFolder}/{batch_name}({batchNum})_settings.txt", "w+") as f:   #save settings
        json.dump(setting_list, f, ensure_ascii=False, indent=4)

# DEFINE SECONDARY MODEL

print(f"\n:check_mark_button: \033[1mMain Functions Initialized\033[0m")
print(f"\n--[ Initializing Secondary Model ]--")

def append_dims(x, n):
    return x[(Ellipsis, *(None,) * (n - x.ndim))]


def expand_to_planes(x, shape):
    return append_dims(x, len(shape)).repeat([1, 1, *shape[2:]])


def alpha_sigma_to_t(alpha, sigma):
    return torch.atan2(sigma, alpha) * 2 / math.pi


def t_to_alpha_sigma(t):
    return torch.cos(t * math.pi / 2), torch.sin(t * math.pi / 2)


@dataclass
class DiffusionOutput:
    v: torch.Tensor
    pred: torch.Tensor
    eps: torch.Tensor


class ConvBlock(nn.Sequential):
    def __init__(self, c_in, c_out):
        super().__init__(
            nn.Conv2d(c_in, c_out, 3, padding=1),
            nn.ReLU(inplace=True),
        )


class SkipBlock(nn.Module):
    def __init__(self, main, skip=None):
        super().__init__()
        self.main = nn.Sequential(*main)
        self.skip = skip if skip else nn.Identity()

    def forward(self, input):
        return torch.cat([self.main(input), self.skip(input)], dim=1)


class FourierFeatures(nn.Module):
    def __init__(self, in_features, out_features, std=1.):
        super().__init__()
        assert out_features % 2 == 0
        self.weight = nn.Parameter(torch.randn([out_features // 2, in_features]) * std)

    def forward(self, input):
        f = 2 * math.pi * input @ self.weight.T
        return torch.cat([f.cos(), f.sin()], dim=-1)


class SecondaryDiffusionImageNet(nn.Module):
    def __init__(self):
        super().__init__()
        c = 64  # The base channel count

        self.timestep_embed = FourierFeatures(1, 16)

        self.net = nn.Sequential(
            ConvBlock(3 + 16, c),
            ConvBlock(c, c),
            SkipBlock([
                nn.AvgPool2d(2),
                ConvBlock(c, c * 2),
                ConvBlock(c * 2, c * 2),
                SkipBlock([
                    nn.AvgPool2d(2),
                    ConvBlock(c * 2, c * 4),
                    ConvBlock(c * 4, c * 4),
                    SkipBlock([
                        nn.AvgPool2d(2),
                        ConvBlock(c * 4, c * 8),
                        ConvBlock(c * 8, c * 4),
                        nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
                    ]),
                    ConvBlock(c * 8, c * 4),
                    ConvBlock(c * 4, c * 2),
                    nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
                ]),
                ConvBlock(c * 4, c * 2),
                ConvBlock(c * 2, c),
                nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            ]),
            ConvBlock(c * 2, c),
            nn.Conv2d(c, 3, 3, padding=1),
        )

    def forward(self, input, t):
        timestep_embed = expand_to_planes(self.timestep_embed(t[:, None]), input.shape)
        v = self.net(torch.cat([input, timestep_embed], dim=1))
        alphas, sigmas = map(partial(append_dims, n=v.ndim), t_to_alpha_sigma(t))
        pred = input * alphas - v * sigmas
        eps = input * sigmas + v * alphas
        return DiffusionOutput(v, pred, eps)


class SecondaryDiffusionImageNet2(nn.Module):
    def __init__(self):
        super().__init__()
        c = 64  # The base channel count
        cs = [c, c * 2, c * 2, c * 4, c * 4, c * 8]

        self.timestep_embed = FourierFeatures(1, 16)
        self.down = nn.AvgPool2d(2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)

        self.net = nn.Sequential(
            ConvBlock(3 + 16, cs[0]),
            ConvBlock(cs[0], cs[0]),
            SkipBlock([
                self.down,
                ConvBlock(cs[0], cs[1]),
                ConvBlock(cs[1], cs[1]),
                SkipBlock([
                    self.down,
                    ConvBlock(cs[1], cs[2]),
                    ConvBlock(cs[2], cs[2]),
                    SkipBlock([
                        self.down,
                        ConvBlock(cs[2], cs[3]),
                        ConvBlock(cs[3], cs[3]),
                        SkipBlock([
                            self.down,
                            ConvBlock(cs[3], cs[4]),
                            ConvBlock(cs[4], cs[4]),
                            SkipBlock([
                                self.down,
                                ConvBlock(cs[4], cs[5]),
                                ConvBlock(cs[5], cs[5]),
                                ConvBlock(cs[5], cs[5]),
                                ConvBlock(cs[5], cs[4]),
                                self.up,
                            ]),
                            ConvBlock(cs[4] * 2, cs[4]),
                            ConvBlock(cs[4], cs[3]),
                            self.up,
                        ]),
                        ConvBlock(cs[3] * 2, cs[3]),
                        ConvBlock(cs[3], cs[2]),
                        self.up,
                    ]),
                    ConvBlock(cs[2] * 2, cs[2]),
                    ConvBlock(cs[2], cs[1]),
                    self.up,
                ]),
                ConvBlock(cs[1] * 2, cs[1]),
                ConvBlock(cs[1], cs[0]),
                self.up,
            ]),
            ConvBlock(cs[0] * 2, cs[0]),
            nn.Conv2d(cs[0], 3, 3, padding=1),
        )

    def forward(self, input, t):
        timestep_embed = expand_to_planes(self.timestep_embed(t[:, None]), input.shape)
        v = self.net(torch.cat([input, timestep_embed], dim=1))
        alphas, sigmas = map(partial(append_dims, n=v.ndim), t_to_alpha_sigma(t))
        pred = input * alphas - v * sigmas
        eps = input * sigmas + v * alphas
        return DiffusionOutput(v, pred, eps)

print("\n:check_mark_button: \033[1mSecondary Model Initialized\33[0m")
print(f"\n--[ :confetti_ball::party_popper: \033[1m\33[32m{appname} Environtment Setup Complete\33[0m :party_popper::confetti_ball: ]--")


In [ ]:
#@title 1.3 Load Disco Diffusion Settings File

# LOAD SETTINGS - Implemented by WASasquatch
#@markdown ##### **Load Settings from File**
#@markdown By default a settings file overrides base diffusion settings, but if you wish, you can select from the following options to overwride more to form crucial parts of, or an entire settings file.
settings_file_path = None #@param{type: 'string'}
#@markdown Loading a settings file to bypass these settings
load_prompts_and_inits = False #@param{type: 'boolean'}
#@markdown Load text and image prompts, as well as their related settings*
load_diffusion_models = True #@param{type: 'boolean'}
#@markdown Load diffusion model settings from file
load_CLIP_models = True #@param{type: 'boolean'}
#@markdown Load CLIP model settings from file

if settings_file_path in ["none",""]:
    settings_file_path = None

# 2. Diffusion and CLIP model settings

In [ ]:
#@markdown ####**Models Settings (note: For pixel art, the best is pixelartdiffusion_expanded):**
diffusion_model = "portrait_generator_v005" #@param ["256x256_diffusion_uncond", "512x512_diffusion_uncond_finetune_008100", "concept_art_generator_v000-1_alpha", "concept_art_generator_v000-2_alpha", "portrait_generator_v001", "portrait_generator_v002", "portrait_generator_v003", "portrait_generator_v004", "portrait_generator_v005", "Architecture_Diffusion_1-5m", "pixelartdiffusion_expanded", "pixel_art_diffusion_hard_256", "pixel_art_diffusion_soft_256", "pixelartdiffusion4k", "watercolordiffusion_2", "watercolordiffusion", "PulpSciFiDiffusion", "Lithography_Diffusion", "Medieval_Diffusion", "Liminal_Diffusion_v1", "Liminal_Diffusion_Source", "Floral_Diffusion", "FeiArt_Handpainted_CG_Diffusion", "Textile_Diffusion", "Isometric_Diffusion_Revrart512px", "Laproper_Diffusion_Deepspace_256", "Schnippi_Diffusion_512x512_V2", "custom"]

go_big_diffusion_model = "Inherit" #@param ["Inherit", "256x256_diffusion_uncond", "512x512_diffusion_uncond_finetune_008100", "concept_art_generator_v000-1_alpha", "portrait_generator_v001", "portrait_generator_v002", "portrait_generator_v003", "portrait_generator_v004", "pixelartdiffusion_expanded", "pixel_art_diffusion_hard_256", "pixel_art_diffusion_soft_256", "pixelartdiffusion4k", "watercolordiffusion_2", "watercolordiffusion", "PulpSciFiDiffusion", "Lithography_Diffusion", "Medieval_Diffusion", "Liminal_Diffusion_v1", "Liminal_Diffusion_Source", "Floral_Diffusion", "FeiArt_Handpainted_CG_Diffusion", "Textile_Diffusion", "Isometric_Diffusion_Revrart512px", "Laproper_Diffusion_Deepspace_256", "Schnippi_Diffusion_512x512_V2", "custom"]
#@markdown `go_big_diffusion_model`: Leave on `Inherit` if you want Go BIG to use `diffusion_model` instead of a custom selection

use_secondary_model = False #@param {type: 'boolean'}
diffusion_sampling_mode = 'ddim' #@param ['plms','ddim']
#@markdown #####**Custom model:**
custom_path = '/content/drive/MyDrive/AI/Checkpoints/concept-art-model/ema_0.9999_070000.pt'#@param {type: 'string'}
#@markdown <br>

#@markdown ###**CLIP settings:**
use_checkpoint = True #@param {type: 'boolean'}
ViTB32 = False #@param{type:"boolean"}
ViTB16 = False #@param{type:"boolean"}
ViTL14 = True #@param{type:"boolean"}
ViTL14_336px = False #@param{type:"boolean"}
RN101 = False #@param{type:"boolean"}
RN50 = False #@param{type:"boolean"}
RN50x4 = True #@param{type:"boolean"}
RN50x16 = False #@param{type:"boolean"}
RN50x64 = False #@param{type:"boolean"}
#@markdown <br>

#@markdown ###**OpenCLIP settings:**
ViTB32_laion2b_e16 = False #@param{type:"boolean"}
ViTB32_laion400m_e31 = False #@param{type:"boolean"}
ViTB32_laion400m_32 = False #@param{type:"boolean"}
ViTB32quickgelu_laion400m_e31 = False #@param{type:"boolean"}
ViTB32quickgelu_laion400m_e32 = False #@param{type:"boolean"}
ViTB16_laion400m_e31 = False #@param{type:"boolean"}
ViTB16_laion400m_e32 = False #@param{type:"boolean"}
RN50_yffcc15m = False #@param{type:"boolean"}
RN50_cc12m = False #@param{type:"boolean"}
RN50_quickgelu_yfcc15m = False #@param{type:"boolean"}
RN50_quickgelu_cc12m = False #@param{type:"boolean"}
RN101_yfcc15m = False #@param{type:"boolean"}
RN101_quickgelu_yfcc15m = False #@param{type:"boolean"}
#@markdown <br>

#@markdown **Note:** If you're having issues with model downloads, check this to compare SHA's:
check_model_SHA = False #@param{type:"boolean"}

if custom_path is not None and custom_path.lower() in ['none', '']:
    custom_path = None

# LOAD SETTINGS - Implemented by WASasquatch
if settings_file_path:

    if exists(settings_file_path):

        with open(settings_file_path) as json_file:

            jsd = json.load(json_file)
        
            if load_diffusion_models:
                print('Loading diffusion model settings from file: '+settings_file_path)
                if jsd.__contains__('use_checkpoint'):
                    use_checkpoint = jsd['use_checkpoint']
                if jsd.__contains__('diffusion_model'):
                    diffusion_model = jsd['diffusion_model']
                if jsd.__contains__('use_secondary_model'):
                    use_secondary_model = jsd['use_secondary_model']
            
            if load_CLIP_models:
                if jsd.__contains__('ViTB32'):
                    ViTB32 = jsd['ViTB32']
                if jsd.__contains__('ViTB16'):
                    ViTB16 = jsd['ViTB16']
                if jsd.__contains__('ViTL14'):
                    ViTL14 = jsd['ViTL14']
                if jsd.__contains__('ViTL14_336px'):
                    ViTL14_336px = jsd['ViTL14_336px']
                if jsd.__contains__('RN101'):
                    RN101 = jsd['RN101']
                if jsd.__contains__('RN50'):
                    RN50 = jsd['RN50']
                if jsd.__contains__('RN50x4'):
                    RN50x4 = jsd['RN50x4']
                if jsd.__contains__('RN50x16'):
                    RN50x16 = jsd['RN50x16']
                if jsd.__contains__('RN50x64'):
                    RN50x64 = jsd['RN50x64']
                if jsd.__contains__('ViTB32_laion2b_e16'):
                    ViTB32_laion2b_e16 = jsd['ViTB32_laion2b_e16']
                if jsd.__contains__('ViTB32_laion400m_e31'):
                    ViTB32_laion400m_e31 = jsd['ViTB32_laion400m_e31']
                if jsd.__contains__('ViTB32_laion400m_32'):
                    ViTB32_laion400m_32 = jsd['ViTB32_laion400m_32']
                if jsd.__contains__('ViTB32quickgelu_laion400m_e31'):
                    ViTB32quickgelu_laion400m_e31 = jsd['ViTB32quickgelu_laion400m_e31']
                if jsd.__contains__('ViTB32quickgelu_laion400m_e32'):
                    ViTB32quickgelu_laion400m_e32 = jsd['ViTB32quickgelu_laion400m_e32']
                if jsd.__contains__('ViTB16_laion400m_e31'):
                    ViTB16_laion400m_e31 = jsd['ViTB16_laion400m_e31']
                if jsd.__contains__('ViTB16_laion400m_e32'):
                    ViTB16_laion400m_e32 = jsd['ViTB16_laion400m_e32']
                if jsd.__contains__('RN50_yffcc15m'):
                    RN50_yffcc15m = jsd['RN50_yffcc15m']
                if jsd.__contains__('RN50_cc12m'):
                    RN50_cc12m = jsd['RN50_cc12m']
                if jsd.__contains__('RN50_quickgelu_yfcc15m'):
                    RN50_quickgelu_yfcc15m = jsd['RN50_quickgelu_yfcc15m']
                if jsd.__contains__('RN50_quickgelu_cc12m'):
                    RN50_quickgelu_cc12m = jsd['RN50_quickgelu_cc12m']
                if jsd.__contains__('RN101_yfcc15m'):
                    RN101_yfcc15m = jsd['RN101_yfcc15m']
                if jsd.__contains__('RN101_quickgelu_yfcc15m'):
                    RN101_quickgelu_yfcc15m = jsd['RN101_quickgelu_yfcc15m']

diff_model_map = {
    '256x256_diffusion_uncond': { 'downloaded': False, 'sha': 'a37c32fffd316cd494cf3f35b339936debdc1576dad13fe57c42399a5dbc78b1', 'uri_list': ['https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt', 'https://www.dropbox.com/s/9tqnqo930mpnpcn/256x256_diffusion_uncond.pt'] },
    '512x512_diffusion_uncond_finetune_008100': { 'downloaded': False, 'sha': '9c111ab89e214862b76e1fa6a1b3f1d329b1a88281885943d2cdbe357ad57648', 'uri_list': ['https://huggingface.co/lowlevelware/512x512_diffusion_unconditional_ImageNet/resolve/main/512x512_diffusion_uncond_finetune_008100.pt', 'https://the-eye.eu/public/AI/models/512x512_diffusion_unconditional_ImageNet/512x512_diffusion_uncond_finetune_008100.pt'] },

    'concept_art_generator_v000-1_alpha': { 'downloaded': False, 'sha': '0c0394148b4fb56234baee533c36f17202b8f936f28427a957f342aa2f18d040', 'uri_list': ['https://huggingface.co/WAS/concept_art_generator/resolve/main/concept_art_generator_v000-1_alpha.pt'] },
    'concept_art_generator_v000-2_alpha': { 'downloaded': False, 'sha': 'a7aee6b8dd73b6f3d377a8c579b99ed2922fc472ead16238074e08bf7517d075', 'uri_list': ['https://huggingface.co/WAS/concept_art_generator/resolve/main/concept_art_generator_v000-2_alpha.pt'] },

    'portrait_generator_v001': { 'downloaded': False, 'sha': 'b7e8c747af880d4480b6707006f1ace000b058dd0eac5bb13558ba3752d9b5b9', 'uri_list': ['https://huggingface.co/felipe3dartist/portrait_generator_v001/resolve/main/portrait_generator_v001_ema_0.9999_1MM.pt'] },
    'portrait_generator_v002': { 'downloaded': False, 'sha': '3bc39e28fd9690dafbbee83cc08d089a3640eca8ed4d14280c4a9d342c56fd7f', 'uri_list': ['https://huggingface.co/WAS/portrait-diffusion/resolve/main/ema_0.9999_165000.pt'] },
    'portrait_generator_v003': { 'downloaded': False, 'sha': 'c0e1739731efe682f6429f2f7f905104b602b415ef8bf0b507034cce6050e8e3', 'uri_list': ['https://huggingface.co/WAS/portrait-diffusion/resolve/main/ema_0.9999_430000.pt'] },
    'portrait_generator_v004': { 'downloaded': False, 'sha': '09daed35b70670b8491f27ef172e2a62f08f3a78326aeb88d23cade1fbb878ab', 'uri_list': ['https://huggingface.co/WAS/portrait-diffusion/resolve/main/ema_0.9999_080000.pt'] },
    'portrait_generator_v005': { 'downloaded': False, 'sha': '47a9e2cb9ddc9ca9adf9a6bde90c5107c74828b02fe7300bcf0a7d0fb30d5abc', 'uri_list': ['https://huggingface.co/felipe3dartist/Portrait_generator_V2.0/resolve/main/portrait_generator_v2.pt'] },

    'Architecture_Diffusion_1-5m': { 'downloaded': False, 'sha': '9a82579a5490e06fde5ff8ac9a37082f66ce10bb03f434b047596f6526ffd95e', 'uri_list': ['https://huggingface.co/jerostephan/Architecture_Diffusion_1.5M/resolve/main/Architecture_Diffusion_1.5M.pt'] },

    'pixelartdiffusion_expanded': { 'downloaded': False, 'sha': 'a73b40556634034bf43b5a716b531b46fb1ab890634d854f5bcbbef56838739a', 'uri_list': ['https://huggingface.co/KaliYuga/PADexpanded/resolve/main/PADexpanded.pt'] },
    'pixel_art_diffusion_hard_256': { 'downloaded': False, 'sha': 'be4a9de943ec06eef32c65a1008c60ad017723a4d35dc13169c66bb322234161', 'uri_list': ['https://huggingface.co/KaliYuga/pixel_art_diffusion_hard_256/resolve/main/pixel_art_diffusion_hard_256.pt'] },
    'pixel_art_diffusion_soft_256': { 'downloaded': False, 'sha': 'd321590e46b679bf6def1f1914b47c89e762c76f19ab3e3392c8ca07c791039c', 'uri_list': ['https://huggingface.co/KaliYuga/pixel_art_diffusion_soft_256/resolve/main/pixel_art_diffusion_soft_256.pt'] },
    'pixelartdiffusion4k': { 'downloaded': False, 'sha': 'a1ba4f13f6dabb72b1064f15d8ae504d98d6192ad343572cc416deda7cccac30', 'uri_list': ['https://huggingface.co/KaliYuga/pixelartdiffusion4k/resolve/main/pixelartdiffusion4k.pt'] },
    'watercolordiffusion_2': { 'downloaded': False, 'sha': '49c281b6092c61c49b0f1f8da93af9b94be7e0c20c71e662e2aa26fee0e4b1a9', 'uri_list': ['https://huggingface.co/KaliYuga/watercolordiffusion_2/resolve/main/watercolordiffusion_2.pt'] },
    'watercolordiffusion': { 'downloaded': False, 'sha': 'a3e6522f0c8f278f90788298d66383b11ac763dd5e0d62f8252c962c23950bd6', 'uri_list': ['https://huggingface.co/KaliYuga/watercolordiffusion/resolve/main/watercolordiffusion.pt'] },
    'PulpSciFiDiffusion': { 'downloaded': False, 'sha': 'b79e62613b9f50b8a3173e5f61f0320c7dbb16efad42a92ec94d014f6e17337f', 'uri_list': ['https://huggingface.co/KaliYuga/PulpSciFiDiffusion/resolve/main/PulpSciFiDiffusion.pt'] },
 
    'Liminal_Diffusion_v1': { 'downloaded': False, 'sha': '87c36b544a367fceb0ca127d0028cd8a6f6b6e069e529b22999259d69c14f042', 'uri_list': ['https://huggingface.co/BrainArtLabs/liminal_diffusion/resolve/main/liminal_diffusion_v1.pt'] },
    'Liminal_Diffusion_Source': { 'downloaded': False, 'sha': 'ce0064b8cea56c8adb4e5aa0ee2d02f65cd8f1baa905cc6504f462f1aac6d6f4', 'uri_list': ['https://huggingface.co/BrainArtLabs/liminal_diffusion/resolve/main/liminal_diffusion_source.pt'] },
    'Medieval_Diffusion': { 'downloaded': False, 'sha': '1b66a0c9749f88b2d9124af7a3ba5c12d2645ffcc33356326269ccda8643a01b', 'uri_list': ['https://huggingface.co/KaliYuga/medievaldiffusion/resolve/main/medievaldiffusion.pt'] },
    'Lithography_Diffusion': { 'downloaded': False, 'sha': 'a3e6522f0c8f278f90788298d66383b11ac763dd5e0d62f8252c962c23950bd6', 'uri_list': ['https://huggingface.co/KaliYuga/lithographydiffusion/resolve/main/lithographydiffusion.pt'] },
    'Floral_Diffusion': { 'downloaded': False, 'sha': '197e9068f1ca0248fd89f9d6ca1f6f851783e04fddd0cc5b0020bfa655ed3aeb', 'uri_list': ['https://huggingface.co/jags/floraldiffusion/resolve/main/floraldiffusion.pt', 'https://www.dropbox.com/s/i0xrhq28ls1e94g/floraldiffusion.pt'] },
    'FeiArt_Handpainted_CG_Diffusion': { 'downloaded': False, 'sha': '85f95f0618f288476ffcec9f48160542ba626f655b3df963543388dcd059f86a', 'uri_list': ['https://huggingface.co/Feiart/FeiArt-Handpainted-CG-Diffusion/resolve/main/FeiArt-Handpainted-CG-Diffusion.pt'] },
    'Textile_Diffusion': { 'downloaded': False, 'sha': '82aa9ac10c67a806929b5399f04b933ffaa98f1a2bb0c18103d6ccd8f5bd2dd4', 'uri_list': ['https://huggingface.co/KaliYuga/textilediffusion/resolve/main/textilediffusion.pt'] },
    'Isometric_Diffusion_Revrart512px': { 'downloaded': False, 'sha': '649bb7d10ea5170b71bc24adfb17f0305955fde38d2bb1bc5428c6d5baf9811c', 'uri_list': ['https://huggingface.co/Revrart/IsometricDiffusionRevrart512px/resolve/main/IsometricDiffusionRevrart512px.pt'] },
    'Laproper_Diffusion_Deepspace_256': { 'downloaded': False, 'sha': '7c0d742d714ee512edda3299b440f22c72e95fda3f32e24c4ba1f03e0c4b0524', 'uri_list': ['https://huggingface.co/laproper/diffusion-deepspace-256/resolve/main/ema_0.9999_102000.pt']},
    'Schnippi_Diffusion_512x512_V2': { 'downloaded': False, 'sha': '9c111ab89e214862b76e1fa6a1b3f1d329b1a88281885943d2cdbe357ad57648', 'uri_list': ['https://huggingface.co/shnippi/shnippi_diffusion/resolve/main/512x512_shnippi_V2.pt']},

    'secondary': { 'downloaded': False, 'sha': '983e3de6f95c88c81b2ca7ebb2c217933be1973b1ff058776b970f901584613a', 'uri_list': ['https://huggingface.co/spaces/huggi/secondary_model_imagenet_2.pth/resolve/main/secondary_model_imagenet_2.pth', 'https://the-eye.eu/public/AI/models/v-diffusion/secondary_model_imagenet_2.pth', 'https://ipfs.pollinations.ai/ipfs/bafybeibaawhhk7fhyhvmm7x24zwwkeuocuizbqbcg5nqx64jq42j75rdiy/secondary_model_imagenet_2.pth'] },
}

# Why do we have these unused Kaliyuga lists?
kaliyuga_pixel_art_model_names = ['pixelartdiffusion_expanded', 'pixel_art_diffusion_hard_256', 'pixel_art_diffusion_soft_256', 'pixelartdiffusion4k', 'PulpSciFiDiffusion']
kaliyuga_watercolor_model_names = ['watercolordiffusion', 'watercolordiffusion_2']
kaliyuga_pulpscifi_model_names = ['PulpSciFiDiffusion']
diffusion_models_256x256_list = ['256x256_diffusion_uncond'] + kaliyuga_pixel_art_model_names + kaliyuga_watercolor_model_names + kaliyuga_pulpscifi_model_names

# Model Functions

def configurate_model(diffusion_model, use_checkpoint = False):
    model_configurations = {
      '512x512_diffusion_uncond_finetune_008100': {
          'attention_resolutions': '32, 16, 8',
          'class_cond': False,
          'diffusion_steps': 1000,
          'rescale_timesteps': True,
          'timestep_respacing': 250, 
          'image_size': 512,
          'learn_sigma': True,
          'noise_schedule': 'linear',
          'num_channels': 256,
          'num_head_channels': 64,
          'num_res_blocks': 2,
          'resblock_updown': True,
          'use_checkpoint': use_checkpoint,
          'use_fp16': not useCPU,
          'use_scale_shift_norm': True,
      },
      '256x256_diffusion_uncond': {
          'attention_resolutions': '32, 16, 8',
          'class_cond': False,
          'diffusion_steps': 1000, 
          'rescale_timesteps': True,
          'timestep_respacing': 250, 
          'image_size': 512,
          'learn_sigma': True,
          'noise_schedule': 'linear',
          'num_channels': 256,
          'num_head_channels': 64,
          'num_res_blocks': 2,
          'resblock_updown': True,
          'use_checkpoint': use_checkpoint,
          'use_fp16': not useCPU,
          'use_scale_shift_norm': True,
      },
      '512x512_OpenAI': {
          'attention_resolutions': '32, 16, 8',
          'class_cond': False,
          'diffusion_steps': 1000,
          'rescale_timesteps': True,
          'image_size': 512,
          'learn_sigma': True,
          'noise_schedule': 'linear',
          'num_channels': 128,
          'num_heads': 4,
          'num_res_blocks': 2,
          'resblock_updown': True,
          'use_checkpoint': use_checkpoint,
          'use_fp16': True,
          'use_scale_shift_norm': True,
      },
      'Architecture_Diffusion_1-5m': {
          'attention_resolutions': '32, 16, 8',
          'class_cond': False,
          'diffusion_steps': 1000,
          'image_size': 512,
          'learn_sigma': True,
          'noise_schedule': 'linear',
          'num_channels': 256,
          'num_head_channels': 64,
          'num_res_blocks': 2,
          'resblock_updown': True,
          'rescale_timesteps': True,
          'timestep_respacing': '250',
          'use_scale_shift_norm': True
      },
      'Liminal_Diffusion': {
          'attention_resolutions': '16',
          'class_cond': False,
          'diffusion_steps': 1000,
          'rescale_timesteps': True,
          'timestep_respacing': 'ddim100',
          'image_size': 256,
          'learn_sigma': True,
          'noise_schedule': 'linear',
          'num_channels': 128,
          'num_heads': 1,
          'num_res_blocks': 2,
          'use_checkpoint': use_checkpoint,
          'use_fp16': True,
          'use_scale_shift_norm': False,
      },
      'Lithography_Diffusion': {
          'attention_resolutions': '16',
          'class_cond': False,
          'diffusion_steps': 1000,
          'rescale_timesteps': True,
          'timestep_respacing': 'ddim100',
          'image_size': 256,
          'learn_sigma': True,
          'noise_schedule': 'linear',
          'num_channels': 128,
          'num_heads': 1,
          'num_res_blocks': 2,
          'use_checkpoint': use_checkpoint,
          'use_fp16': True,
          'use_scale_shift_norm': False,
      },
      'Medieval_Diffusion': {
          'attention_resolutions': '16',
          'class_cond': False,
          'diffusion_steps': 1000,
          'rescale_timesteps': True,
          'timestep_respacing': 'ddim100',
          'image_size': 256,
          'learn_sigma': True,
          'noise_schedule': 'linear',
          'num_channels': 128,
          'num_heads': 1,
          'num_res_blocks': 2,
          'use_checkpoint': use_checkpoint,
          'use_fp16': True,
          'use_scale_shift_norm': False,
      },
      'Floral_Diffusion': {
          'attention_resolutions': '16',
          'class_cond': False,
          'diffusion_steps': 1000,
          'rescale_timesteps': True,
          'timestep_respacing': 'ddim100',
          'image_size': 256,
          'learn_sigma': True,
          'noise_schedule': 'linear',
          'num_channels': 128,
          'num_heads': 1,        
          'num_res_blocks': 2,
          'use_checkpoint': use_checkpoint,
          'use_fp16': True,
          'use_scale_shift_norm': False,
      },
      'FeiArt_Handpainted_CG_Diffusion': {
          'attention_resolutions': '32,16,8',
          'class_cond': False,
          'diffusion_steps': 1000,
          'rescale_timesteps': True,
          'timestep_respacing': 'ddim100',
          'image_size': 512,
          'learn_sigma': True,
          'noise_schedule': 'linear',
          'num_channels': 256,
          'num_head_channels': 64,
          'num_res_blocks': 2,
          'resblock_updown': True,
          'use_checkpoint': use_checkpoint,
          'use_fp16': True,
          'use_scale_shift_norm': True,
      },
      'Textile_Diffusion': {
          'attention_resolutions': '16',
          'class_cond': False,
          'diffusion_steps': 1000,
          'rescale_timesteps': True,
          'timestep_respacing': 'ddim100',
          'image_size': 256,
          'learn_sigma': True,
          'noise_schedule': 'linear',
          'num_channels': 128,
          'num_heads': 1,
          'num_res_blocks': 2,
          'use_checkpoint': use_checkpoint,
          'use_fp16': True,
          'use_scale_shift_norm': False,
      },
      'Isometric_Diffusion_Revrart512px': {
          'attention_resolutions': '32, 16, 8',
          'class_cond': False,
          'diffusion_steps': 1000,
          'rescale_timesteps': True,
          'timestep_respacing': 'ddim100',
          'image_size': 512,
          'learn_sigma': True,
          'noise_schedule': 'linear',
          'num_channels': 256,
          'num_head_channels': 64,
          'num_res_blocks': 2,
          'resblock_updown': True,
          'use_checkpoint': use_checkpoint,
          'use_fp16': True,
          'use_scale_shift_norm': True,
      }, 
      'Laproper_Diffusion_Deepspace_256': {
          'attention_resolutions': '32, 16, 8',
          'class_cond': False,
          'diffusion_steps': 1000,
          'rescale_timesteps': True,
          'timestep_respacing': 'ddim100',
          'image_size': 256,
          'learn_sigma': True,
          'noise_schedule': 'linear',
          'num_channels': 128,
          'num_heads': 4,
          'num_res_blocks': 2,
          'resblock_updown': True,
          'use_checkpoint': use_checkpoint,
          'use_fp16': True,
          'use_scale_shift_norm': True,
      },
      'Schnippi_Diffusion_512x512_V2': {
          'attention_resolutions': '32, 16, 8',
          'class_cond': False,
          'diffusion_steps': 1000, 
          'rescale_timesteps': True,
          'timestep_respacing': 250, 
          'image_size': 512,
          'learn_sigma': True,
          'noise_schedule': 'linear',
          'num_channels': 256,
          'num_head_channels': 64,
          'num_res_blocks': 2,
          'resblock_updown': True,
          'use_checkpoint': use_checkpoint,
          'use_fp16': not useCPU,
          'use_scale_shift_norm': True,
      },
      'Kaliyuga': {
          'attention_resolutions': '16',
          'class_cond': False,
          'diffusion_steps': 1000,
          'rescale_timesteps': True,
          'timestep_respacing': 'ddim100',
          'image_size': 256,
          'learn_sigma': True,
          'noise_schedule': 'linear',
          'num_channels': 128,
          'num_heads': 1,
          'num_res_blocks': 2,
          'use_checkpoint': use_checkpoint,
          'use_fp16': True,
          'use_scale_shift_norm': False,
        },
    }

    tmp_model_config = model_and_diffusion_defaults()

    if diffusion_model in ['512x512_diffusion_uncond_finetune_008100', 
                           'Schnippi_Diffusion_512x512_V2']:
        tmp_model_config.update(model_configurations['512x512_diffusion_uncond_finetune_008100'])
    elif diffusion_model == '256x256_diffusion_uncond':
        tmp_model_config.update(model_configurations['256x256_diffusion_uncond'])
    elif diffusion_model in ['concept_art_generator_v000-1_alpha',
                             'concept_art_generator_v000-2_alpha',
                             'portrait_generator_v001', 
                             'portrait_generator_v002', 
                             'portrait_generator_v003', 
                             'portrait_generator_v004',
                             'portrait_generator_v005']:
        tmp_model_config.update(model_configurations['512x512_OpenAI'])
    elif diffusion_model == 'Architecture_Diffusion_1-5m':
        tmp_model_config.update(model_configurations['Architecture_Diffusion_1-5m'])
    elif diffusion_model in ['Liminal_Diffusion_v1', 
                             'liminal_diffusion_source']:
        tmp_model_config.update(model_configurations['Liminal_Diffusion'])
    elif diffusion_model == 'Lithography_Diffusion': 
        tmp_model_config.update(model_configurations['Lithography_Diffusion'])
    elif diffusion_model == 'Medieval_Diffusion': 
        tmp_model_config.update(model_configurations['Medieval_Diffusion'])
    elif diffusion_model == 'Floral_Diffusion': 
        tmp_model_config.update(model_configurations['Floral_Diffusion'])
    elif diffusion_model == 'FeiArt_Handpainted_CG_Diffusion':
        tmp_model_config.update(model_configurations['FeiArt_Handpainted_CG_Diffusion'])
    elif diffusion_model == 'Textile_Diffusion':
        tmp_model_config.update(model_configurations['Textile_Diffusion'])
    elif diffusion_model == 'Isometric_Diffusion_Revrart512px':
        tmp_model_config.update(model_configurations['Isometric_Diffusion_Revrart512px'])
    elif diffusion_model == 'Laproper_Diffusion_Deepspace_256':
        tmp_model_config.update(model_configurations['Laproper_Diffusion_Deepspace_256'])
    else:
        tmp_model_config.update(model_configurations['Kaliyuga'])

    return tmp_model_config

def get_model_filename(diffusion_model_name):
  model_uri = diff_model_map[diffusion_model_name]['uri_list'][0]
  model_filename = os.path.basename(urlparse(model_uri).path)
  return model_filename

def check_sha_hash(diffusion_model_name, model_local_path):
  with open(model_local_path, "rb") as f: return True if hashlib.sha256(f.read()).hexdigest() == diff_model_map[diffusion_model_name]['sha'] else False

def download_model(diffusion_model_name, uri_index=0):
  if diffusion_model_name is 'custom':
    return
  model_filename = get_model_filename(diffusion_model_name)
  model_local_path = os.path.join(model_path, model_filename)
  print("Downloading Diffusion Model: ", diffusion_model_name)
  if os.path.exists(model_local_path):
    if check_model_SHA:
      print(f"Checking {model_filename} SHA256 Hash...")
      if check_sha_hash(diffusion_model_name, model_local_path):
        print(f'{diffusion_model_name} SHA256 matches.')
        diff_model_map[diffusion_model_name]['downloaded'] = True
      else:
        print(f"{diffusion_model_name} SHA doesn't match. Will redownload it...")
        os.remove(model_local_path)
        return download_model(diffusion_model_name)
    print(f"{model_filename} \033[32malready downloaded.\033[0;0m")
  else:
    for model_uri in diff_model_map[diffusion_model_name]['uri_list']:
      print(f"Downloading {diffusion_model_name} from: {model_uri}")
      #!wget -q --show-progress --no-cache $model_uri -P $model_path
      wget(model_uri, model_path)
      if os.path.exists(model_local_path):
        if check_model_SHA:
          print(f"Checking {model_filename} SHA256 Hash...")
          if check_sha_hash(diffusion_model_name, model_local_path):
            print(f'{diffusion_model_name} SHA256 matches.')
            diff_model_map[diffusion_model_name]['downloaded'] = True
            print(f"{diffusion_model_name} \033[32mdownloaded successfully.\033[0;0m")
            return
          else:
            print(f"{diffusion_model_name} \033[31mSHA doesn't match!\033[0;0m Attempting to redownload")
            os.remove(model_local_path)
            return download_model(diffusion_model_name)
        else:
          diff_model_map[diffusion_model_name]['downloaded'] = True
          print(f"{diffusion_model_name} \033[32mdownloaded successfully.\033[0;0m")
          return
      else:
        print(f"\033[31mFailed to download\033[0;0m {diffusion_model_name} from: {model_uri}")
        if len(diff_model_map[diffusion_model_name]['uri_list']) > 1:
          print("Attempting to download from another source...")
        else:
          print(f"No additional download resources available for {diffusion_model_name}")


# Download the diffusion model(s)
download_model(diffusion_model)
if use_secondary_model:
    download_model('secondary')
if go_big_diffusion_model != "Inherit":
    download_model(go_big_diffusion_model)

# Configurate diffusion model
if diffusion_model is not 'custom':
    model_config = configurate_model(diffusion_model, use_checkpoint)
if go_big_diffusion_model is not ['Inherit', 'custom']:
    go_big_model_config = configurate_model(go_big_diffusion_model, use_checkpoint)
if use_secondary_model:
    secondary_model = SecondaryDiffusionImageNet2()
    secondary_model.load_state_dict(torch.load(f'{model_path}/secondary_model_imagenet_2.pth', map_location='cpu'))
    secondary_model.eval().requires_grad_(False).to(device)

clip_models = []
if ViTB32: print("Downloading CLIP Model: ViT-B/32"); clip_models.append(clip.load('ViT-B/32', jit=False)[0].eval().requires_grad_(False).to(device))
if ViTB16: print("Downloading CLIP Model: ViT-B/16"); clip_models.append(clip.load('ViT-B/16', jit=False)[0].eval().requires_grad_(False).to(device))
if ViTL14: print("Downloading CLIP Model: ViT-L/14"); clip_models.append(clip.load('ViT-L/14', jit=False)[0].eval().requires_grad_(False).to(device))
if ViTL14_336px: print("Downloading CLIP Model: ViT-L/14@336px"); clip_models.append(clip.load('ViT-L/14@336px', jit=False)[0].eval().requires_grad_(False).to(device))
if RN50: print("Downloading CLIP Model: RN50"); clip_models.append(clip.load('RN50', jit=False)[0].eval().requires_grad_(False).to(device))
if RN50x4: print("Downloading CLIP Model: RN50x4"); clip_models.append(clip.load('RN50x4', jit=False)[0].eval().requires_grad_(False).to(device))
if RN50x16: print("Downloading CLIP Model: RN50x16"); clip_models.append(clip.load('RN50x16', jit=False)[0].eval().requires_grad_(False).to(device))
if RN50x64: print("Downloading CLIP Model: RN50x64"); clip_models.append(clip.load('RN50x64', jit=False)[0].eval().requires_grad_(False).to(device))
if RN101: print("Downloading CLIP Model: RN101");clip_models.append(clip.load('RN101', jit=False)[0].eval().requires_grad_(False).to(device))

if ViTB32_laion2b_e16: print("Downloading CLIP Model: ViT-B/32 laion2b_e16"); clip_models.append(open_clip.create_model('ViT-B-32', pretrained='laion2b_e16').eval().requires_grad_(False).to(device))
if ViTB32_laion400m_e31: print("Downloading CLIP Model: ViT-B-32 laion400m_e31"); clip_models.append(open_clip.create_model('ViT-B-32', pretrained='laion400m_e31').eval().requires_grad_(False).to(device))
if ViTB32_laion400m_32: print("Downloading CLIP Model: ViT-B/32 laion400m_e32"); clip_models.append(open_clip.create_model('ViT-B-32', pretrained='laion400m_e32').eval().requires_grad_(False).to(device))
if ViTB32quickgelu_laion400m_e31: print("Downloading CLIP Model: ViT-B-32-quickgelu laion400m_e31"); clip_models.append(open_clip.create_model('ViT-B-32-quickgelu', pretrained='laion400m_e31').eval().requires_grad_(False).to(device))
if ViTB32quickgelu_laion400m_e32: print("Downloading CLIP Model: ViT-B-32-quickgelu laion400m_e32"); clip_models.append(open_clip.create_model('ViT-B-32-quickgelu', pretrained='laion400m_e32').eval().requires_grad_(False).to(device))
if ViTB16_laion400m_e31: print("Downloading CLIP Model: ViT-B-16 laion400m_e31"); clip_models.append(open_clip.create_model('ViT-B-16', pretrained='laion400m_e31').eval().requires_grad_(False).to(device))
if ViTB16_laion400m_e32: print("Downloading CLIP Model: ViT-B-16 laion400m_e32"); clip_models.append(open_clip.create_model('ViT-B-16', pretrained='laion400m_e32').eval().requires_grad_(False).to(device))
if RN50_yffcc15m: print("Downloading CLIP Model: RN50 yfcc15m"); clip_models.append(open_clip.create_model('RN50', pretrained='yfcc15m').eval().requires_grad_(False).to(device))
if RN50_cc12m: print("Downloading CLIP Model: RN50 yfcc15m"); clip_models.append(open_clip.create_model('RN50', pretrained='cc12m').eval().requires_grad_(False).to(device))
if RN50_quickgelu_yfcc15m: print("Downloading CLIP Model: RN50-quickgelu yfcc15m"); clip_models.append(open_clip.create_model('RN50-quickgelu', pretrained='yfcc15m').eval().requires_grad_(False).to(device))
if RN50_quickgelu_cc12m: print("Downloading CLIP Model: RN50-quickgelu cc12m"); clip_models.append(open_clip.create_model('RN50-quickgelu', pretrained='cc12m').eval().requires_grad_(False).to(device))
if RN101_yfcc15m: print("Downloading CLIP Model: RN101 yfcc15m"); clip_models.append(open_clip.create_model('RN101', pretrained='yfcc15m').eval().requires_grad_(False).to(device))
if RN101_quickgelu_yfcc15m: print("Downloading CLIP Model: RN101=quickgelu yfcc15m"); clip_models.append(open_clip.create_model('RN101-quickgelu', pretrained='yfcc15m').eval().requires_grad_(False).to(device))

normalize = T.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
lpips_model = lpips.LPIPS(net='vgg').to(device)

# <a name="reference_guide">Reference</a> ([View on Github](https://github.com/WASasquatch/discostream/wiki/Quick-Reference-Wiki))



**Diffusion Settings Reference**
---
Disco Diffusion is complex, and continually evolving with new features.  The most current documentation on on Disco Diffusion settings can be found in the unofficial guidebook:

[Zippy's Disco Diffusion Cheatsheet](https://docs.google.com/document/d/1l8s7uS2dGqjztYSjPpzlmXLjl5PM3IGkRWI3IiCuK7g/edit)

We also encourage users to join the [Disco Diffusion User Discord](https://discord.gg/XGZrFFCRfN) to learn from the active user community.

This section below is outdated as of v2

Setting | Description | Default
--- | --- | ---
**Your vision:**
`text_prompts` | A description of what you'd like the machine to generate. Think of it like writing the caption below your image on a website. | `N/A`
`image_prompts` | Think of these images more as a description of their contents. | `N/A`
**Image quality:**
`clip_guidance_scale`  | Controls how much the image should look like the prompt. | `1000`
`tv_scale` | Controls the smoothness of the final output. | `150`
`range_scale` | Controls how far out of range RGB values are allowed to be. | `150`
`sat_scale` | Controls how much saturation is allowed. From nshepperd's JAX notebook. | `0`
`cutn` | Controls how many crops to take from the image. | `16`
`cutn_batches` | Accumulate CLIP gradient from multiple batches of cuts. | `2`
**Go BIG Settings:**
`go_big` | A boolean to enable Go BIG tiled upscale diffusion. | `False`
`go_big_scale` | The Go BIG multiplier factor | `2`+
`gobig_skip_perent` | Percentage of steps to skip when doing Go BIG tiled diffusion runs. | `85`
`go_big_init` | A path or URL to a Go BIG init image. This will skip the diffusion run. | `None`
`go_big_init_skipresize` | Whether to skip resizing the go BIG init image | `False`
`render_mask` | A alpha mask of areas to diffuse (white), and not diffuse (black) | `None`
`init_masked` | A alpha mask that can compliement the render mask in Go BIG runs. | `None`
**Init settings:**
`init_image` | URL or local path | `None`
`init_scale` | This enhances the effect of the init image, a good value is 1000 | `0`
`skip_steps` | Controls the starting point along the diffusion timesteps | `0`
`perlin_init` | Option to start with random perlin noise | `False`
`perlin_mode` | (`gray`, `color`) | `mixed`
**Advanced:**
`skip_augs` | Controls whether to skip torchvision augmentations | `False`
`randomize_class` | Controls whether the imagenet class is randomly changed each iteration | `True`
`clip_denoised` | Determines whether CLIP discriminates a noisy or denoised image | `False`
`clamp_grad` | Experimental: Using adaptive clip grad in the cond_fn | `True`
`seed`  | Choose a random seed and print it at end of run for reproduction | `random_seed`
`fuzzy_prompt` | Controls whether to add multiple noisy prompts to the prompt losses | `False`
`rand_mag` | Controls the magnitude of the random noise | `0.1`
`eta` | DDIM hyperparameter | `0.5`
`use_vertical_symmetry` | Enforce symmetry over x axis of the image on [`tr_st`*`steps` for `tr_st` in `transformation_steps`] steps of the diffusion process | `False`
`use_horizontal_symmetry` | Enforce symmetry over y axis of the image on [`tr_st`*`steps` for `tr_st` in `transformation_steps`] steps of the diffusion process | `False`
`transformation_steps` | Steps (expressed in percentages) in which the symmetry is enforced | `[0.01]`
`video_init_flow_warp` | Flow warp enabled | True
`video_init_flow_blend` | 0 - you get raw input, 1 - you get warped diffused previous frame  | `0.999`
`video_init_check_consistency` | TBD check forward-backward flow consistency (uncheck unless there are too many warping artifacts) | `False`

<br>

<font size="5">**Model settings**</font>
<hr>

Setting | Description | Default
--- | --- | ---
**Diffusion:**
`timestep_respacing` | Modify this value to decrease the number of timesteps. | ddim100
`diffusion_steps` || 1000
**Diffusion:**
`clip_models` | Models of CLIP to load. Typically the more, the better but they all come at a hefty VRAM cost. | ViT-B/32, ViT-B/16, RN50x4

## Noodle Soup Prompts

To use a term database, simply use one of keys below. 

For example if you wanted beauty adjective, you would write `_adj-beauty_` in your prompt. 

## Terminology Keys

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Adjective Types
   - `_adj-architecture_` - A list of architectural adjectives and styles
   - `_adj-beauty_` - A list of beauty adjectives for people (maybe things?)
   - `_adj-general_` - A list of general adjectives for people/things.
   - `_adj-horror_` - A list of horror adjectives
### Art Types
   - `_artist_` - A comprehensive list of artists by **MisterRuffian** <font size="2">(<font color="7e87f6">Discord</font> Misterruffian#2891)</font>
   - `_color_` - A comprehensive list of colors
   - `_portrait-type_` - A list of common portrait types/poses
   - `_style_` - A list of art styles and mediums
### Computer Graphics Types
   - `_3d-terms_` - A list of 3D graphics terminology
   - `_color-palette_` - A list of computer and video game console color palettes
   - `_hd_` - A list of high definition resolution terms
### Miscellaneous Types
   - `_details_` - A list of detail descriptors
   - `_site_` - A list of websites to query
   - `_gen-modififer_` - A list of general modifiers adopted from [Weird Wonderful AI Art](https://weirdwonderfulai.art/)
   - `_neg-weight_` - A lsit of negative weight ideas
   - `_punk_` - A list of punk modifier (eg. cyberpunk)
   - ` _pop-culture_` - A list of popular culture movies, shows, etc
   - `_pop-location_` - A list of popular tourist locations
   - `_fantasy-setting_` - A list of fantasy location settings
   - `_fantasy-creature_` - A list of fantasy creatures
### Noun Types
   - `_noun-beauty_` - A list of beauty related nouns
   - `_noun-emote_` - A list of emotions and expressions
   - `_noun-fantasy_` - A list of fantasy nouns
   - `_noun-general_` - A list of general nouns
   - `_noun-horror_` - A list of horror nouns
### People Types
   - `_bodyshape_` - A list of body shapes
   - `_celeb_` - A list of celebrities
   - `_eyecolor_` - A list of eye colors
   - `_hair_` - A list of hair types
   - `_nationality_` - A list of nationalities
   - `_occputation_` A list of occupation types
   - `_skin-color_` - A list of skin tones
   - `_identity-young_` A list of young identifiers
   - `_identity-adult_` A list of adult identifiers
   - `_identity_` A list of general identifiers
### Photography / Image / Film Types
   - `_aspect-ratio_` - A list of common aspect ratios
   - `_cameras_` - A list of camera models *(including manufactuerer)*
   - `_camera-manu_` - A list of camera manufacturers
   - `_f-stop_` - A list of camera aperture f-stop
   - `_focal-length_` - A list of focal length ranges
   - `_photo-term_` - A list of photography terms relating to photos

<br>

## **Easing Function Aliases**
 - Easing Integer: `easeint(st=40, en=1, s=800, fn=cut_easing_fn)` 
 - Easing Float: `easeflt(st=40, en=1, s=800, fn=cut_easing_fn)`
<br><br>

# Custom model settings 
Modify in accordance with your training settings and run the cell

In [ ]:
#@markdown ####**Custom Model Settings:**

#@markdown This is only relevant if you're using a "custom" model. Define these settings according to your custom model.

# Default Settings
custom_attention_resolutions = '32, 16, 8' #@param{type: 'string'}
custom_class_cond = False #@param{type: 'boolean'}
custom_diffusion_steps = 1000 #@param{type: 'number'}
custom_rescale_timesteps = True #@param{type: 'boolean'}
custom_image_size = 512 #@param{type: 'number'}
custom_learn_sigma = True #@param{type: 'boolean'}
custom_noise_schedule = 'linear' #@param{type: 'string'}
custom_num_channels = 128 #@param{type: 'number'}
custom_num_heads = 4 #@param{type: 'number'}
custom_num_res_blocks = 2 #@param{type: 'number'}
custom_resblock_updown = True #@param{type: 'boolean'}
custom_use_checkpoint = True #@param{type: 'boolean'}
custom_use_fp16 = True #@param{type: 'boolean'}
custom_use_scale_shift_norm = True #@param{type: 'boolean'}
#@markdown <br>

custom_model_config = {
          'attention_resolutions': custom_attention_resolutions,
          'class_cond': custom_class_cond,
          'diffusion_steps': custom_diffusion_steps,
          'rescale_timesteps': custom_rescale_timesteps,
          'image_size': custom_image_size,
          'learn_sigma': custom_learn_sigma,
          'noise_schedule': custom_noise_schedule,
          'num_channels': custom_num_channels,
          'num_heads': custom_num_heads,
          'num_res_blocks': custom_num_res_blocks,
          'resblock_updown': custom_resblock_updown,
          'use_checkpoint': custom_use_checkpoint,
          'use_fp16': custom_use_fp16,
          'use_scale_shift_norm': custom_use_scale_shift_norm,
      }

custom_model_settings_override = "" #@param{type: 'string'}
#@markdown Here you can paste the dictionary settings of your model.
#@markdown **Example:**
#@markdown `{ 'attention_resolutions': '32, 16, 8', 'class_cond': False, 'diffusion_steps': 1000, 'rescale_timesteps': True, 'image_size': 512, 'learn_sigma': True, 'noise_schedule': 'linear', 'num_channels': 128, 'num_heads': 4, 'num_res_blocks': 2, 'resblock_updown': True, 'use_checkpoint': True, 'use_fp16': True, 'use_scale_shift_norm': True }`
if custom_model_settings_override.lower() not in [None, "none", ""]:
    custom_model_config = eval(custom_model_settings_override)

custom_model = model_and_diffusion_defaults()
if diffusion_model is 'custom' or go_big_diffusion_model is 'custom':
    custom_model.update(custom_model_config)

# 3. Settings

In [ ]:
#@markdown ##**Basic Settings:** <br><br>

#@markdown ### **Basic Diffusion Settings** 
batch_name = 'DiscoStreamIt' #@param{type: 'string'}
steps =  250#@param [25,50,100,150,250,500,1000]{type: 'raw', allow-input: true}
width_height_for_512x512_models = [512,768] #@param{type: 'raw'}
clip_guidance_scale =  1500#@param{type: 'number'}
tv_scale = 1000#@param{type: 'number'}
range_scale = 150#@param{type: 'number'}
sat_scale = 0#@param{type: 'number'}
cutn_batches = 1#@param{type: 'number'}
skip_augs = False#@param{type: 'boolean'}

#@markdown ####**Image dimensions to be used for 256x256 models (e.g. pixelart models):**
width_height_for_256x256_models = [256,256] #@param{type: 'raw'}
#@markdown --- 
#@markdown <br><br>

#@markdown ### **Go <font color="00a000">BIG</font> Settings**
#@markdown #### <font size="4" color="CC0000">*WARNING*</font>*: go_big_scale is a multipler* - if you go above **4** you might be waiting a long time
#@markdown **NOTE**: If you have go_big_init set it will start with that image and skip the initial render

go_big = False #@param{type: 'boolean'}
go_big_scale = 2 #@param{type: 'number'}
gobig_skip_percent = 80#@param{type: 'number'}
gobig_overlap = 64 #@param{type: 'number'}
gobig_maximize = False #@param{type: 'boolean'}
#@markdown (maximize extends the gobig canvas to use areas that would otherwise be wasted, resulting in a larger final image at no additional render time cost)
go_big_init = None #@param{type: 'string'}
#@markdown Example: /content/drive/MyDrive/AI/Disco_Diffusion/init_images/image.png
go_big_init_skipresize = False #@param{type: 'boolean'}
#@markdown Check the above if you already resized the image. Please make sure go_big_scale is set to the multiplier you used.
render_mask = None #@param{type: 'string'}
#@markdown render_mask is a grayscale image that tells us where to draw (white) and not draw (black).
#@markdown useful when using an init for go_big to avoid adding unwanted details in empty areas
init_masked = None #@param{type: 'string'}
#@markdown init_masked is an alternate init image that can be used in conjunction with render_mask.
#@markdown When using a render_mask, the white areas will be rendered with init_masked instead of init_image


#@markdown --- 
#@markdown <br><br>

#@markdown ###**Video Init Basic Settings:**
video_init_steps = 100 #@param [25,50,100,150,250,500,1000]{type: 'raw', allow-input: true}
video_init_clip_guidance_scale = 1000 #@param{type: 'number'}
video_init_tv_scale = 0.1#@param{type: 'number'}
video_init_range_scale = 150#@param{type: 'number'}
video_init_sat_scale = 300#@param{type: 'number'}
video_init_cutn_batches = 4#@param{type: 'number'}
video_init_skip_steps = 50 #@param{type: 'integer'}

#@markdown ---
#@markdown <br><br>

#@markdown ###**Init Image Settings:**
#@markdown Init Image is either a string, or list, you must include the correct syntax.<br>**Example:**<br> - `None` (capitalized)<br> - `"path/to/my/init_image.png"` (with quotation marks)<br> - `["path/to/my/init_image1.png", "path/to/my/init_image2.png"]` (with brackets, quotation marks, and commas)
init_image = None #@param{type: 'raw'}
init_scale = 0 #@param{type: 'integer'}
skip_steps =  0 #@param{type: 'integer'}
#@markdown *Make sure you set skip_steps to ~50% of your steps if you want to use an init image.* 
#@markdown <br><br>
#@markdown ###**Init Image Prep**
#@markdown Prep a init_image for easier improvisation by guided diffusion.
do_prep_init_image = False #@param{type: 'boolean'}
do_go_big_slice_prep = False #@param{type: 'boolean'}
gaussian_blur_radius = 2.5 #@param{type: 'number'}
#@markdown Gaussian Blur Radius in Pixels
gaussian_noise_amount = 1 #@param{type: 'number'}
#@markdown Gaussian Noise Amount: `0` - `1`+
gaussian_noise_blur_radius = 1.25 #@param{type: 'number'}
#@markdown Gaussiean Noise Blur Radius in Pixels
gaussian_noise_greyscale = False #@param{type: 'boolean'}
#@markdown Gaussian Noise Greyscale Mode
gaussian_noise_double = False #@param{type: 'boolean'}
#@markdown Double the Gaussian Noise amount (good for low noise level)
do_preview_init_prep = True #@param{type: 'boolean'}
#@markdown Do not display init prep previews

width_height = width_height_for_256x256_models if diffusion_model in diffusion_models_256x256_list else width_height_for_512x512_models

#Get corrected sizes
side_x = (width_height[0]//64)*64;
side_y = (width_height[1]//64)*64;
if side_x != width_height[0] or side_y != width_height[1]:
    print(f'Changing output size to {side_x}x{side_y}. Dimensions must by multiples of 64.')

#Make folder for batch
batchFolder = f'{outDirPath}/{batch_name}'
createPath(batchFolder)

if go_big_init != None and go_big_init.lower() in ['none','']:
    go_big_init = None
if render_mask != None and render_mask.lower() in ['none','']:
    render_mask = None
if init_masked != None and init_masked.lower() in ['none','']:
    init_masked = None
if init_image != None and ( type(init_image) is not list and init_image.lower() in ['none',''] ):
    init_image = None
elif init_image != None and type(init_image) is list:
    new_init_list = []
    for init in init_image:
        if init != None and init.lower() not in ['none','']:
            new_init_list.append(init)
    if new_init_list:
        init_image = new_init_list

if skip_steps >= steps:
    raise Exception(f"Skip steps cannot equal, or exceed steps. Invalid value: {skip_steps} where steps: {steps}")

if not install_go_big_v2_support and go_big:
    print(pil_warning+"\n")
    print("\33[31mDisabling Go BIG!\33[0m")
    go_big = False

# Go BIG Calculations
if go_big:

  if go_big_init_skipresize == True and go_big_init == None:
    go_big_init_skipresize = False
    raise Exception('You enabled skipresize but did not provide a go_big_init image')

  if go_big_init != None:
    print(f'side_x was {side_x}, side_y was {side_y}')
    print(f'width_height was {width_height}')
    try:
      temp_image = PIL.Image.open(fetch(go_big_init)).convert("RGB")
    except AttributeError as e:
        if "'PIL.TiffTags' has no attribute 'IFD'" in str(e):
          raise Exception("\33[33m\u001b[4mWARNING:\33[0m \033[1mPIL.TiffTags requires a restart. Please select Runtime -> Restart Runtime, and then run all the cells again.\033[0m")
          
    side_x, side_y = temp_image.size
    print(f'side_x is now {side_x}, side_y is now {side_y}')
    width_height[0] = side_x
    width_height[1] = side_y
    print(f'width_height is now {width_height}')
    temp_image.close
    side_x = (width_height[0]//64)*64;
    side_y = (width_height[1]//64)*64;
    if side_x != width_height[0] or side_y != width_height[1]:
      print('ERROR: Your go big init resolution was NOT a multiple of 64.')
      raise Exception('Exiting due to improperly sized go big init.')

if do_prep_init_image and do_preview_init_prep:
    if init_image is not None:
        print(":hourglass_not_done: Loading preview(s)...", LINE_UP, end=LINE_CLEAR, flush=True)
        
        if type(init_image) is str:
            temp_init = init_image_prep(init_image, gaussian_blur_radius, gaussian_noise_amount, gaussian_noise_blur_radius, gaussian_noise_greyscale, gaussian_noise_double)
            displayJsImage('', temp_init)
        elif type(init_image) is list:
            for init in init_image:
                temp_init = init_image_prep(init, gaussian_blur_radius, gaussian_noise_amount, gaussian_noise_blur_radius, gaussian_noise_greyscale, gaussian_noise_double)
                displayJsImage(os.path.basename(init), temp_init)
        print(":check_mark_button:")




### Animation Settings

In [ ]:
#@markdown ####**Animation Mode:**
animation_mode = 'None' #@param ['None', '2D', '3D', 'Video Input'] {type:'string'}
#@markdown *For animation, you probably want to turn `cutn_batches` to 1 to make it quicker.*


#@markdown ---

#@markdown ####**Video Input Settings:**
if is_colab:
    video_init_path = "/content/drive/MyDrive/init.mp4" #@param {type: 'string'}
else:
    video_init_path = "init.mp4" #@param {type: 'string'}
extract_nth_frame = 2 #@param {type: 'number'}
persistent_frame_output_in_batch_folder = True #@param {type: 'boolean'}
video_init_seed_continuity = False #@param {type: 'boolean'}
#@markdown #####**Video Optical Flow Settings:**
video_init_flow_warp = True #@param {type: 'boolean'}
# Call optical flow from video frames and warp prev frame with flow
video_init_flow_blend =  0.999#@param {type: 'number'} #0 - take next frame, 1 - take prev warped frame
video_init_check_consistency = False #Insert param here when ready
video_init_blend_mode = "optical flow" #@param ['None', 'linear', 'optical flow']
# Call optical flow from video frames and warp prev frame with flow
if animation_mode == "Video Input":
    if persistent_frame_output_in_batch_folder or (not is_colab): #suggested by Chris the Wizard#8082 at discord
        videoFramesFolder = f'{batchFolder}/videoFrames'
    else:
        videoFramesFolder = f'/content/videoFrames'
    createPath(videoFramesFolder)
    print(f"Exporting Video Frames (1 every {extract_nth_frame})...")
    try:
        for f in pathlib.Path(f'{videoFramesFolder}').glob('*.jpg'):
            f.unlink()
    except:
        print('')
    vf = f'select=not(mod(n\,{extract_nth_frame}))'
    if os.path.exists(video_init_path):
        subprocess.run(['ffmpeg', '-i', f'{video_init_path}', '-vf', f'{vf}', '-vsync', 'vfr', '-q:v', '2', '-loglevel', 'error', '-stats', f'{videoFramesFolder}/%04d.jpg'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    else: 
        print(f'\nWARNING!\n\nVideo not found: {video_init_path}.\nPlease check your video path.\n')
    #!ffmpeg -i {video_init_path} -vf {vf} -vsync vfr -q:v 2 -loglevel error -stats {videoFramesFolder}/%04d.jpg


#@markdown ---

#@markdown ####**2D Animation Settings:**
#@markdown `zoom` is a multiplier of dimensions, 1 is no zoom.
#@markdown All rotations are provided in degrees.

key_frames = True #@param {type:"boolean"}
max_frames = 10000#@param {type:"number"}

if animation_mode == "Video Input":
    max_frames = len(glob(f'{videoFramesFolder}/*.jpg'))

interp_spline = 'Linear' #Do not change, currently will not look good. param ['Linear','Quadratic','Cubic']{type:"string"}
angle = "0:(0)"#@param {type:"string"}
zoom = "0: (1), 10: (1.05)"#@param {type:"string"}
translation_x = "0: (0)"#@param {type:"string"}
translation_y = "0: (0)"#@param {type:"string"}
translation_z = "0: (10.0)"#@param {type:"string"}
rotation_3d_x = "0: (0)"#@param {type:"string"}
rotation_3d_y = "0: (0)"#@param {type:"string"}
rotation_3d_z = "0: (0)"#@param {type:"string"}
midas_depth_model = "dpt_large"#@param {type:"string"}
midas_weight = 0.3#@param {type:"number"}
near_plane = 200#@param {type:"number"}
far_plane = 10000#@param {type:"number"}
fov = 40#@param {type:"number"}
padding_mode = 'border'#@param {type:"string"}
sampling_mode = 'bicubic'#@param {type:"string"}

#======= TURBO MODE
#@markdown ---
#@markdown ####**Turbo Mode (3D anim only):**
#@markdown (Starts after frame 10,) skips diffusion steps and just uses depth map to warp images for skipped frames.
#@markdown Speeds up rendering by 2x-4x, and may improve image coherence between frames.
#@markdown For different settings tuned for Turbo Mode, refer to the original Disco-Turbo Github: https://github.com/zippy731/disco-diffusion-turbo

turbo_mode = False #@param {type:"boolean"}
turbo_steps = "3" #@param ["2","3","4","5","6"] {type:"string"}
turbo_preroll = 10 # frames

#insist turbo be used only w 3d anim.
if turbo_mode and animation_mode != '3D':
    print('=====')
    print('Turbo mode only available with 3D animations. Disabling Turbo.')
    print('=====')
    turbo_mode = False

#@markdown ---

#@markdown ####**Coherency Settings:**
#@markdown `frame_scale` tries to guide the new frame to looking like the old one. A good default is 1500.
frames_scale = 1500 #@param{type: 'integer'}
#@markdown `frame_skip_steps` will blur the previous frame - higher values will flicker less but struggle to add enough new detail to zoom into.
frames_skip_steps = '60%' #@param ['40%', '50%', '60%', '70%', '80%'] {type: 'string'}

#@markdown ####**Video Init Coherency Settings:**
#@markdown `frame_scale` tries to guide the new frame to looking like the old one. A good default is 1500.
video_init_frames_scale = 15000 #@param{type: 'integer'}
#@markdown `frame_skip_steps` will blur the previous frame - higher values will flicker less but struggle to add enough new detail to zoom into.
video_init_frames_skip_steps = '70%' #@param ['40%', '50%', '60%', '70%', '80%'] {type: 'string'}

#======= VR MODE
#@markdown ---
#@markdown ####**VR Mode (3D anim only):**
#@markdown Enables stereo rendering of left/right eye views (supporting Turbo) which use a different (fish-eye) camera projection matrix.   
#@markdown Note the images you're prompting will work better if they have some inherent wide-angle aspect
#@markdown The generated images will need to be combined into left/right videos. These can then be stitched into the VR180 format.
#@markdown Google made the VR180 Creator tool but subsequently stopped supporting it. It's available for download in a few places including https://www.patrickgrunwald.de/vr180-creator-download
#@markdown The tool is not only good for stitching (videos and photos) but also for adding the correct metadata into existing videos, which is needed for services like YouTube to identify the format correctly.
#@markdown Watching YouTube VR videos isn't necessarily the easiest depending on your headset. For instance Oculus have a dedicated media studio and store which makes the files easier to access on a Quest https://creator.oculus.com/manage/mediastudio/
#@markdown 
#@markdown The command to get ffmpeg to concat your frames for each eye is in the form: `ffmpeg -framerate 15 -i frame_%4d_l.png l.mp4` (repeat for r)

vr_mode = False #@param {type:"boolean"}
#@markdown `vr_eye_angle` is the y-axis rotation of the eyes towards the center
vr_eye_angle = 0.5 #@param{type:"number"}
#@markdown interpupillary distance (between the eyes)
vr_ipd = 5.0 #@param{type:"number"}

#insist VR be used only w 3d anim.
if vr_mode and animation_mode != '3D':
    print('=====')
    print('VR mode only available with 3D animations. Disabling VR.')
    print('=====')
    vr_mode = False


def parse_key_frames(string, prompt_parser=None):
    """Given a string representing frame numbers paired with parameter values at that frame,
    return a dictionary with the frame numbers as keys and the parameter values as the values.

    Parameters
    ----------
    string: string
        Frame numbers paired with parameter values at that frame number, in the format
        'framenumber1: (parametervalues1), framenumber2: (parametervalues2), ...'
    prompt_parser: function or None, optional
        If provided, prompt_parser will be applied to each string of parameter values.
    
    Returns
    -------
    dict
        Frame numbers as keys, parameter values at that frame number as values

    Raises
    ------
    RuntimeError
        If the input string does not match the expected format.
    
    Examples
    --------
    >>> parse_key_frames("10:(Apple: 1| Orange: 0), 20: (Apple: 0| Orange: 1| Peach: 1)")
    {10: 'Apple: 1| Orange: 0', 20: 'Apple: 0| Orange: 1| Peach: 1'}

    >>> parse_key_frames("10:(Apple: 1| Orange: 0), 20: (Apple: 0| Orange: 1| Peach: 1)", prompt_parser=lambda x: x.lower()))
    {10: 'apple: 1| orange: 0', 20: 'apple: 0| orange: 1| peach: 1'}
    """
    import re
    pattern = r'((?P<frame>[0-9]+):[\s]*[\(](?P<param>[\S\s]*?)[\)])'
    frames = dict()
    for match_object in re.finditer(pattern, string):
        frame = int(match_object.groupdict()['frame'])
        param = match_object.groupdict()['param']
        if prompt_parser:
            frames[frame] = prompt_parser(param)
        else:
            frames[frame] = param

    if frames == {} and len(string) != 0:
        raise RuntimeError('Key Frame string not correctly formatted')
    return frames

def get_inbetweens(key_frames, integer=False):
    """Given a dict with frame numbers as keys and a parameter value as values,
    return a pandas Series containing the value of the parameter at every frame from 0 to max_frames.
    Any values not provided in the input dict are calculated by linear interpolation between
    the values of the previous and next provided frames. If there is no previous provided frame, then
    the value is equal to the value of the next provided frame, or if there is no next provided frame,
    then the value is equal to the value of the previous provided frame. If no frames are provided,
    all frame values are NaN.

    Parameters
    ----------
    key_frames: dict
        A dict with integer frame numbers as keys and numerical values of a particular parameter as values.
    integer: Bool, optional
        If True, the values of the output series are converted to integers.
        Otherwise, the values are floats.
    
    Returns
    -------
    pd.Series
        A Series with length max_frames representing the parameter values for each frame.
    
    Examples
    --------
    >>> max_frames = 5
    >>> get_inbetweens({1: 5, 3: 6})
    0    5.0
    1    5.0
    2    5.5
    3    6.0
    4    6.0
    dtype: float64

    >>> get_inbetweens({1: 5, 3: 6}, integer=True)
    0    5
    1    5
    2    5
    3    6
    4    6
    dtype: int64
    """
    key_frame_series = pd.Series([np.nan for a in range(max_frames)])

    for i, value in key_frames.items():
        key_frame_series[i] = value
    key_frame_series = key_frame_series.astype(float)
    
    interp_method = interp_spline

    if interp_method == 'Cubic' and len(key_frames.items()) <=3:
      interp_method = 'Quadratic'
    
    if interp_method == 'Quadratic' and len(key_frames.items()) <= 2:
      interp_method = 'Linear'
      
    
    key_frame_series[0] = key_frame_series[key_frame_series.first_valid_index()]
    key_frame_series[max_frames-1] = key_frame_series[key_frame_series.last_valid_index()]
    # key_frame_series = key_frame_series.interpolate(method=intrp_method,order=1, limit_direction='both')
    key_frame_series = key_frame_series.interpolate(method=interp_method.lower(),limit_direction='both')
    if integer:
        return key_frame_series.astype(int)
    return key_frame_series

def split_prompts(prompts):
    prompt_series = pd.Series([np.nan for a in range(max_frames)])
    for i, prompt in prompts.items():
        prompt_series[i] = prompt
    # prompt_series = prompt_series.astype(str)
    prompt_series = prompt_series.ffill().bfill()
    return prompt_series

if key_frames:
    try:
        angle_series = get_inbetweens(parse_key_frames(angle))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `angle` correctly for key frames.\n"
            "Attempting to interpret `angle` as "
            f'"0: ({angle})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        angle = f"0: ({angle})"
        angle_series = get_inbetweens(parse_key_frames(angle))

    try:
        zoom_series = get_inbetweens(parse_key_frames(zoom))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `zoom` correctly for key frames.\n"
            "Attempting to interpret `zoom` as "
            f'"0: ({zoom})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        zoom = f"0: ({zoom})"
        zoom_series = get_inbetweens(parse_key_frames(zoom))

    try:
        translation_x_series = get_inbetweens(parse_key_frames(translation_x))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `translation_x` correctly for key frames.\n"
            "Attempting to interpret `translation_x` as "
            f'"0: ({translation_x})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        translation_x = f"0: ({translation_x})"
        translation_x_series = get_inbetweens(parse_key_frames(translation_x))

    try:
        translation_y_series = get_inbetweens(parse_key_frames(translation_y))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `translation_y` correctly for key frames.\n"
            "Attempting to interpret `translation_y` as "
            f'"0: ({translation_y})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        translation_y = f"0: ({translation_y})"
        translation_y_series = get_inbetweens(parse_key_frames(translation_y))

    try:
        translation_z_series = get_inbetweens(parse_key_frames(translation_z))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `translation_z` correctly for key frames.\n"
            "Attempting to interpret `translation_z` as "
            f'"0: ({translation_z})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        translation_z = f"0: ({translation_z})"
        translation_z_series = get_inbetweens(parse_key_frames(translation_z))

    try:
        rotation_3d_x_series = get_inbetweens(parse_key_frames(rotation_3d_x))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `rotation_3d_x` correctly for key frames.\n"
            "Attempting to interpret `rotation_3d_x` as "
            f'"0: ({rotation_3d_x})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        rotation_3d_x = f"0: ({rotation_3d_x})"
        rotation_3d_x_series = get_inbetweens(parse_key_frames(rotation_3d_x))

    try:
        rotation_3d_y_series = get_inbetweens(parse_key_frames(rotation_3d_y))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `rotation_3d_y` correctly for key frames.\n"
            "Attempting to interpret `rotation_3d_y` as "
            f'"0: ({rotation_3d_y})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        rotation_3d_y = f"0: ({rotation_3d_y})"
        rotation_3d_y_series = get_inbetweens(parse_key_frames(rotation_3d_y))

    try:
        rotation_3d_z_series = get_inbetweens(parse_key_frames(rotation_3d_z))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `rotation_3d_z` correctly for key frames.\n"
            "Attempting to interpret `rotation_3d_z` as "
            f'"0: ({rotation_3d_z})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        rotation_3d_z = f"0: ({rotation_3d_z})"
        rotation_3d_z_series = get_inbetweens(parse_key_frames(rotation_3d_z))

else:
    angle = float(angle)
    zoom = float(zoom)
    translation_x = float(translation_x)
    translation_y = float(translation_y)
    translation_z = float(translation_z)
    rotation_3d_x = float(rotation_3d_x)
    rotation_3d_y = float(rotation_3d_y)
    rotation_3d_z = float(rotation_3d_z)

In [ ]:
#@title Install RAFT for Video input animation mode only
#@markdown Run once per session. Doesn't download again if model path exists.
#@markdown Use force download to reload raft models if needed
force_download = False #@param {type:'boolean'}
if animation_mode == 'Video Input':
    try:
        from raft import RAFT
    except:
        if not os.path.exists(os.path.join(PROJECT_DIR, 'RAFT')):
            gitclone('https://github.com/princeton-vl/RAFT', os.path.join(PROJECT_DIR, 'RAFT'))
        sys.path.append(f'{PROJECT_DIR}/RAFT')

    if (not (os.path.exists(f'{root_path}/RAFT/models'))) or force_download:
        createPath(f'{root_path}/RAFT')
        os.chdir(f'{root_path}/RAFT')
        sub_p_res = subprocess.run(['bash', f'{PROJECT_DIR}/RAFT/download_models.sh'], stdout=subprocess.PIPE).stdout.decode('utf-8')
        print(sub_p_res)
        os.chdir(PROJECT_DIR)

In [ ]:
#@title Define optical flow functions for Video input animation mode only
if animation_mode == 'Video Input':
    in_path = videoFramesFolder
    flo_folder = f'{in_path}/out_flo_fwd'
    path = f'{PROJECT_DIR}/RAFT/core'
    import sys
    sys.path.append(f'{PROJECT_DIR}/RAFT/core')
    os.chdir(f'{PROJECT_DIR}/RAFT/core')
    print(os.getcwd())
  
    print("Renaming RAFT core's utils.utils to raftutils.utils (to avoid a naming conflict with AdaBins)")
    if not os.path.exists(f'{PROJECT_DIR}/RAFT/core/raftutils'):
        os.rename(f'{PROJECT_DIR}/RAFT/core/utils', f'{PROJECT_DIR}/RAFT/core/raftutils')
        sub_p_res = subprocess.run(['sed', '-i', 's/from utils.utils/from raftutils.utils/g', f'{PROJECT_DIR}/RAFT/core/corr.py'], stdout=subprocess.PIPE).stdout.decode('utf-8')
        sub_p_res = subprocess.run(['sed', '-i', 's/from utils.utils/from raftutils.utils/g', f'{PROJECT_DIR}/RAFT/core/raft.py'], stdout=subprocess.PIPE).stdout.decode('utf-8')

    from raftutils.utils import InputPadder
    from raft import RAFT
    from raftutils import flow_viz
    import numpy as np
    import argparse, PIL, cv2
    from PIL import Image, ImageOps, ImageStat, ImageEnhance, ImageDraw
    from tqdm.notebook import tqdm
    from glob import glob
    import torch
  
    args2 = argparse.Namespace()
    args2.small = False
    args2.mixed_precision = True
  
  
    TAG_CHAR = np.array([202021.25], np.float32)
  
    def writeFlow(filename,uv,v=None):
        """ 
        https://github.com/NVIDIA/flownet2-pytorch/blob/master/utils/flow_utils.py
        Copyright 2017 NVIDIA CORPORATION
  
        Licensed under the Apache License, Version 2.0 (the "License");
        you may not use this file except in compliance with the License.
        You may obtain a copy of the License at
  
            http://www.apache.org/licenses/LICENSE-2.0
  
        Unless required by applicable law or agreed to in writing, software
        distributed under the License is distributed on an "AS IS" BASIS,
        WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
        See the License for the specific language governing permissions and
        limitations under the License.
        
        Write optical flow to file.
        
        If v is None, uv is assumed to contain both u and v channels,
        stacked in depth.
        Original code by Deqing Sun, adapted from Daniel Scharstein.
        """
        nBands = 2
  
        if v is None:
            assert(uv.ndim == 3)
            assert(uv.shape[2] == 2)
            u = uv[:,:,0]
            v = uv[:,:,1]
        else:
            u = uv
  
        assert(u.shape == v.shape)
        height,width = u.shape
        f = open(filename,'wb')
        # write the header
        f.write(TAG_CHAR)
        np.array(width).astype(np.int32).tofile(f)
        np.array(height).astype(np.int32).tofile(f)
        # arrange into matrix form
        tmp = np.zeros((height, width*nBands))
        tmp[:,np.arange(width)*2] = u
        tmp[:,np.arange(width)*2 + 1] = v
        tmp.astype(np.float32).tofile(f)
        f.close()
  
    def load_img(img, size):
        img = Image.open(img).convert('RGB').resize(size)
        return torch.from_numpy(np.array(img)).permute(2,0,1).float()[None,...].cuda()
  
    def get_flow(frame1, frame2, model, iters=20):
        padder = InputPadder(frame1.shape)
        frame1, frame2 = padder.pad(frame1, frame2)
        _, flow12 = model(frame1, frame2, iters=iters, test_mode=True)
        flow12 = flow12[0].permute(1, 2, 0).detach().cpu().numpy()
  
        return flow12
  
    def warp_flow(img, flow):
        h, w = flow.shape[:2]
        flow = flow.copy()
        flow[:, :, 0] += np.arange(w)
        flow[:, :, 1] += np.arange(h)[:, np.newaxis]
        res = cv2.remap(img, flow, None, cv2.INTER_LINEAR)
        return res
  
    def makeEven(_x):
        return _x if (_x % 2 == 0) else _x+1
  
    def fit(img,maxsize=512):
        maxdim = max(*img.size)
        if maxdim>maxsize:
            # if True:
            ratio = maxsize/maxdim
            x,y = img.size
            size = (makeEven(int(x*ratio)),makeEven(int(y*ratio))) 
            img = img.resize(size)
        return img
  
    def warp(frame1, frame2, flo_path, blend=0.5, weights_path=None):
        flow21 = np.load(flo_path)
        frame1pil = np.array(frame1.convert('RGB').resize((flow21.shape[1],flow21.shape[0])))
        frame1_warped21 = warp_flow(frame1pil, flow21)
        # frame2pil = frame1pil
        frame2pil = np.array(frame2.convert('RGB').resize((flow21.shape[1],flow21.shape[0])))
    
        if weights_path:
            # TBD
            pass
        else:
            blended_w = frame2pil*(1-blend) + frame1_warped21*(blend)
  
        return  PIL.Image.fromarray(blended_w.astype('uint8'))
  
    in_path = videoFramesFolder
    flo_folder = f'{in_path}/out_flo_fwd'
  
    temp_flo = in_path+'/temp_flo'
    flo_fwd_folder = in_path+'/out_flo_fwd'
    # TBD flow backwards!
  
    os.chdir(PROJECT_DIR)

In [ ]:
#@title Generate optical flow and consistency maps
#@markdown Run once per init video

if animation_mode == "Video Input":
    import gc

    force_flow_generation = False #@param {type:'boolean'}
    in_path = videoFramesFolder
    flo_folder = f'{in_path}/out_flo_fwd'

    if not video_init_flow_warp:
        print('video_init_flow_warp not set, skipping')

    if (animation_mode == 'Video Input') and (video_init_flow_warp):
        flows = glob(flo_folder+'/*.*')
        if (len(flows)>0) and not force_flow_generation:
            print(f'Skipping flow generation:\nFound {len(flows)} existing flow files in current working folder: {flo_folder}.\nIf you wish to generate new flow files, check force_flow_generation and run this cell again.')
    
        if (len(flows)==0) or force_flow_generation:
            frames = sorted(glob(in_path+'/*.*'));
            if len(frames)<2: 
                print(f'WARNING!\nCannot create flow maps: Found {len(frames)} frames extracted from your video input.\nPlease check your video path.')
            if len(frames)>=2:
        
                raft_model = torch.nn.DataParallel(RAFT(args2))
                raft_model.load_state_dict(torch.load(f'{root_path}/RAFT/models/raft-things.pth'))
                raft_model = raft_model.module.cuda().eval()
        
                for f in pathlib.Path(f'{flo_fwd_folder}').glob('*.*'):
                    f.unlink()
        
                temp_flo = in_path+'/temp_flo'
                flo_fwd_folder = in_path+'/out_flo_fwd'
        
                createPath(flo_fwd_folder)
                createPath(temp_flo)
        
                # TBD Call out to a consistency checker?
        
                framecount = 0
                for frame1, frame2 in tqdm(zip(frames[:-1], frames[1:]), total=len(frames)-1):
        
                    out_flow21_fn = f"{flo_fwd_folder}/{frame1.split('/')[-1]}"
            
                    frame1 = load_img(frame1, width_height)
                    frame2 = load_img(frame2, width_height)
            
                    flow21 = get_flow(frame2, frame1, raft_model)
                    np.save(out_flow21_fn, flow21)
            
                    if video_init_check_consistency:
                        # TBD
                        pass

                del raft_model 
                gc.collect()

### Extra Settings
#### **Partial Saves | Advanced Settings | Cutn Scheduling | Symmetry**

In [ ]:
#@markdown ###**Saving:**

intermediate_saves = [100,125,140]#@param{type: 'raw'}
intermediates_in_subfolder = False #@param{type: 'boolean'}
#@markdown Intermediate steps will save a copy at your specified intervals. You can either format it as a single integer or a list of specific steps 

#@markdown A value of `2` will save a copy at 33% and 66%. 0 will save none.

#@markdown A value of `[5, 9, 34, 45]` will save at steps 5, 9, 34, and 45. (Make sure to include the brackets)


if type(intermediate_saves) is not list:
    if intermediate_saves:
        steps_per_checkpoint = math.floor((steps - skip_steps - 1) // (intermediate_saves+1))
        steps_per_checkpoint = steps_per_checkpoint if steps_per_checkpoint > 0 else 1
        print(f'Will save every {steps_per_checkpoint} steps')
    else:
        steps_per_checkpoint = steps+10
else:
    steps_per_checkpoint = None

if intermediate_saves and intermediates_in_subfolder is True:
    partialFolder = f'{batchFolder}/partials'
    createPath(partialFolder)

#@markdown <br>

#@markdown ---

#@markdown ###**Advanced Settings:**
#@markdown *There are a few extra advanced settings available if you double click this cell.*

#@markdown *Perlin init will replace your init, so uncheck if using one.*

perlin_init = False  #@param{type: 'boolean'}
perlin_mode = 'mixed' #@param ['mixed', 'color', 'gray']
set_seed = 'random_seed' #@param{type: 'string'}
increment_n_batch_seed = True #@param {type: 'boolean'}
eta = 0.8 #@param{type: 'number'}
clamp_grad = True #@param{type: 'boolean'}
clamp_max = 0.05 #@param{type: 'number'}

#@markdown <br>

#@markdown ---

### EXTRA ADVANCED SETTINGS:
#@markdown #### **Advanced Extra Settings:**
randomize_class = True #@param{type: 'boolean'}
clip_denoised = False #@param{type: 'boolean'}
fuzzy_prompt = False #@param{type: 'boolean'}
rand_mag = 0.05 #@param{type: 'number'}

#@markdown <br>

#@markdown ---

#@markdown ### **Cutn Scheduling**

#@markdown To use easing, for cut_overview use the following format `[40]*100+dd_easing_fw(start=40, end=1, steps=800)+[1]*100`
#@markdown and for cut_innercut use the following format `[1]*100+dd_easing_bw(start=1, end=40, steps=800)+[40]*100`
#@markdown info on easing functions: https://github.com/semitable/easing-functions/blob/master/docs/cubic.png?raw=true
#@markdown more info on easing functions: https://github.com/semitable/easing-functions

def easeint(st, en, s, fn): return dd_easing_int(start=st, end=en, steps=s, easing_fn=fn)
def easefloat(st, en, s, fn): return dd_easing_float(start=st, end=en, steps=s, easing_fn=fn)

cut_easing_fn = "ExponentialEaseInOut" #@param ["LinearInOut", "QuadEaseInOut", "CubicEaseInOut", "QuarticEaseInOut", "QuinticEaseInOut", "SineEaseInOut", "CircularEaseInOut", "ExponentialEaseInOut", "ElasticEaseInOut", "BackEaseInOut", "BounceEaseInOut"]
cut_overview = "dd_easing_int(start=22, end=15, steps=250, easing_fn='ExponentialEaseIn')+dd_easing_int(start=15, end=0, steps=900, easing_fn='ExponentialEaseOut')+[0]*100" #@param {type: 'string'}
cut_overview_preset = "None" #@param ["None", "DEFAULT -> [22]*200+dd_easing_int(start=22, end=11, steps=350, easing_fn='ExponentialEaseIn')+dd_easing_int(start=11, end=0, steps=350, easing_fn='ExponentialEaseOut')+[0]*100", "DD DEFAULT -> [12]*400+[4]*600"]
cut_innercut = "[22-x for x in eval(cut_overview)]" #@param {type: 'string'}
cut_innercut_preset = "None" #@param ["None", "DEFAULT -> [22-x for x in eval(cut_overview)]", "DD DEFAULT -> [4]*400+[12]*600"]
cut_ic_pow = "[0.05]*500+dd_easing_float(start=0, end=2.5, steps=250, easing_fn=cut_easing_fn)+dd_easing_float(start=2.5, end=5, steps=250, easing_fn=cut_easing_fn)" #@param {type: 'string'}
cut_ic_pow_preset = "None" #@param ["None", "DEFAULT -> [0]*500+dd_easing_float(start=0, end=16, steps=500, easing_fn=cut_easing_fn)", "DD DEFAULT -> [1]*1000", "PORTRAIT DEFAULT -> [0.05]*1000"]
cut_icgray_p = "[0.2]*400+[0]*600" #@param {type: 'string'}
cut_icgray_p_preset = "None" #@param ["None", "DEFAULT -> [0.6]*100+[0.2]*100+[0.1]*100+[0.05]*100+[0]*600", "DD DEFAULT ->[0.2]*400+[0]*600", "[0.2]*400+[0.1]*600"]


# Overwrite settings with presets if selected
if cut_overview_preset != "None": cut_overview = cut_overview_preset.split(' -> ')[1] if ' -> ' in cut_overview_preset else cut_overview_preset
if cut_innercut_preset != "None": cut_innercut = cut_innercut_preset.split(' -> ')[1] if ' -> ' in cut_innercut_preset else cut_innercut_preset
if cut_ic_pow_preset != "None": cut_ic_pow = cut_ic_pow_preset.split(' -> ')[1] if ' -> ' in cut_ic_pow_preset else cut_ic_pow_preset
if cut_icgray_p_preset != "None": cut_icgray_p = cut_icgray_p.split(' -> ')[1] if ' -> ' in cut_icgray_p else cut_icgray_p


import numpy as np
import matplotlib.pyplot as plt
def easing_normalize(arr, t_min, t_max):
  norm_arr = []
  for i in arr:
    norm_arr.append((((i - min(arr))*(t_max - t_min))/(max(arr) - min(arr))) + t_min)
  return norm_arr

def dd_easing_int(start=1, end=9, steps=1000, easing_fn="ExponentialEaseInOut"): return list(np.rint(easing_normalize(list(map(eval(easing_fn + "(start=start, end=end, duration=steps)"), np.arange(0, steps, 1))), min([start, end]), max([start, end]))).astype(int))

def dd_easing_float(start=1, end=9, steps=1000, easing_fn="ExponentialEaseInOut"): return list(easing_normalize(list(map(eval(easing_fn + "(start=start, end=end, duration=steps)"), np.arange(0, steps, 1))), min([start, end]), max([start, end])))

plt.plot(np.arange(0, 1000, 1), eval(cut_overview)[0:1000], label='overview cuts')
plt.plot(np.arange(0, 1000, 1), eval(cut_innercut)[0:1000], label='inner cuts')

total_cuts = []
cut_overview_evaled = eval(cut_overview)
cut_innercut_evaled = eval(cut_innercut)
for i in np.arange(0, 1000, 1):
  total_cuts = total_cuts + [cut_overview_evaled[i] + cut_innercut_evaled[i]]

plt.plot(np.arange(0, 1000, 1), total_cuts[0:1000], label='total cuts')
plt.plot(np.arange(0, 1000, 1), eval(cut_ic_pow)[0:1000], label='ic_pow')
plt.legend(loc='best')
plt.show()

#@markdown <br>

#@markdown ---

#@markdown ### **KaliYuga Cutn Scheduling:**

#@markdown KaliYuga model settings. Refer to [cut_ic_pow](https://ezcharts.miraheze.org/wiki/Category:Cut_ic_pow) as a guide. Values between 1 and 100 all work.
pad_or_pulp_cut_overview = "[15]*100+[15]*100+[12]*100+[12]*100+[6]*100+[4]*100+[2]*200+[0]*200" #@param {type: 'string'}
pad_or_pulp_cut_innercut = "[1]*100+[1]*100+[4]*100+[4]*100+[8]*100+[8]*100+[10]*200+[10]*200" #@param {type: 'string'}
pad_or_pulp_cut_ic_pow = "[12]*300+[12]*100+[12]*50+[12]*50+[10]*100+[10]*100+[10]*300" #@param {type: 'string'}
pad_or_pulp_cut_icgray_p = "[0.87]*100+[0.78]*50+[0.73]*50+[0.64]*60+[0.56]*40+[0.50]*50+[0.33]*100+[0.19]*150+[0]*400" #@param {type: 'string'}

watercolor_cut_overview = "[14]*200+[12]*200+[4]*400+[0]*200" #@param {type: 'string'}
watercolor_cut_innercut = "[2]*200+[4]*200+[12]*400+[12]*200" #@param {type: 'string'}
watercolor_cut_ic_pow = "[12]*300+[12]*100+[12]*50+[12]*50+[10]*100+[10]*100+[10]*300" #@param {type: 'string'}
watercolor_cut_icgray_p = "[0.7]*100+[0.6]*100+[0.45]*100+[0.3]*100+[0]*600" #@param {type: 'string'}

if (diffusion_model in kaliyuga_pixel_art_model_names) or (diffusion_model in kaliyuga_pulpscifi_model_names):
    cut_overview = pad_or_pulp_cut_overview
    cut_innercut = pad_or_pulp_cut_innercut
    cut_ic_pow = pad_or_pulp_cut_ic_pow
    cut_icgray_p = pad_or_pulp_cut_icgray_p
elif diffusion_model in kaliyuga_watercolor_model_names:
    cut_overview = watercolor_cut_overview
    cut_innercut = watercolor_cut_innercut
    cut_ic_pow = watercolor_cut_ic_pow
    cut_icgray_p = watercolor_cut_icgray_p

#@markdown <br>

#@markdown ---

#@markdown ### **Symmetry Transformation Settings:**
use_vertical_symmetry = False #@param {type:"boolean"}
use_horizontal_symmetry = False #@param {type:"boolean"}
transformation_percent = [0.09] #@param

# 4. Prompt Setup

`animation_mode: None` will only use the first set. `animation_mode: 2D / Video` will run through them per the set frames and hold on to the last one.

For help with Noodle Soup Prompts see the <a href="#reference_guide">Reference section</a>

In [ ]:
# Note: If using a pixelart diffusion model, try adding "#pixelart" to the end of the prompt for a stronger effect. It'll tend to work a lot better!
# Note: Prompts now support scheduling. Ex: "My awesome Disco Diffusion Prompt:dd_easing_float(start=15, end=0, steps=400, easing_fn=cut_easing_fn)"


text_prompts = {
    
	0: [
			"_adj-beauty_ 3d matte portrait featuring a _noun-beauty_ woman by _artist_, _artist_, _site_:3", 
			"_gen-modifier_, _punk_, _details_, _3d-terms_, _hd_, _gen-modifier_, _punk_, _details_, _3d-terms_, _hd_, _style_:5",
			"_neg-weight_, _neg-weight_, _neg-weight_, _neg-weight_:-1"
	],

  # 100: ['This set of prompts runs at step 100'],

}

image_prompts = {
    # 0:['ImagePromptsWorkButArentVeryGood.png:2',],
}




### PROMPT PREVIEW CODE - No need to edit

#@title Preview Prompts
use_this_prompt = False #@param{type: 'boolean'}
#@markdown Use this random prompt instead of generating randomly on diffusion run.<br>**Note:** The parsed random prompt will be saved to the settings file, not your noodle soup keys.
generate_nsp_at_runtime = True #@param{type: 'boolean'}
#@markdown Use random prompts for each `n_batch`. **Note:** this will disable `use_this_prompt`
new_prompts = None
if nsp: new_prompts = nsp_parse(text_prompts) 
if generate_nsp_at_runtime: print("\033[1mThe following text prompts are just an example. Generating prompts at runtime...\033[0m\n")
if new_prompts: import pprint; print(pprint.pformat(new_prompts))

# 5. Render Images / Frames

In [ ]:
#@title **Render Diffusion Still Image**
display_rate = 10 #@param{type: 'number'}
#@markdown ---
#@markdown ####**Batch Settings**

n_batches = 5 #@param{type: 'number'}
#@markdown **Note:** `n_batches` ignored with animation modes.<br>
#@markdown **Note:** `n_batches` must be `1` for Go BIG. 

if gpu_warning:
  print(gpu_warning+"\n")

if pil_warning:
  print(pil_warning+"\n")

if animation_mode == 'Video Input':
    steps = video_init_steps

#Update Model Settings
timestep_respacing = f'ddim{steps}'
diffusion_steps = (1000//steps)*steps if steps < 1000 else steps
if diffusion_model is not 'custom':
    model_config.update({
        'timestep_respacing': timestep_respacing,
        'diffusion_steps': diffusion_steps,
    })
else:
    custom_model.update({
        'timestep_respacing': timestep_respacing,
        'diffusion_steps': diffusion_steps,
    })
if go_big_diffusion_model is not 'Inherit':
    if go_big_diffusion_model is not 'custom':
        go_big_model_config.update({
            'timestep_respacing': timestep_respacing,
            'diffusion_steps': diffusion_steps,
        })
    else:
        custom_model.update({
            'timestep_respacing': timestep_respacing,
            'diffusion_steps': diffusion_steps,
        })

batch_size = 1 

def move_files(start_num, end_num, old_folder, new_folder):
    for i in range(start_num, end_num):
        old_file = old_folder + f'/{batch_name}({batchNum})_{i:04}.png'
        new_file = new_folder + f'/{batch_name}({batchNum})_{i:04}.png'
        os.rename(old_file, new_file)

#@markdown ---
#@markdown #### **Resume Render**

resume_run = False #@param{type: 'boolean'}
run_to_resume = 'latest' #@param{type: 'string'}
resume_from_frame = 'latest' #@param{type: 'string'}
retain_overwritten_frames = False #@param{type: 'boolean'}
if retain_overwritten_frames:
    retainFolder = f'{batchFolder}/retained'
    createPath(retainFolder)

#@markdown ---
#@markdown #### **Extra Save Settings**

save_metadata = False #@param{type: 'boolean'}
meta_author = "None" #@param{type: 'string'}
meta_copyright = "None" #@param{type: 'string'}
save_plot_graph = False#@param{type: 'boolean'}

if meta_author is not None and meta_author.lower() in ['','none']:
    meta_author = None
if meta_copyright is not None and meta_copyright.lower() in ['','none']:
    meta_copyright = None

# Parse Noodle Soup Prompts
text_prompts = new_prompts if (use_this_prompt == True and generate_nsp_at_runtime == False) else text_prompts


skip_step_ratio = int(frames_skip_steps.rstrip("%")) / 100
calc_frames_skip_steps = math.floor(steps * skip_step_ratio)

if animation_mode == 'Video Input':
    frames = sorted(glob(in_path+'/*.*'));
    if len(frames)==0: 
        sys.exit("ERROR: 0 frames found.\nPlease check your video input path and rerun the video settings cell.")
    flows = glob(flo_folder+'/*.*')
    if (len(flows)==0) and video_init_flow_warp:
        sys.exit("ERROR: 0 flow files found.\nPlease rerun the flow generation cell.")

if steps <= calc_frames_skip_steps:
    sys.exit("ERROR: You can't skip more steps than your total steps")

if resume_run:
    if run_to_resume == 'latest':
        try:
            batchNum
        except:
            batchNum = len(glob(f"{batchFolder}/{batch_name}(*)_settings.txt"))-1
    else:
        batchNum = int(run_to_resume)
    if resume_from_frame == 'latest':
        start_frame = len(glob(batchFolder+f"/{batch_name}({batchNum})_*.png"))
        if animation_mode != '3D' and turbo_mode == True and start_frame > turbo_preroll and start_frame % int(turbo_steps) != 0:
            start_frame = start_frame - (start_frame % int(turbo_steps))
    else:
        start_frame = int(resume_from_frame)+1
        if animation_mode != '3D' and turbo_mode == True and start_frame > turbo_preroll and start_frame % int(turbo_steps) != 0:
            start_frame = start_frame - (start_frame % int(turbo_steps))
        if retain_overwritten_frames is True:
            existing_frames = len(glob(batchFolder+f"/{batch_name}({batchNum})_*.png"))
            frames_to_save = existing_frames - start_frame
            print(f'Moving {frames_to_save} frames to the Retained folder')
            move_files(start_frame, existing_frames, batchFolder, retainFolder)
else:
    start_frame = 0
    batchNum = len(glob(batchFolder+"/*.txt"))
    while os.path.isfile(f"{batchFolder}/{batch_name}({batchNum})_settings.txt") or os.path.isfile(f"{batchFolder}/{batch_name}-{batchNum}_settings.txt"):
        batchNum += 1

if go_big:
    print(f'Starting Run: {batch_name}({batchNum}) at frame {start_frame} with Go BIG!')
else:
    print(f'Starting Run: {batch_name}({batchNum}) at frame {start_frame}')

if set_seed == 'random_seed':
    random.seed()
    seed = random.randint(0, 2**32)
    # print(f'Using seed: {seed}')
else:
    seed = int(set_seed)

# LOAD SETTINGS - Implemented by WASasquatch
if settings_file_path:

    import json
    from os.path import exists

    if exists(settings_file_path):

        with open(settings_file_path) as json_file:

            jsd = json.load(json_file)

            if jsd:
                print('Loading diffusion settings from file: '+settings_file_path)
            if load_prompts_and_inits:
                print('Loading text, image, and resolution prompt settings from file: '+settings_file_path)

            if jsd.__contains__('clip_guidance_scale'):
                clip_guidance_scale = jsd['clip_guidance_scale']
            if jsd.__contains__('tv_scale'):
                tv_scale = jsd['tv_scale']
            if jsd.__contains__('range_scale'):
                range_scale =  jsd['range_scale']
            if jsd.__contains__('sat_scale'):
                sat_scale = jsd['sat_scale']
            if jsd.__contains__('cutn_batches'):
                cutn_batches = jsd['cutn_batches']
            if jsd.__contains__('interp_spline'):
                interp_spline = jsd['interp_spline']
            if jsd.__contains__('randomize_class'):
                randomize_class = jsd['randomize_class']
            if jsd.__contains__('clip_denoised'):
                clip_denoised = jsd['clip_denoised']
            if jsd.__contains__('clamp_gras'):
                clamp_grad = jsd['clamp_gras']
            if jsd.__contains__('clamp_max'):
                clamp_max = jsd['clamp_max']
            if jsd.__contains__('fuzzy_prompt'):
                fuzzy_prompt = jsd['fuzzy_prompt']
            if jsd.__contains__('max_frames'):
                max_frames = jsd['max_frames']
            if jsd.__contains__('rand_mag'):
                rand_mag = jsd['rand_mag']
            if jsd.__contains__('eta'):
                eta = jsd['eta']
            if jsd.__contains__('steps'):  
                steps = jsd['steps']
            if jsd.__contains__('diffusion_steps'):
                diffusion_steps = jsd['diffusion_steps']
            if jsd.__contains__('diffusion_sampling_mode'):
                diffusion_sampling_mode = jsd['diffusion_sampling_mode']
            if jsd.__contains__('cut_overview'):
                cut_overview = jsd['cut_overview']
            if jsd.__contains__('cut_innercut'):
                cut_innercut = jsd['cut_innercut']
            if jsd.__contains__('cut_ic_pow'):
                cut_ic_pow = jsd['cut_ic_pow']
            if jsd.__contains__('cut_icgray_p'):
                cut_icgray_p = jsd['cut_icgray_p']
            if jsd.__contains__('key_frames'):
                key_frames = jsd['key_frames']
            if jsd.__contains__('angle'):
                angle = jsd['angle']
            if jsd.__contains__('zoom'):
                zoom = jsd['zoom']
            if jsd.__contains__('translation_x'):
                translation_x = jsd['translation_x']
            if jsd.__contains__('translation_y'):
                translation_y = jsd['translation_y']
            if jsd.__contains__('translation_z'):
                translation_z = jsd['translation_z']
            if jsd.__contains__('translation_3d_x'):
                translation_3d_x = jsd['translation_3d_x']
            if jsd.__contains__('translation_3d_y'):
                translation_3d_y = jsd['translation_3d_y']
            if jsd.__contains__('translation_3d_z'):
                translation_3d_z = jsd['translation_3d_z']
            if jsd.__contains__('midas_depth_model'):
                midas_depth_model = jsd['midas_depth_model']
            if jsd.__contains__('midas_weight'):
                midas_weight = jsd['midas_weight']
            if jsd.__contains__('near_plane'):
                near_plane = jsd['near_plane']
            if jsd.__contains__('far_plane'):
                far_plane = jsd['far_plane']
            if jsd.__contains__('fov'):
                fov = jsd['fov']
            if jsd.__contains__('padding_mode'):
                padding_mode = jsd['padding_mode']
            if jsd.__contains__('sampling_mode'):
                sampling_mode = jsd['sampling_mode']
            if jsd.__contains__('extract_nth_frame'):
                extract_nth_frame = jsd['extract_nth_frame']
            if jsd.__contains__('turbo_mode'):
                turbo_mode = jsd['turbo_mode']
            if jsd.__contains__('turbo_steps'):
                turbo_steps = jsd['turbo_steps']
            if jsd.__contains__('turbo_preroll'):
                turbo_preroll = jsd['turbo_preroll']
            if jsd.__contains__('use_horizontal_symmetry'):
                use_horizontal_symmetry = jsd['use_horizontal_symmetry']
            if jsd.__contains__('use_vertical_symmetry'):
                use_vertical_symmetry = jsd['use_vertical_symmetry']
            if jsd.__contains__('transformation_percent'):
                transformation_percent = jsd['transformation_percent']

            if load_prompts_and_inits:

                if jsd.__contains__('text_prompts'):
                    text_prompts = jsd['text_prompts']
                if jsd.__contains__('generate_nsp_at_runtime'):
                    text_prompts = jsd['generate_nsp_at_runtime']
                if jsd.__contains__('image_prompts'):
                    image_prompts = jsd['image_prompts']

                if jsd.__contains__('init_image'):
                    if jsd['init_image'].lower() not in ['none', '']:
                        init_image = jsd['init_image']
                if jsd.__contains__('init_scale'):
                    init_scale = jsd['init_scale']
                if jsd.__contains__('skip_steps'):
                    skip_steps = jsd['skip_steps']
                if jsd.__contains__('do_prep_init_image'):
                    do_prep_init_image = jsd['do_prep_init_image']
                if jsd.__contains__('do_go_big_slice_prep'):
                    do_go_big_slice_prep = jsd['do_go_big_slice_prep']
                if jsd.__contains__('gaussian_blur_radius'):
                    gassian_blur_radius = jsd['gaussian_blur_radius']
                if jsd.__contains__('gaussian_noise_amount'):
                    gaussian_noise_amount = jsd['gaussian_noise_amount']
                if jsd.__contains__('gaussian_noise_blur_radius'):
                    gaussian_noise_blur_radius = jsd['gaussian_noise_blur_radius']
                if jsd.__contains__('gaussian_noise_greyscale'):
                    gaussian_noise_greyscale = jsd['gaussian_noise_greyscale']
                if jsd.__contains__('gaussian_noise_double'):
                    gaussian_noise_double = jsd['gaussian_noise_double']

                if jsd.__contains__('go_big'):
                    go_big = jsd['go_big']
                if jsd.__contains__('go_big_scale'):
                    go_big_scale = jsd['go_big_scale']
                if jsd.__contains__('go_big_skip_percent'):
                    go_big_skip_percent = jsd['go_big_skip_percent']
                if jsd.__contains__('gobig_overlap'):
                    gobig_overlap = jsd['gobig_overlap']
                if jsd.__contains__('gobig_maximize'):
                    gobig_maximize = jsd['gobig_maximize']
                if jsd.__contains__('go_big_init'):
                    if jsd['go_big_init'].lower() not in ['none', '']:
                        go_big_init = jsd['go_big_init']
                if jsd.__contains__('go_big_init_skipresize'):
                    go_big_init_skipresize = jsd['go_big_init_skipresize']
                if jsd.__contains__('init_masked'):
                    if jsd['init_masked'].lower() not in ['none', '']:
                        init_masked = jsd['init_masked']
                if jsd.__contains__('render_mask'):
                    if jsd['render_mask'].lower() not in ['none', '']:
                        render_mask = jsd['render_mask']

                if jsd.__contains__('frames_scale'):
                    frames_scale = jsd['frames_scale']
                if jsd.__contains__('frames_skip_steps'):
                    frames_skip_steps = jsd['frames_skip_steps']
                    skip_step_ratio = int(frames_skip_steps.rstrip("%")) / 100
                    calc_frames_skip_steps = math.floor(steps * skip_step_ratio)
                if jsd.__contains__('perlin_init'):
                    perlin_init = jsd['perlin_init']
                if jsd.__contains__('perlin_mode'):
                    perlin_mode = jsd['perlin_mode']
                if jsd.__contains__('width') and jsd.__contains__('height'):
                    width_height = [jsd['width'], jsd['height']]
                    side_x = jsd['width']
                    side_y = jsd['height']
                if jsd.__contains__('video_init_seed_continuity'):
                    video_init_seed_continuity = jsd['video_init_seed_continuity']
                if jsd.__contains__('video_init_steps'):
                    video_init_steps = jsd['video_init_steps']
                if jsd.__contains__('video_init_clip_guidance_scale'):
                    video_init_clip_guidance_scale = jsd['video_init_clip_guidance_scale']
                if jsd.__contains__('video_init_tv_scale'):
                    video_init_tv_scale = jsd['video_init_tv_scale']
                if jsd.__contains__('video_init_range_scale'):
                    video_init_range_scale = jsd['video_init_range_scale']
                if jsd.__contains__('video_init_sat_scale'):
                    video_init_sat_scale = jsd['video_init_sat_scale']
                if jsd.__contains__('video_init_cutn_batches'):
                    video_init_cutn_batches = jsd['video_init_cutn_batches']
                if jsd.__contains__('video_init_skip_steps'):
                    video_init_skip_steps = jsd['video_init_skip_steps']
                if jsd.__contains__('video_init_frames_scale'):
                    video_init_frames_scale = jsd['video_init_frames_scale']
                if jsd.__contains__('video_init_frames_skip_steps'):
                    video_init_frames_skip_steps = jsd['video_init_frames_skip_steps']
                if jsd.__contains__('video_init_flow_warp'):
                    video_init_flow_warp = jsd['video_init_flow_warp']
                if jsd.__contains__('video_init_flow_blend'):
                    video_init_flow_blend = jsd['video_init_flow_blend']
                if jsd.__contains__('video_init_check_consistency'):
                    video_init_check_consistency = jsd['video_init_check_consistency']
                if jsd.__contains__('video_init_blend_mode'):
                    video_init_blend_mode = jsd['video_init_blend_mode']

# n_batches patch for Go BIG
if go_big and n_batches > 1:
  print("\33[33m\u001b[4mWARNING:\33[0m n_batches is above 1, but Go BIG is active! Changing n_batches to 1...")
  n_batches = 1
  import time
  time.sleep(10)

args = {
    'batchNum': batchNum,
    'generate_nsp_at_runtime': generate_nsp_at_runtime,
    'prompts_series':split_prompts(text_prompts) if text_prompts else None,
    'image_prompts_series':split_prompts(image_prompts) if image_prompts else None,
    'seed': seed,
    'display_rate':display_rate,
    'n_batches':n_batches if animation_mode == 'None' else 1,
    'batch_size':batch_size,
    'batch_name': batch_name,
    'steps': steps,
    'diffusion_sampling_mode': diffusion_sampling_mode,
    'width_height': width_height,
    'clip_guidance_scale': clip_guidance_scale,
    'tv_scale': tv_scale,
    'range_scale': range_scale,
    'sat_scale': sat_scale,
    'cutn_batches': cutn_batches,
    'init_image': init_image,
    'init_scale': init_scale,
    'skip_steps': skip_steps,
    'do_prep_init_image': do_prep_init_image,
    'do_go_big_slice_prep': do_go_big_slice_prep,
    'gaussian_blur_radius': gaussian_blur_radius,
    'gaussian_noise_amount': gaussian_noise_amount,
    'gaussian_noise_blur_radius': gaussian_noise_blur_radius,
    'gaussian_noise_greyscale': gaussian_noise_greyscale,
    'gaussian_noise_double': gaussian_noise_double,
    'go_big': go_big,
    'go_big_scale': go_big_scale,
    'gobig_skip_percent': gobig_skip_percent,
    'gobig_overlap': gobig_overlap,
    'gobig_maximize': gobig_maximize,
    'go_big_init': go_big_init,
    'go_big_init_skipresize': go_big_init_skipresize,
    'init_masked': init_masked,
    'render_mask': render_mask,
    'side_x': side_x,
    'side_y': side_y,
    'timestep_respacing': timestep_respacing,
    'diffusion_steps': diffusion_steps,
    'animation_mode': animation_mode,
    'video_init_path': video_init_path,
    'extract_nth_frame': extract_nth_frame,
    'video_init_seed_continuity': video_init_seed_continuity,
    'key_frames': key_frames,
    'max_frames': max_frames if animation_mode != "None" else 1,
    'interp_spline': interp_spline,
    'start_frame': start_frame,
    'angle': angle,
    'zoom': zoom,
    'translation_x': translation_x,
    'translation_y': translation_y,
    'translation_z': translation_z,
    'rotation_3d_x': rotation_3d_x,
    'rotation_3d_y': rotation_3d_y,
    'rotation_3d_z': rotation_3d_z,
    'midas_depth_model': midas_depth_model,
    'midas_weight': midas_weight,
    'near_plane': near_plane,
    'far_plane': far_plane,
    'fov': fov,
    'padding_mode': padding_mode,
    'sampling_mode': sampling_mode,
    'angle_series':angle_series,
    'zoom_series':zoom_series,
    'translation_x_series':translation_x_series,
    'translation_y_series':translation_y_series,
    'translation_z_series':translation_z_series,
    'rotation_3d_x_series':rotation_3d_x_series,
    'rotation_3d_y_series':rotation_3d_y_series,
    'rotation_3d_z_series':rotation_3d_z_series,
    'frames_scale': frames_scale,
    'skip_step_ratio': skip_step_ratio,
    'calc_frames_skip_steps': calc_frames_skip_steps,
    'text_prompts': text_prompts,
    'image_prompts': image_prompts,
    'cut_overview': eval(cut_overview),
    'cut_innercut': eval(cut_innercut),
    'cut_ic_pow': eval(cut_ic_pow),
    'cut_icgray_p': eval(cut_icgray_p),
    'intermediate_saves': intermediate_saves,
    'intermediates_in_subfolder': intermediates_in_subfolder,
    'steps_per_checkpoint': steps_per_checkpoint,
    'perlin_init': perlin_init,
    'perlin_mode': perlin_mode,
    'set_seed': set_seed,
    'eta': eta,
    'clamp_grad': clamp_grad,
    'clamp_max': clamp_max,
    'skip_augs': skip_augs,
    'randomize_class': randomize_class,
    'clip_denoised': clip_denoised,
    'fuzzy_prompt': fuzzy_prompt,
    'rand_mag': rand_mag,
    'turbo_mode':turbo_mode,
    'turbo_steps':turbo_steps,
    'turbo_preroll':turbo_preroll,
    'use_vertical_symmetry': use_vertical_symmetry,
    'use_horizontal_symmetry': use_horizontal_symmetry,
    'transformation_percent': transformation_percent,
    'video_init_steps': video_init_steps,
    'video_init_clip_guidance_scale': video_init_clip_guidance_scale,
    'video_init_tv_scale': video_init_tv_scale,
    'video_init_range_scale': video_init_range_scale,
    'video_init_sat_scale': video_init_sat_scale,
    'video_init_cutn_batches': video_init_cutn_batches,
    'video_init_skip_steps': video_init_skip_steps,
    'video_init_frames_scale': video_init_frames_scale,
    'video_init_frames_skip_steps': video_init_frames_skip_steps,
    'video_init_flow_warp':video_init_flow_warp,
    'video_init_flow_blend':video_init_flow_blend,
    'video_init_check_consistency':video_init_check_consistency,
    'video_init_blend_mode':video_init_blend_mode,
    'save_metadata': save_metadata,
    'meta_author': meta_author,
    'meta_copyright': meta_copyright,
}

if animation_mode == 'Video Input':
    # This isn't great in terms of what will get saved to the settings.. but it should work.
    args['steps'] = args['video_init_steps']
    args['clip_guidance_scale'] = args['video_init_clip_guidance_scale']
    args['tv_scale'] = args['video_init_tv_scale']
    args['range_scale'] = args['video_init_range_scale']
    args['sat_scale'] = args['video_init_sat_scale']
    args['cutn_batches'] = args['video_init_cutn_batches']
    args['skip_steps'] = args['video_init_skip_steps']
    args['frames_scale'] = args['video_init_frames_scale']
    args['frames_skip_steps'] = args['video_init_frames_skip_steps']

args = SimpleNamespace(**args)

# Standard Run Setup

if go_big_init == None:

    print('Preparing model...')
    if diffusion_model == 'custom':
          model, diffusion = create_model_and_diffusion(**custom_model)
          print(f'Using model: {os.path.basename(custom_path)}')
          model.load_state_dict(torch.load(custom_path, map_location='cpu'))
    else:
          model, diffusion = create_model_and_diffusion(**model_config)
          print(f'Using model: {get_model_filename(diffusion_model)}')
          model.load_state_dict(torch.load(f'{model_path}/{get_model_filename(diffusion_model)}', map_location='cpu'))
    model.requires_grad_(False).eval().to(device)
    for name, param in model.named_parameters():
          if 'qkv' in name or 'norm' in name or 'proj' in name:
              param.requires_grad_()
    if model_config['use_fp16']:
          model.convert_to_fp16()


# FUNCTIONS FOR GO BIG MODE
def addalpha(im, mask):
    imr, img, imb, ima = im.split()
    mmr, mmg, mmb, mma = mask.split()
    im = Image.merge('RGBA', [imr, img, imb, mma])  # we want the RGB from the original, but the transparency from the mask
    return(im)

# Alternative method composites a grid of images at the positions provided
def grid_merge(source, slices):
    source.convert("RGBA")
    for slice, posx, posy in slices: # go in reverse to get proper stacking
      #print(posx, posy)
      source.alpha_composite(slice, (posx, posy))
    return source

def grid_coords(target, original, overlap):
    #generate a list of coordinate tuples for our sections, in order of how they'll be rendered
    #target should be the size for the gobig result, original is the size of each chunk being rendered
    center = []
    target_x, target_y = target
    center_x = int(target_x / 2)
    center_y = int(target_y / 2)
    original_x, original_y = original
    x = center_x - int(original_x / 2)
    y = center_y - int(original_y / 2)
    center.append((x,y)) #center chunk
    uy = y #up
    uy_list = []
    dy = y #down
    dy_list = []
    lx = x #left
    lx_list = []
    rx = x #right
    rx_list = []
    while uy > 0: #center row vertical up
        uy = uy - original_y + overlap
        uy_list.append((lx, uy))
    while (dy + original_y) <= target_y: #center row vertical down
        dy = dy + original_y - overlap
        dy_list.append((rx, dy))
    while lx > 0:
        lx = lx - original_x + overlap
        lx_list.append((lx, y))
        uy = y
        while uy > 0:
            uy = uy - original_y + overlap
            uy_list.append((lx, uy))
        dy = y
        while (dy + original_y) <= target_y:
            dy = dy + original_y - overlap
            dy_list.append((lx, dy))
    while (rx + original_x) <= target_x:
        rx = rx + original_x - overlap
        rx_list.append((rx, y))
        uy = y
        while uy > 0:
            uy = uy - original_y + overlap
            uy_list.append((rx, uy))
        dy = y
        while (dy + original_y) <= target_y:
            dy = dy + original_y - overlap
            dy_list.append((rx, dy))
    # calculate a new size that will fill the canvas, which will be optionally used in grid_slice and go_big
    last_coordx, last_coordy = dy_list[-1:][0]
    render_edgey = last_coordy + original_y # outer edge of the render canvas
    render_edgex = last_coordx + original_x # outer edge of the render canvas
    scalarx = render_edgex / target_x
    scalary = render_edgey / target_y
    if scalarx <= scalary:
        new_edgex = int(target_x * scalarx)
        new_edgey = int(target_y * scalarx)
    else:
        new_edgex = int(target_x * scalary)
        new_edgey = int(target_y * scalary)
    # now put all the chunks into one master list of coordinates (essentially reverse of how we calculated them so that the central slices will be on top)
    result = []
    for coords in dy_list[::-1]:
        result.append(coords)
    for coords in uy_list[::-1]:
        result.append(coords)
    for coords in rx_list[::-1]:
        result.append(coords)
    for coords in lx_list[::-1]:
        result.append(coords)
    result.append(center[0])
    return result, (new_edgex, new_edgey)

# Alternative method uses a grid of images that each equal the size of the original render
def grid_slice(source, overlap, og_size, maximize=False): 
    width, height = og_size # size of the slices to be rendered
    coordinates, new_size = grid_coords(source.size, og_size, overlap)
    if maximize == True:
        source = source.resize(new_size, get_resampling_mode()) # minor concern that we're resizing twice
        coordinates, new_size = grid_coords(source.size, og_size, overlap) # re-do the coordinates with the new canvas size
    # loc_width and loc_height are the center point of the goal size, and we'll start there and work our way out
    slices = []
    for coordinate in coordinates:
        x, y = coordinate
        slices.append(((source.crop((x, y, x+width, y+height))), x, y))
    global slices_todo
    slices_todo = len(slices) - 1
    return slices, new_size

import traceback

try:

  if go_big_init is not None:

    progress_image = 'progress.png'
    try:
      go_big_temp_image = Image.open(fetch(go_big_init)).convert("RGBA")
    except AttributeError as e:
        if "'PIL.TiffTags' has no attribute 'IFD'" in str(e):
          print("\33[33m\u001b[4mWARNING:\33[0m \033[1mPIL.TiffTags requires a restart. Please select Runtime -> Restart Runtime, and then run all the cells again.\033[0m")
          raise SystemExit
    go_big_temp_image.save(progress_image)
    go_big_temp_image.close()
    #shutil.copy(go_big_init, progress_image)

  else:

    do_run()

  if go_big:

    current_time = datetime.now().strftime('%y%m%d-%H%M%S_%f')
    # Resize initial progress.png to new size, setup some filenames
    original_output_image = (f'{batchFolder}/{batch_name}_original_output_{current_time}.png')
    final_output_image = (f'{batchFolder}/{batch_name}_final_output_{current_time}.png')
    progress_image = 'progress.png'
    input_image = Image.open(fetch(progress_image)).convert('RGBA')
    input_image.save(original_output_image)
    slice_image = (f'{batchFolder}/{batch_name}_slice.png')
    slice_rmask = (f'{batchFolder}/{batch_name}_slice_rmask.png')
    slice_imask = (f'{batchFolder}/{batch_name}_slice_imask.png')
    if render_mask is not None:
        source_render_mask = Image.open(render_mask).convert('RGBA')
    else:
        source_render_mask = None
    if go_big_init_skipresize == False:
      og_size = (side_x, side_y)
      reside_x = side_x * go_big_scale
      reside_y = side_y * go_big_scale
      source_image = input_image.resize((reside_x, reside_y), get_resampling_mode())
      if source_render_mask is not None:
        source_render_mask = source_render_mask((reside_x, reside_y), get_resampling_mode())
    else:
      source_image = input_image
      og_size = (int(side_x / go_big_scale), int(side_y / go_big_scale)) 
    input_image.close()
    if init_masked is not None:
        source_imask = Image.open(init_masked).convert('RGBA')
        source_imask = source_imask.resize(source_image.size, get_resampling_mode())
    else:
        source_imask = None
    # Slice source_image into overlapping slices
    #print(f'debug: before slicing, slices_todo is {slices_todo}')
    slices, new_canvas_size = grid_slice(source_image, gobig_overlap, og_size, gobig_maximize)
    if gobig_maximize == True:
      source_image = source_image.resize(new_canvas_size, get_resampling_mode())
      slices, new_canvas_size = grid_slice(source_image, gobig_overlap, og_size)
      print(f'Go BIG maximize is on. New size is now {source_image.size}')
    if source_render_mask is not None:
        rmasks = grid_slice(source_render_mask, og_size)
    else:
        rmasks = None
    if source_imask is not None:
        imasks = grid_slice(source_imask, gobig_overlap, og_size)
    else:
        imasks = None
    # Render each slice, with init image paramaters, etc.
    betterslices = []
    i = 1
    seed_list = []
    init_seed = seed
    for count, chunk_w_coords in enumerate(slices):
        current_slice = count
        chunk, coord_x, coord_y = chunk_w_coords
        if rmasks is not None:
            chunk_rmask = rmasks[count][0]
        else:
            chunk_rmask = None
        if imasks is not None:
            chunk_imask = imasks[count][0]
        else:
            chunk_imask = None

        # Reset underlying systems for another run
        seed = seed + 1
        args.seed = seed
        seed_list.append(seed)

        # Go BIG Diffusion Model
        print('Preparing model for next run...')
        if go_big_diffusion_model != "Inherit":
            model, diffusion = create_model_and_diffusion(**go_big_model_config)
            if go_big_diffusion_model == 'custom':
                model, diffusion = create_model_and_diffusion(**custom_model)
                print(f'Using Custom Model: {os.path.basename(custom_path)}')
                model.load_state_dict(torch.load(custom_path, map_location='cpu'))
            else:
                model, diffusion = create_model_and_diffusion(**go_big_model_config)
                dm_name = get_model_filename(go_big_diffusion_model)
                print("Using Model:", dm_name)
                model.load_state_dict(torch.load(f'{model_path}/{dm_name}', map_location='cpu'))
        else:
            if diffusion_model == 'custom':
                model, diffusion = create_model_and_diffusion(**custom_model)
                print(f'Using Custom Model: {os.path.basename(custom_path)}')
                model.load_state_dict(torch.load(custom_path, map_location='cpu'))
            else:
                model, diffusion = create_model_and_diffusion(**model_config)
                dm_name = get_model_filename(diffusion_model)
                print("Using Model:", dm_name)
                model.load_state_dict(torch.load(f'{model_path}/{dm_name}', map_location='cpu'))
        model.requires_grad_(False).eval().to(device)
        for name, param in model.named_parameters():
            if 'qkv' in name or 'norm' in name or 'proj' in name:
                param.requires_grad_()
        if model_config['use_fp16']:
            model.convert_to_fp16()
        gc.collect()
        if "cuda" in str(device):
          with torch.cuda.device(device):
            torch.cuda.empty_cache()

        #now do the next run
        chunk.save(slice_image)
        if chunk_rmask is not None:
            chunk_rmask.save(slice_rmask)
        if chunk_imask is not None:
            chunk_imask.save(slice_imask)
        args.init_image = slice_image
        init_image = slice_image
        if chunk_rmask is not None:
            args.render_mask = slice_rmask
            render_mask = slice_rmask
        if chunk_imask is not None:
            init_masked = slice_imask
            args.init_masked = slice_imask
        args.init_image = slice_image
        args.skip_steps = int(steps * (gobig_skip_percent / 100))
        args.side_x, args.side_y = chunk.size
        args.perlin_init = False
        perlin_init = False
        args.symmetry_loss = False
        do_run()
        print(f'Finished run, grabbing {progress_image} and adding it to slices...')
        resultslice = Image.open(progress_image).convert('RGBA')
        betterslices.append((resultslice.copy(), coord_x, coord_y))
        resultslice.save(f'{batchFolder}/{batch_name}_slice_output_{current_time}_{i}.png')
        resultslice.close() # hopefully this will allow subsequent images to actually save.
        i += 1
    # For each slice, use addalpha to add an alpha mask
    alpha_gradient = Image.new('L', (args.side_x, 1), color=0xFF)
    a = 0
    for x in range(args.side_x):
        a +=4 # add 4 to alpha at each pixel, to give us a 64 pixel overlap gradient
        if a < 255:
            alpha_gradient.putpixel((x, 0), a)
        else:
            alpha_gradient.putpixel((x, 0), 255)
        # print '{}, {:.2f}, {}'.format(x, float(x) / width, a)
    # generate an alpha mask for compositing the chunks
    alpha = Image.new('L', (args.side_x, args.side_y), color=0xFF)
    alpha_gradient = ImageDraw.Draw(alpha)
    a = 0
    i = 0
    overlap = gobig_overlap
    shape = ((args.side_x, args.side_y), (0,0))
    while i < overlap:
        alpha_gradient.rectangle(shape, fill = a)
        a += 4
        i += 1
        shape = ((args.side_x - i, args.side_y - i), (i,i))
    mask = Image.new('RGBA', (args.side_x, args.side_y), color=0)
    mask.putalpha(alpha)
    finished_slices = []
    for betterslice, x, y in betterslices:
        finished_slice = addalpha(betterslice, mask)
        finished_slices.append((finished_slice, x, y))

    # Once we have all our images, mergeimgs back onto source.png, then save
    final_output = grid_merge(source_image, finished_slices)
    final_output.save(final_output_image)
    print(f'\n\n\033[32mGo BIG is Complete!\33[0m\n\n\33[33m\u001b[4mNote:\33[0m Your output is saved as {final_output_image}!')

    
  if go_big:
    print(f"\nInitial Render Seed: {init_seed}\nSlice Seeds:")
    pprint(seed_list, indent=4)
  else:
    print('Base Seed:', seed)

except RuntimeError as e:

  if 'out of memory' in str(e):
    print(f"\u001b[31m\u001b[1m\u001b[4mCRITICAL ERROR\u001b[0m: {gpu_name} ran out of memory! Flushing memory to save session...")
    if gpu_warning:
      print(gpu_warning)
    print('\nSeed used for this failed run:', seed)
    pass
  else:
    raise e

except KeyboardInterrupt as e:

  print('\33[33mExecution interrupted by user.\33[0m')
  print('\nSeed used for this exited run:', seed)
  pass

#except Exception as e:
#
#  print(f"\n\33[31mAn exxception has occured:\33[0m\n {str(e)}\n")
#
#  if gpu_warning:
#    print(gpu_warning+"\n")
#
#  errors = io.StringIO()
#  traceback.print_exc(file=errors)
#  contents = str(errors.getvalue())
#  print(contents)
#  errors.close()
#  
#  print("\nAttempting to continue...")
#
#  pass

finally:

    print("\n:recycling_symbol: Cleaning up...")
    current_slice = None
    go_big_prompts = None
    runtime_prompts = []
    gc.collect()
    torch.cuda.empty_cache()
    print(":check_mark_button: Cleanup complete.")

In [ ]:
import PIL
# @title ### **Render Video Frame Sequence**
#@markdown Video file will save in the same folder as your images.
from tqdm.notebook import trange
skip_video_for_run_all = True #@param {type: 'boolean'}

if animation_mode == 'Video Input':
    frames = sorted(glob(in_path+'/*.*'));
    if len(frames)==0: 
        sys.exit("ERROR: 0 frames found.\nPlease check your video input path and rerun the video settings cell.")
    flows = glob(flo_folder+'/*.*')
    if (len(flows)==0) and video_init_flow_warp:
        sys.exit("ERROR: 0 flow files found.\nPlease rerun the flow generation cell.")

blend =  0.5#@param {type: 'number'}
video_init_check_consistency = False #@param {type: 'boolean'}
if skip_video_for_run_all == True:
    print('Skipping video creation, uncheck skip_video_for_run_all if you want to run it')

else:
    # import subprocess in case this cell is run without the above cells
    import subprocess
    from base64 import b64encode

    latest_run = batchNum

    folder = batch_name #@param
    run = latest_run #@param
    final_frame = 'final_frame'


    init_frame = 1#@param {type:"number"} This is the frame where the video will start
    last_frame = final_frame#@param {type:"number"} You can change i to the number of the last frame you want to generate. It will raise an error if that number of frames does not exist.
    fps = 12#@param {type:"number"}
    # view_video_in_cell = True #@param {type: 'boolean'}

    frames = []
    # tqdm.write('Generating video...')

    if last_frame == 'final_frame':
        last_frame = len(glob(batchFolder+f"/{folder}({run})_*.png"))
        print(f'Total frames: {last_frame}')

    image_path = f"{outDirPath}/{folder}/{folder}({run})_%04d.png"
    filepath = f"{outDirPath}/{folder}/{folder}({run}).mp4"

    if (video_init_blend_mode == 'optical flow') and (animation_mode == 'Video Input'):
        image_path = f"{outDirPath}/{folder}/flow/{folder}({run})_%04d.png"
        filepath = f"{outDirPath}/{folder}/{folder}({run})_flow.mp4"
        if last_frame == 'final_frame':
            last_frame = len(glob(batchFolder+f"/flow/{folder}({run})_*.png"))
        flo_out = batchFolder+f"/flow"
        createPath(flo_out)
        frames_in = sorted(glob(batchFolder+f"/{folder}({run})_*.png"))
        shutil.copy(frames_in[0], flo_out)
        for i in trange(init_frame, min(len(frames_in), last_frame)):
            frame1_path = frames_in[i-1]
            frame2_path = frames_in[i]
  
            frame1 = PIL.Image.open(frame1_path)
            frame2 = PIL.Image.open(frame2_path)
            frame1_stem = f"{(int(frame1_path.split('/')[-1].split('_')[-1][:-4])+1):04}.jpg"
            flo_path = f"/{flo_folder}/{frame1_stem}.npy"
            weights_path = None
            if video_init_check_consistency:
                # TBD
                pass
            warp(frame1, frame2, flo_path, blend=blend, weights_path=weights_path).save(batchFolder+f"/flow/{folder}({run})_{i:04}.png")
    if video_init_blend_mode == 'linear':
        image_path = f"{outDirPath}/{folder}/blend/{folder}({run})_%04d.png"
        filepath = f"{outDirPath}/{folder}/{folder}({run})_blend.mp4"
        if last_frame == 'final_frame':
            last_frame = len(glob(batchFolder+f"/blend/{folder}({run})_*.png"))
        blend_out = batchFolder+f"/blend"
        createPath(blend_out)
        frames_in = glob(batchFolder+f"/{folder}({run})_*.png")
        shutil.copy(frames_in[0], blend_out)
        for i in trange(1, len(frames_in)):
            frame1_path = frames_in[i-1]
            frame2_path = frames_in[i]
    
            frame1 = PIL.Image.open(frame1_path)
            frame2 = PIL.Image.open(frame2_path)
          
            frame = PIL.Image.fromarray((np.array(frame1)*(1-blend) + np.array(frame2)*(blend)).astype('uint8')).save(batchFolder+f"/blend/{folder}({run})_{i:04}.png")


    cmd = [
        'ffmpeg',
        '-y',
        '-vcodec',
        'png',
        '-r',
        str(fps),
        '-start_number',
        str(init_frame),
        '-i',
        image_path,
        '-frames:v',
        str(last_frame+1),
        '-c:v',
        'libx264',
        '-vf',
        f'fps={fps}',
        '-pix_fmt',
        'yuv420p',
        '-crf',
        '17',
        '-preset',
        'veryslow',
        filepath
    ]

    process = subprocess.Popen(cmd, cwd=f'{batchFolder}', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(stderr, defaultprint=True)
        raise RuntimeError(stderr)
    else:
        print("The video is ready and saved to the images folder")

    # if view_video_in_cell:
    #     mp4 = open(filepath,'rb').read()
    #     data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    #     display.HTML(f'<video width=400 controls><source src="{data_url}" type="video/mp4"></video>')